<a href="https://colab.research.google.com/github/vparro/MLprobabilistico/blob/master/Paper_accidenets_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Bibliotecas**

In [ ]:
%%capture
# instala biblioteca para leitura de excel e biblioteca gráfica
!pip install xlrd
!pip install -U plotly
# !pip install plotly==4.8.1

# Exportando figuras com melhores resoluções
!pip install -U kaleido

In [ ]:
# Importa a biblioteca pandas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.options.plotting.backend = 'plotly'

# Importa biblioteca para conversão de datas
from datetime import datetime

# Bilbioteca gráfica utilizada neste estudo
import plotly.express as px
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# biblioteca numérica
import scipy.stats as stats
import scipy
from sklearn.metrics import r2_score
import statsmodels.api as sm

In [ ]:
# Diretório de imagens
import os
if not os.path.exists('images'):
    os.mkdir('images')

# if not os.path.exists('/content/drive/MyDrive/Trabalho/KLB/images'):
#     os.mkdir('/content/drive/MyDrive/Trabalho/KLB/images')

# **Funções**

In [ ]:
def graphics_plot_prf(y, y_name, yaxes_title, title='Title', bins=1,
                      x_tick=1, save=False, save_name=None):
    '''
    y = [y1, y2, y3, y4]
    y_name = ['y1 name', 'y2 name', 'y3 name', 'y4 name']
    bins = number of bins for x scale
    title = 'Title'
    yaxes_title = ['Title yaxes1', 'Title yaxes2', 'Title yaxes3', 'Title yaxes4']
    x_tick = dtick for axis x
    save = True -> save graphics as image (images/title.eps)
    '''

    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{'secondary_y': True}]])

    # Markers
    # markers = ['circle','star','diamond','x']

    # Lines
    dash = ['dash','dot','dashdot']

    for i, v in enumerate(y):
        if (len(yaxes_title) == 1) | (i < len(y) - 1):
            on = False
        else:
            on = True

        if i == 0:
            # Add traces
            fig.add_trace(go.Scatter(x=np.linspace(1, 24, bins),
                                     y=v, name=y_name[i],
                                     # marker_symbol=markers[i],
                                     # marker_size=7,
                                     mode='lines',
                                     line=dict(color='rgb(0, 0, 0)',
                                               # shape='spline',
                                               width=2)
                                     ),
                          secondary_y=False)

        else:
            # Add traces
            fig.add_trace(go.Scatter(x=np.linspace(1, 24, bins),
                                     y=v, name=y_name[i],
                                     # marker_symbol=markers[i],
                                     # marker_size=7,
                                     mode='lines',
                                     line=dict(color='rgb(0, 0, 0)',
                                               width=2,
                                               # shape='spline',
                                               dash=dash[i-1])
                                     ),
                          secondary_y=on)

    # Add figure title
    fig.update_layout(title_text=title)

    # Set x-axis title
    fig.update_xaxes(title_text='Time of day')

    # Set y-axes titles
    if len(yaxes_title) > 1:
        fig.update_yaxes(title_text='<b>'+ yaxes_title[0] +'</b>', secondary_y=False)
        fig.update_yaxes(title_text='<b>'+ yaxes_title[1] +'</b>', secondary_y=True)
    else:
        fig.update_yaxes(title_text='<b>'+ yaxes_title[0] +'</b>', secondary_y=False)

    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='black')
    fig.update_yaxes(showgrid=False, gridwidth=1, gridcolor='black')

    fig.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
    fig.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=True)

    fig.update_layout(paper_bgcolor='rgba(0,0,0,0)',
                      plot_bgcolor='rgba(0,0,0,0)')

    fig.update_layout(xaxis=dict(tickmode='linear', tick0=0, dtick=x_tick),
                      legend=dict(orientation='h', yanchor='bottom', y=1.01,
                                  xanchor='right', x=0.95))
    fig.show()

    if save:
        fig.write_image('images/'+ save_name +'.png')

        # fig.write_image('/content/drive/MyDrive/Trabalho/KLB/images/'+ save_name +'.png')

In [ ]:
def show_R2_pvalue(x, y):

    # adicionar uma constante para a regressão linear
    x = sm.add_constant(x)

    # ajustar um modelo de regressão linear
    modelo = sm.OLS(y, x).fit()

    # extrair o R² e o p-value e imprimir os resultados
    print(f'R²: {modelo.rsquared:.5f}')
    print(f'p-value: {modelo.f_pvalue:.5e}')

# **Primeiras análises**

## **Dados Fluxo**

In [ ]:
%%capture
!gdown 1Kru8e3M6AjhEDuVXH7Tqx36wZjz4-Xc0

In [ ]:
# Fluxo
dfFluxo = pd.read_csv('DNIT.csv')

# dfFluxo['avgtotal'].loc[2::3]   ## [start:stop:step] - se ficar livre vai até o final
dfDNIT = pd.DataFrame(dfFluxo['avgtotal'].loc[2::3].values.reshape(4,24)).T
dfDNIT.tail(3)

,0,1,2,3
21,217.674901,172.611702,164.695324,160.408976
22,183.319164,143.476845,136.172760,132.808623
23,131.100690,104.440947,97.717366,94.281760


## **Estradas federais - Geral**

**2013**

In [ ]:
%%capture
!gdown 13TZAmFfbKhEYuIbYIMOW1NMISGXP0JyO

In [ ]:
df13 = pd.read_csv('datatran2013.csv', encoding = 'ISO-8859-1', sep=';')
df13['Hora'] = pd.to_datetime(df13['horario'], format='%H:%M:%S').dt.hour
df13.shape

In [ ]:
df13.head(2)

In [ ]:
counts, bins = np.histogram(df13['Hora'], bins=range(0, 25, 1))
acidentes13 = np.array(counts)/sum(np.array(counts))
fluxo13 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa13 = (acidentes13/fluxo13)/sum(acidentes13/fluxo13)

In [ ]:
graphics_plot_prf(y=[taxa13, fluxo13, acidentes13],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='2013 - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='geral_2013')

In [ ]:
# DataFrame com as informações das taxas
df_taxas = pd.DataFrame(data=range(1,25), columns=['hora'])
df_taxas['geral_13'] = taxa13
df_taxas.head()

**2014**

In [ ]:
%%capture
!gdown 13VRBoeuJpEOc-5iFBovLvZCP8RFFVaml

In [ ]:
df14 = pd.read_csv('datatran2014.csv', encoding = 'ISO-8859-1', sep=';')
df14['Hora'] = pd.to_datetime(df14['horario'], format='%H:%M:%S').dt.hour
df14.shape

In [ ]:
df14.head(2)

In [ ]:
counts, bins = np.histogram(df14['Hora'], bins=range(0, 25, 1))
acidentes14 = np.array(counts)/sum(np.array(counts))
fluxo14 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa14 = (acidentes14/fluxo14)/sum(acidentes14/fluxo14)

In [ ]:
graphics_plot_prf(y=[taxa14, fluxo14, acidentes14],
                  y_name=['2014 - Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='geral_2014')

**2015**

In [ ]:
%%capture
!gdown 13f3pn98iHub-p5hN1Hzho49Xo9UHp96N

In [ ]:
df15 = pd.read_csv('datatran2015.csv', encoding = 'ISO-8859-1', sep=';')
df15['Hora'] = pd.to_datetime(df15['horario'], format='%H:%M:%S').dt.hour
df15.shape

In [ ]:
df15.head(2)

In [ ]:
counts, bins = np.histogram(df15['Hora'], bins=range(0, 25, 1))
acidentes15 = np.array(counts)/sum(np.array(counts))
fluxo15 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa15 = (acidentes15/fluxo15)/sum(acidentes15/fluxo15)

In [ ]:
graphics_plot_prf(y=[taxa15, fluxo15, acidentes15],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='2015 - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='geral_2015')

**2016**

In [ ]:
%%capture
!gdown 13Zc1iIdYWnQ2oQSn8o6CcxZHwrtn6jcu

In [ ]:
df16 = pd.read_csv('datatran2016.csv', encoding = 'ISO-8859-1', sep=';')
df16['Hora'] = pd.to_datetime(df16['horario'], format='%H:%M:%S').dt.hour
df16.shape

In [ ]:
df16.head(2)

In [ ]:
counts, bins = np.histogram(df16['Hora'], bins=range(0, 25, 1))
acidentes16 = np.array(counts)/sum(np.array(counts))
fluxo16 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa16 = (acidentes16/fluxo16)/sum(acidentes16/fluxo16)

In [ ]:
graphics_plot_prf(y=[taxa16, fluxo16, acidentes16],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='2016 - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='geral_2016')

**2017**

In [ ]:
%%capture
!gdown 143SYBlyOq2CMO1ySbIvDFkMn9PW8Vi7d

In [ ]:
df17 = pd.read_csv('datatran2017.csv', encoding = 'ISO-8859-1', sep=';')
df17['Hora'] = pd.to_datetime(df17['horario'], format='%H:%M:%S').dt.hour
df17.shape

In [ ]:
df17.head(2)

In [ ]:
counts, bins = np.histogram(df17['Hora'], bins=range(0, 25, 1))
acidentes17 = np.array(counts)/sum(np.array(counts))
fluxo17 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa17 = (acidentes17/fluxo17)/sum(acidentes17/fluxo17)

In [ ]:
graphics_plot_prf(y=[taxa17, fluxo17, acidentes17],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='2017 - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='geral_2017')

In [ ]:
df_taxas['geral_14'] = taxa14
df_taxas['geral_15'] = taxa15
df_taxas['geral_16'] = taxa16
df_taxas['geral_17'] = taxa17

## **Dados para um único veículo**

Filtros para tipo de pista + Caminhões + Capotamento ou Tombamento


In [ ]:
%%capture
!gdown 1rLrFckRbr_oScvziSCodjC2_UxoEOAMk # PRF 2013 - 1 veículo
!gdown 1--JAT8gfd9CBtIlNBBj5yB_C5ipq9p0u # PRF 2014 - 1 veículo
!gdown 1-4vkJOgGW9W__P9zFhbVQkT3Uarwmbbv # PRF 2015 - 1 veículo
!gdown 1-6cj1H9JsUtdIZtO78x3HhdSFjpsqPj_ # PRF 2016 - 1 veículo
!gdown 1-6tc2J-omr-39k9n3FWApTqnJrZFIbnp # PRF 2017 - 1 veículo

In [ ]:
# Datasets já filtrados para veículo único
df_prf13 = pd.read_csv('prf_2013_1veiculo.csv')
df_prf13['Hora'] = pd.to_datetime(df_prf13['horario'], format='%H:%M:%S').dt.hour
df_prf14 = pd.read_csv('prf_2014_1veiculo.csv')
df_prf14['Hora'] = pd.to_datetime(df_prf14['horario'], format='%H:%M:%S').dt.hour
df_prf15 = pd.read_csv('prf_2015_1veiculo.csv')
df_prf15['Hora'] = pd.to_datetime(df_prf15['horario'], format='%H:%M:%S').dt.hour
df_prf16 = pd.read_csv('prf_2016_1veiculo.csv')
df_prf16['Hora'] = pd.to_datetime(df_prf16['horario'], format='%H:%M:%S').dt.hour
df_prf17 = pd.read_csv('prf_2017_1veiculo.csv')
df_prf17['Hora'] = pd.to_datetime(df_prf17['horario'], format='%H:%M:%S').dt.hour

**2013**

In [ ]:
# Filtragem Caminhões + 'Tombamento' ou 'Capotamento'
list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
caminhao = df_prf13.tipo_veiculo.isin(list_caminhao)

list_acid = ['Capotamento', 'Tombamento']
capot_tomb = df_prf13.tipo_acidente.isin(list_acid)

prf13 = df_prf13[(caminhao) & (capot_tomb)]

In [ ]:
prf13.tipo_veiculo.unique()

In [ ]:
prf13.tipo_acidente.unique()

In [ ]:
prf13.tipo_pista.unique()

In [ ]:
# 2013 somente com pista 'Simples' e 'Dupla'
df13_simples_dupla = prf13[prf13.tipo_pista.isin(['Simples', 'Dupla'])]
df13_simples_dupla.tipo_pista.unique()

In [ ]:
# 2013 somente com pista 'Simples'
df13_simples = prf13[prf13.tipo_pista.isin(['Simples'])]
df13_simples.tipo_pista.unique()

In [ ]:
# 2013 somente com pista 'Dupla'
df13_dupla = prf13[prf13.tipo_pista.isin(['Dupla'])]
df13_dupla.tipo_pista.unique()

**2014**

In [ ]:
# Filtragem Caminhões + 'Tombamento' ou 'Capotamento'
list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
caminhao = df_prf14.tipo_veiculo.isin(list_caminhao)

list_acid = ['Capotamento', 'Tombamento']
capot_tomb = df_prf14.tipo_acidente.isin(list_acid)

prf14 = df_prf14[(caminhao) & (capot_tomb)]

In [ ]:
prf14.tipo_veiculo.unique()

In [ ]:
prf14.tipo_acidente.unique()

In [ ]:
prf14.tipo_pista.unique()

In [ ]:
# 2014 somente com pista 'Simples' e 'Dupla'
df14_simples_dupla = prf14[prf14.tipo_pista.isin(['Simples', 'Dupla'])]
df14_simples_dupla.tipo_pista.unique()

In [ ]:
# 2014 somente com pista 'Simples'
df14_simples = prf14[prf14.tipo_pista.isin(['Simples'])]
df14_simples.tipo_pista.unique()

In [ ]:
# 2014 somente com pista 'Dupla'
df14_dupla = prf14[prf14.tipo_pista.isin(['Dupla'])]
df14_dupla.tipo_pista.unique()

**2015**

In [ ]:
# Filtragem Caminhões + 'Tombamento' ou 'Capotamento'
list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
caminhao = df_prf15.tipo_veiculo.isin(list_caminhao)

list_acid = ['Capotamento', 'Tombamento']
capot_tomb = df_prf15.tipo_acidente.isin(list_acid)

prf15 = df_prf15[(caminhao) & (capot_tomb)]

In [ ]:
prf15.tipo_veiculo.unique()

In [ ]:
prf15.tipo_acidente.unique()

In [ ]:
prf15.tipo_pista.unique()

In [ ]:
# 2015 somente com pista 'Simples' e 'Dupla'
df15_simples_dupla = prf15[prf15.tipo_pista.isin(['Simples', 'Dupla'])]
df15_simples_dupla.tipo_pista.unique()

In [ ]:
# 2015 somente com pista 'Simples'
df15_simples = prf15[prf15.tipo_pista.isin(['Simples'])]
df15_simples.tipo_pista.unique()

In [ ]:
# 2015 somente com pista 'Dupla'
df15_dupla = prf15[prf15.tipo_pista.isin(['Dupla'])]
df15_dupla.tipo_pista.unique()

**2016**

In [ ]:
# Filtragem Caminhões + 'Tombamento' ou 'Capotamento'
list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
caminhao = df_prf16.tipo_veiculo.isin(list_caminhao)

list_acid = ['Capotamento', 'Tombamento']
capot_tomb = df_prf16.tipo_acidente.isin(list_acid)

prf16 = df_prf16[(caminhao) & (capot_tomb)]

In [ ]:
prf16.tipo_veiculo.unique()

In [ ]:
prf16.tipo_acidente.unique()

In [ ]:
prf16.tipo_pista.unique()

In [ ]:
# 2016 somente com pista 'Simples' e 'Dupla'
df16_simples_dupla = prf16[prf16.tipo_pista.isin(['Simples', 'Dupla'])]
df16_simples_dupla.tipo_pista.unique()

In [ ]:
# 2016 somente com pista 'Simples'
df16_simples = prf16[prf16.tipo_pista.isin(['Simples'])]
df16_simples.tipo_pista.unique()

In [ ]:
# 2016 somente com pista 'Dupla'
df16_dupla = prf16[prf16.tipo_pista.isin(['Dupla'])]
df16_dupla.tipo_pista.unique()

**2017**

In [ ]:
# Filtragem Caminhões + 'Tombamento' ou 'Capotamento'
list_caminhao = ['Caminhão', 'Caminhão-trator']
caminhao = df_prf17.tipo_veiculo.isin(list_caminhao)

list_acid = ['Capotamento', 'Tombamento']
capot_tomb = df_prf17.tipo_acidente.isin(list_acid)

prf17 = df_prf17[(caminhao) & (capot_tomb)]

In [ ]:
prf17.tipo_veiculo.unique()

In [ ]:
prf17.tipo_acidente.unique()

In [ ]:
prf17.tipo_pista.unique()

In [ ]:
# 2017 somente com pista 'Simples' e 'Dupla'
df17_simples_dupla = prf17[prf17.tipo_pista.isin(['Simples', 'Dupla'])]
df17_simples_dupla.tipo_pista.unique()

In [ ]:
# 2017 somente com pista 'Simples'
df17_simples = prf17[prf17.tipo_pista.isin(['Simples'])]
df17_simples.tipo_pista.unique()

In [ ]:
# 2017 somente com pista 'Dupla'
df17_dupla = prf17[prf17.tipo_pista.isin(['Dupla'])]
df17_dupla.tipo_pista.unique()

## **Estrada simples + dupla**

### **Um veículo + Reta**

**2013**

In [ ]:
df13_simples_dupla.tracado_via.unique()

In [ ]:
# Filtragem 'Reta'
reta = df13_simples_dupla.tracado_via == 'Reta'

df13_sd_reta = df13_simples_dupla[(reta)]

In [ ]:
df13_sd_reta.tracado_via.unique()

In [ ]:
counts, bins = np.histogram(df13_sd_reta['Hora'], bins=range(0, 25, 1))
acidentes13 = np.array(counts)/sum(np.array(counts))
fluxo13 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa13 = (acidentes13/fluxo13)/sum(acidentes13/fluxo13)

In [ ]:
graphics_plot_prf(y=[taxa13, fluxo13, acidentes13],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a straight section of the road (2013) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_dupla_reta_2013')

**2014**

In [ ]:
df14_simples_dupla.tracado_via.unique()

In [ ]:
# Filtragem 'Reta'
reta = df14_simples_dupla.tracado_via == 'Reta'

df14_sd_reta = df14_simples_dupla[(reta)]

In [ ]:
df14_sd_reta.tracado_via.unique()

In [ ]:
counts, bins = np.histogram(df14_sd_reta['Hora'], bins=range(0, 25, 1))
acidentes14 = np.array(counts)/sum(np.array(counts))
fluxo14 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa14 = (acidentes14/fluxo14)/sum(acidentes14/fluxo14)

In [ ]:
graphics_plot_prf(y=[taxa14, fluxo14, acidentes14],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a straight section of the road (2014) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_dupla_reta_2014')

**2015**

In [ ]:
df15_simples_dupla.tracado_via.unique()

In [ ]:
# Filtragem 'Reta'
reta = df15_simples_dupla.tracado_via == 'Reta'

df15_sd_reta = df15_simples_dupla[(reta)]

In [ ]:
df15_sd_reta.tracado_via.unique()

In [ ]:
counts, bins = np.histogram(df15_sd_reta['Hora'], bins=range(0, 25, 1))
acidentes15 = np.array(counts)/sum(np.array(counts))
fluxo15 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa15 = (acidentes15/fluxo15)/sum(acidentes15/fluxo15)

In [ ]:
graphics_plot_prf(y=[taxa15, fluxo15, acidentes15],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a straight section of the road (2015) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_dupla_reta_2015')

**2016**

In [ ]:
df16_simples_dupla.tracado_via.unique()

In [ ]:
# Filtragem 'Reta'
reta = df16_simples_dupla.tracado_via == 'Reta'

df16_sd_reta = df16_simples_dupla[(reta)]

In [ ]:
df16_sd_reta.tracado_via.unique()

In [ ]:
counts, bins = np.histogram(df16_sd_reta['Hora'], bins=range(0, 25, 1))
acidentes16 = np.array(counts)/sum(np.array(counts))
fluxo16 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa16 = (acidentes16/fluxo16)/sum(acidentes16/fluxo16)

In [ ]:
graphics_plot_prf(y=[taxa16, fluxo16, acidentes16],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a straight section of the road (2016) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_dupla_reta_2016')

**2017**

In [ ]:
df17_simples_dupla.tracado_via.unique()

In [ ]:
# Filtragem 'Reta'
reta = df17_simples_dupla.tracado_via == 'Reta'

df17_sd_reta = df17_simples_dupla[(reta)]

In [ ]:
df17_sd_reta.tracado_via.unique()

In [ ]:
counts, bins = np.histogram(df17_sd_reta['Hora'], bins=range(0, 25, 1))
acidentes17 = np.array(counts)/sum(np.array(counts))
fluxo17 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa17 = (acidentes17/fluxo17)/sum(acidentes17/fluxo17)

In [ ]:
graphics_plot_prf(y=[taxa17, fluxo17, acidentes17],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a straight section of the road (2017) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_dupla_reta_2017')

In [ ]:
# Salvando as taxas
df_taxas['simples_dupla_reta_13'] = taxa13
df_taxas['simples_dupla_reta_14'] = taxa14
df_taxas['simples_dupla_reta_15'] = taxa15
df_taxas['simples_dupla_reta_16'] = taxa16
df_taxas['simples_dupla_reta_17'] = taxa17

### **Um veículo + Reta + feridos**

**2013**

In [ ]:
df13_simples_dupla.tracado_via.unique()

In [ ]:
np.sort(df13_simples_dupla.feridos.unique())

In [ ]:
# Filtragem 'Reta' + pelo menos 1 ferido
reta = df13_simples_dupla.tracado_via == 'Reta'

feridos = df13_simples_dupla.feridos > 0

df13_sd_reta_feridos = df13_simples_dupla[(reta) & (feridos)]

In [ ]:
df13_sd_reta_feridos.tracado_via.unique()

In [ ]:
np.sort(df13_sd_reta_feridos.feridos.unique())

In [ ]:
counts, bins = np.histogram(df13_sd_reta_feridos['Hora'], bins=range(0, 25, 1))
acidentes13 = np.array(counts)/sum(np.array(counts))
fluxo13 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa13 = (acidentes13/fluxo13)/sum(acidentes13/fluxo13)

In [ ]:
graphics_plot_prf(y=[taxa13, fluxo13, acidentes13],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a straight section of the road with at least one person injured (2013) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_dupla_reta_feridos_2013')

**2014**

In [ ]:
df14_simples_dupla.tracado_via.unique()

In [ ]:
np.sort(df14_simples_dupla.feridos.unique())

In [ ]:
# Filtragem 'Reta' + pelo menos 1 ferido
reta = df14_simples_dupla.tracado_via == 'Reta'

feridos = df14_simples_dupla.feridos > 0

df14_sd_reta_feridos = df14_simples_dupla[(reta) & (feridos)]

In [ ]:
df14_sd_reta_feridos.tracado_via.unique()

In [ ]:
np.sort(df14_sd_reta_feridos.feridos.unique())

In [ ]:
counts, bins = np.histogram(df14_sd_reta_feridos['Hora'], bins=range(0, 25, 1))
acidentes14 = np.array(counts)/sum(np.array(counts))
fluxo14 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa14 = (acidentes14/fluxo14)/sum(acidentes14/fluxo14)

In [ ]:
graphics_plot_prf(y=[taxa14, fluxo14, acidentes14],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a straight section of the road with at least one person injured (2014) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_dupla_reta_feridos_2014')

**2015**

In [ ]:
df15_simples_dupla.tracado_via.unique()

In [ ]:
np.sort(df15_simples_dupla.feridos.unique())

In [ ]:
# Filtragem 'Reta' + pelo menos 1 ferido
reta = df15_simples_dupla.tracado_via == 'Reta'

feridos = df15_simples_dupla.feridos > 0

df15_sd_reta_feridos = df15_simples_dupla[(reta) & (feridos)]

In [ ]:
df15_sd_reta_feridos.tracado_via.unique()

In [ ]:
np.sort(df15_sd_reta_feridos.feridos.unique())

In [ ]:
counts, bins = np.histogram(df15_sd_reta_feridos['Hora'], bins=range(0, 25, 1))
acidentes15 = np.array(counts)/sum(np.array(counts))
fluxo15 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa15 = (acidentes15/fluxo15)/sum(acidentes15/fluxo15)

In [ ]:
graphics_plot_prf(y=[taxa15, fluxo15, acidentes15],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a straight section of the road with at least one person injured (2015) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_dupla_reta_feridos_2015')

**2016**

In [ ]:
df16_simples_dupla.tracado_via.unique()

In [ ]:
np.sort(df16_simples_dupla.feridos.unique())

In [ ]:
# Filtragem 'Reta' + pelo menos 1 ferido
reta = df16_simples_dupla.tracado_via == 'Reta'

feridos = df16_simples_dupla.feridos > 0

df16_sd_reta_feridos = df16_simples_dupla[(reta) & (feridos)]

In [ ]:
df16_sd_reta_feridos.tracado_via.unique()

In [ ]:
np.sort(df16_sd_reta_feridos.feridos.unique())

In [ ]:
counts, bins = np.histogram(df16_sd_reta_feridos['Hora'], bins=range(0, 25, 1))
acidentes16 = np.array(counts)/sum(np.array(counts))
fluxo16 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa16 = (acidentes16/fluxo16)/sum(acidentes16/fluxo16)

In [ ]:
graphics_plot_prf(y=[taxa16, fluxo16, acidentes16],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a straight section of the road with at least one person injured (2016) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_dupla_reta_feridos_2016')

**2017**

In [ ]:
df17_simples_dupla.tracado_via.unique()

In [ ]:
np.sort(df17_simples_dupla.feridos.unique())

In [ ]:
# Filtragem 'Reta' + pelo menos 1 ferido
reta = df17_simples_dupla.tracado_via == 'Reta'

feridos = df17_simples_dupla.feridos > 0

df17_sd_reta_feridos = df17_simples_dupla[(reta) & (feridos)] #& (caminhao) & (capot_tomb)]

In [ ]:
df17_sd_reta_feridos.tracado_via.unique()

In [ ]:
np.sort(df17_sd_reta_feridos.feridos.unique())

In [ ]:
counts, bins = np.histogram(df17_sd_reta_feridos['Hora'], bins=range(0, 25, 1))
acidentes17 = np.array(counts)/sum(np.array(counts))
fluxo17 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa17 = (acidentes17/fluxo17)/sum(acidentes17/fluxo17)

In [ ]:
graphics_plot_prf(y=[taxa17, fluxo17, acidentes17],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a straight section of the road with at least one person injured (2017) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_dupla_reta_feridos_2017')

In [ ]:
# Salvando as taxas
df_taxas['simples_dupla_reta_feridos_13'] = taxa13
df_taxas['simples_dupla_reta_feridos_14'] = taxa14
df_taxas['simples_dupla_reta_feridos_15'] = taxa15
df_taxas['simples_dupla_reta_feridos_16'] = taxa16
df_taxas['simples_dupla_reta_feridos_17'] = taxa17

### **Um veículo + Curva**

**2013**

In [ ]:
df13_simples_dupla.tracado_via.unique()

In [ ]:
# Filtragem 'Curva'
curva = df13_simples_dupla.tracado_via == 'Curva'

df13_sd_curva = df13_simples_dupla[(curva)]

In [ ]:
df13_sd_curva.tracado_via.unique()

In [ ]:
counts, bins = np.histogram(df13_sd_curva['Hora'], bins=range(0, 25, 1))
acidentes13 = np.array(counts)/sum(np.array(counts))
fluxo13 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa13 = (acidentes13/fluxo13)/sum(acidentes13/fluxo13)

In [ ]:
graphics_plot_prf(y=[taxa13, fluxo13, acidentes13],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a curve section of the road (2013) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_dupla_curva_2013')

**2014**

In [ ]:
df14_simples_dupla.tracado_via.unique()

In [ ]:
# Filtragem 'Curva'
curva = df14_simples_dupla.tracado_via == 'Curva'

df14_sd_curva = df14_simples_dupla[(curva)]

In [ ]:
df14_sd_curva.tracado_via.unique()

In [ ]:
counts, bins = np.histogram(df14_sd_curva['Hora'], bins=range(0, 25, 1))
acidentes14 = np.array(counts)/sum(np.array(counts))
fluxo14 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa14 = (acidentes14/fluxo14)/sum(acidentes14/fluxo14)

In [ ]:
graphics_plot_prf(y=[taxa14, fluxo14, acidentes14],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a curve section of the road (2014) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_dupla_curva_2014')

**2015**

In [ ]:
df15_simples_dupla.tracado_via.unique()

In [ ]:
# Filtragem 'Curva'
curva = df15_simples_dupla.tracado_via == 'Curva'

df15_sd_curva = df15_simples_dupla[(curva)]

In [ ]:
df15_sd_curva.tracado_via.unique()

In [ ]:
counts, bins = np.histogram(df15_sd_curva['Hora'], bins=range(0, 25, 1))
acidentes15 = np.array(counts)/sum(np.array(counts))
fluxo15 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa15 = (acidentes15/fluxo15)/sum(acidentes15/fluxo15)

In [ ]:
graphics_plot_prf(y=[taxa15, fluxo15, acidentes15],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a curve section of the road (2015) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_dupla_curva_2015')

**2016**

In [ ]:
df16_simples_dupla.tracado_via.unique()

In [ ]:
# Filtragem 'Curva'
curva = df16_simples_dupla.tracado_via == 'Curva'

df16_sd_curva = df16_simples_dupla[(curva)]

In [ ]:
df16_sd_curva.tracado_via.unique()

In [ ]:
counts, bins = np.histogram(df16_sd_curva['Hora'], bins=range(0, 25, 1))
acidentes16 = np.array(counts)/sum(np.array(counts))
fluxo16 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa16 = (acidentes16/fluxo16)/sum(acidentes16/fluxo16)

In [ ]:
graphics_plot_prf(y=[taxa16, fluxo16, acidentes16],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a curve section of the road (2016) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_dupla_curva_2016')

**2017**

In [ ]:
df17_simples_dupla.tracado_via.unique()

In [ ]:
# Filtragem 'Curva'
curva = df17_simples_dupla.tracado_via == 'Curva'

df17_sd_curva = df17_simples_dupla[(curva)]

In [ ]:
df17_sd_curva.tracado_via.unique()

In [ ]:
counts, bins = np.histogram(df17_sd_curva['Hora'], bins=range(0, 25, 1))
acidentes17 = np.array(counts)/sum(np.array(counts))
fluxo17 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa17 = (acidentes17/fluxo17)/sum(acidentes17/fluxo17)

In [ ]:
graphics_plot_prf(y=[taxa17, fluxo17, acidentes17],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a curve section of the road (2017) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_dupla_curva_2017')

In [ ]:
# Salvando as taxas
df_taxas['simples_dupla_curva_13'] = taxa13
df_taxas['simples_dupla_curva_14'] = taxa14
df_taxas['simples_dupla_curva_15'] = taxa15
df_taxas['simples_dupla_curva_16'] = taxa16
df_taxas['simples_dupla_curva_17'] = taxa17

### **Um veículo + Curva + feridos**

**2013**

In [ ]:
df13_simples_dupla.tracado_via.unique()

In [ ]:
np.sort(df13_simples_dupla.feridos.unique())

In [ ]:
# Filtragem 'Curva' + pelo menos 1 ferido
curva = df13_simples_dupla.tracado_via == 'Curva'

feridos = df13_simples_dupla.feridos > 0

df13_sd_curva_feridos = df13_simples_dupla[(curva) & (feridos)]

In [ ]:
df13_sd_curva_feridos.tracado_via.unique()

In [ ]:
np.sort(df13_sd_curva_feridos.feridos.unique())

In [ ]:
counts, bins = np.histogram(df13_sd_curva_feridos['Hora'], bins=range(0, 25, 1))
acidentes13 = np.array(counts)/sum(np.array(counts))
fluxo13 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa13 = (acidentes13/fluxo13)/sum(acidentes13/fluxo13)

In [ ]:
graphics_plot_prf(y=[taxa13, fluxo13, acidentes13],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a curve section of the road with at least one person injured (2013) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_dupla_curva_feridos_2013')

**2014**

In [ ]:
df14_simples_dupla.tracado_via.unique()

In [ ]:
np.sort(df14_simples_dupla.feridos.unique())

In [ ]:
# Filtragem 'Curva' + pelo menos 1 ferido
curva = df14_simples_dupla.tracado_via == 'Curva'

feridos = df14_simples_dupla.feridos > 0

df14_sd_curva_feridos = df14_simples_dupla[(curva) & (feridos)]

In [ ]:
df14_sd_curva_feridos.tracado_via.unique()

In [ ]:
np.sort(df14_sd_curva_feridos.feridos.unique())

In [ ]:
counts, bins = np.histogram(df14_sd_curva_feridos['Hora'], bins=range(0, 25, 1))
acidentes14 = np.array(counts)/sum(np.array(counts))
fluxo14 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa14 = (acidentes14/fluxo14)/sum(acidentes14/fluxo14)

In [ ]:
graphics_plot_prf(y=[taxa14, fluxo14, acidentes14],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a curve section of the road with at least one person injured (2014) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_dupla_curva_feridos_2014')

**2015**

In [ ]:
df15_simples_dupla.tracado_via.unique()

In [ ]:
np.sort(df15_simples_dupla.feridos.unique())

In [ ]:
# Filtragem 'Curva' + pelo menos 1 ferido
curva = df15_simples_dupla.tracado_via == 'Curva'

feridos = df15_simples_dupla.feridos > 0

df15_sd_curva_feridos = df15_simples_dupla[(curva) & (feridos)]

In [ ]:
df15_sd_curva_feridos.tracado_via.unique()

In [ ]:
np.sort(df15_sd_curva_feridos.feridos.unique())

In [ ]:
counts, bins = np.histogram(df15_sd_curva_feridos['Hora'], bins=range(0, 25, 1))
acidentes15 = np.array(counts)/sum(np.array(counts))
fluxo15 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa15 = (acidentes15/fluxo15)/sum(acidentes15/fluxo15)

In [ ]:
graphics_plot_prf(y=[taxa15, fluxo15, acidentes15],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a curve section of the road with at least one person injured (2015) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_dupla_curva_feridos_2015')

**2016**

In [ ]:
df16_simples_dupla.tracado_via.unique()

In [ ]:
np.sort(df16_simples_dupla.feridos.unique())

In [ ]:
# Filtragem 'Curva' + pelo menos 1 ferido
curva = df16_simples_dupla.tracado_via == 'Curva'

feridos = df16_simples_dupla.feridos > 0

df16_sd_curva_feridos = df16_simples_dupla[(curva) & (feridos)]

In [ ]:
df16_sd_curva_feridos.tracado_via.unique()

In [ ]:
np.sort(df16_sd_curva_feridos.feridos.unique())

In [ ]:
counts, bins = np.histogram(df16_sd_curva_feridos['Hora'], bins=range(0, 25, 1))
acidentes16 = np.array(counts)/sum(np.array(counts))
fluxo16 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa16 = (acidentes16/fluxo16)/sum(acidentes16/fluxo16)

In [ ]:
graphics_plot_prf(y=[taxa16, fluxo16, acidentes16],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a curve section of the road with at least one person injured (2016) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_dupla_curva_feridos_2016')

**2017**

In [ ]:
df17_simples_dupla.tracado_via.unique()

In [ ]:
np.sort(df17_simples_dupla.feridos.unique())

In [ ]:
# Filtragem 'Curva' + pelo menos 1 ferido
curva = df17_simples_dupla.tracado_via == 'Curva'

feridos = df17_simples_dupla.feridos > 0

df17_sd_curva_feridos = df17_simples_dupla[(curva) & (feridos)]

In [ ]:
df17_sd_curva_feridos.tracado_via.unique()

In [ ]:
np.sort(df17_sd_curva_feridos.feridos.unique())

In [ ]:
counts, bins = np.histogram(df17_sd_curva_feridos['Hora'], bins=range(0, 25, 1))
acidentes17 = np.array(counts)/sum(np.array(counts))
fluxo17 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa17 = (acidentes17/fluxo17)/sum(acidentes17/fluxo17)

In [ ]:
graphics_plot_prf(y=[taxa17, fluxo17, acidentes17],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a curve section of the road with at least one person injured (2017) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_dupla_curva_feridos_2017')

In [ ]:
# Salvando as taxas
df_taxas['simples_dupla_curva_feridos_13'] = taxa13
df_taxas['simples_dupla_curva_feridos_14'] = taxa14
df_taxas['simples_dupla_curva_feridos_15'] = taxa15
df_taxas['simples_dupla_curva_feridos_16'] = taxa16
df_taxas['simples_dupla_curva_feridos_17'] = taxa17

### **Um veículo + Reta + Curva**

**2013**

In [ ]:
df13_simples_dupla.tracado_via.unique()

In [ ]:
# Filtragem 'Reta' ou 'Curva'
reta_curva = df13_simples_dupla.tracado_via.isin(['Reta', 'Curva'])

df13_sd_reta_curva = df13_simples_dupla[(reta_curva)]

In [ ]:
df13_sd_reta_curva.tracado_via.unique()

In [ ]:
counts, bins = np.histogram(df13_sd_reta_curva['Hora'], bins=range(0, 25, 1))
acidentes13 = np.array(counts)/sum(np.array(counts))
fluxo13 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa13 = (acidentes13/fluxo13)/sum(acidentes13/fluxo13)

In [ ]:
graphics_plot_prf(y=[taxa13, fluxo13, acidentes13],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a straight or curve section of the road (2013) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_dupla_reta_curva_2013')

**2014**

In [ ]:
df14_simples_dupla.tracado_via.unique()

In [ ]:
# Filtragem 'Reta' ou 'Curva'
reta_curva = df14_simples_dupla.tracado_via.isin(['Reta', 'Curva'])

df14_sd_reta_curva = df14_simples_dupla[(reta_curva)]

In [ ]:
df14_sd_reta_curva.tracado_via.unique()

In [ ]:
counts, bins = np.histogram(df14_sd_reta_curva['Hora'], bins=range(0, 25, 1))
acidentes14 = np.array(counts)/sum(np.array(counts))
fluxo14 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa14 = (acidentes14/fluxo14)/sum(acidentes14/fluxo14)

In [ ]:
graphics_plot_prf(y=[taxa14, fluxo14, acidentes14],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a straight or curve section of the road (2014) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_dupla_reta_curva_2014')

**2015**

In [ ]:
df15_simples_dupla.tracado_via.unique()

In [ ]:
# Filtragem 'Reta' ou 'Curva'
reta_curva = df15_simples_dupla.tracado_via.isin(['Reta', 'Curva'])

df15_sd_reta_curva = df15_simples_dupla[(reta_curva)]

In [ ]:
df15_sd_reta_curva.tracado_via.unique()

In [ ]:
counts, bins = np.histogram(df15_sd_reta_curva['Hora'], bins=range(0, 25, 1))
acidentes15 = np.array(counts)/sum(np.array(counts))
fluxo15 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa15 = (acidentes15/fluxo15)/sum(acidentes15/fluxo15)

In [ ]:
graphics_plot_prf(y=[taxa15, fluxo15, acidentes15],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a straight or curve section of the road (2015) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_dupla_reta_curva_2015')

**2016**

In [ ]:
df16_simples_dupla.tracado_via.unique()

In [ ]:
# Filtragem 'Reta' ou 'Curva'
reta_curva = df16_simples_dupla.tracado_via.isin(['Reta', 'Curva'])

df16_sd_reta_curva = df16_simples_dupla[(reta_curva)]

In [ ]:
df16_sd_reta_curva.tracado_via.unique()

In [ ]:
counts, bins = np.histogram(df16_sd_reta_curva['Hora'], bins=range(0, 25, 1))
acidentes16 = np.array(counts)/sum(np.array(counts))
fluxo16 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa16 = (acidentes16/fluxo16)/sum(acidentes16/fluxo16)

In [ ]:
graphics_plot_prf(y=[taxa16, fluxo16, acidentes16],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a straight or curve section of the road (2016) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_dupla_reta_curva_2016')

**2017**

In [ ]:
df17_simples_dupla.tracado_via.unique()

In [ ]:
# Filtragem 'Reta' ou 'Curva'
reta_curva = df17_simples_dupla.tracado_via.isin(['Reta', 'Curva'])

df17_sd_reta_curva = df17_simples_dupla[(reta_curva)]

In [ ]:
df17_sd_reta_curva.tracado_via.unique()

In [ ]:
counts, bins = np.histogram(df17_sd_reta_curva['Hora'], bins=range(0, 25, 1))
acidentes17 = np.array(counts)/sum(np.array(counts))
fluxo17 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa17 = (acidentes17/fluxo17)/sum(acidentes17/fluxo17)

In [ ]:
graphics_plot_prf(y=[taxa17, fluxo17, acidentes17],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a straight or curve section of the road (2017) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_dupla_reta_curva_2017')

In [ ]:
# Salvando as taxas
df_taxas['simples_dupla_reta_curva_13'] = taxa13
df_taxas['simples_dupla_reta_curva_14'] = taxa14
df_taxas['simples_dupla_reta_curva_15'] = taxa15
df_taxas['simples_dupla_reta_curva_16'] = taxa16
df_taxas['simples_dupla_reta_curva_17'] = taxa17

### **Um veículo + Reta + Curva + feridos**

**2013**

In [ ]:
df13_simples_dupla.tracado_via.unique()

In [ ]:
np.sort(df13_simples_dupla.feridos.unique())

In [ ]:
# Filtragem 'Reta' ou 'Curva' + pelo menos 1 ferido
reta_curva = df13_simples_dupla.tracado_via.isin(['Reta', 'Curva'])

feridos = df13_simples_dupla.feridos > 0

df13_sd_reta_curva_feridos = df13_simples_dupla[(reta_curva) & (feridos)]

In [ ]:
df13_sd_reta_curva_feridos.tracado_via.unique()

In [ ]:
np.sort(df13_sd_reta_curva_feridos.feridos.unique())

In [ ]:
counts, bins = np.histogram(df13_sd_reta_curva_feridos['Hora'], bins=range(0, 25, 1))
acidentes13 = np.array(counts)/sum(np.array(counts))
fluxo13 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa13 = (acidentes13/fluxo13)/sum(acidentes13/fluxo13)

In [ ]:
graphics_plot_prf(y=[taxa13, fluxo13, acidentes13],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a straight or curve section of the road with at least one person injured (2013) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_dupla_reta_curva_feridos_2013')

**2014**

In [ ]:
df14_simples_dupla.tracado_via.unique()

In [ ]:
np.sort(df14_simples_dupla.feridos.unique())

In [ ]:
# Filtragem 'Reta' ou 'Curva' + pelo menos 1 ferido
reta_curva = df14_simples_dupla.tracado_via.isin(['Reta', 'Curva'])

feridos = df14_simples_dupla.feridos > 0

df14_sd_reta_curva_feridos = df14_simples_dupla[(reta_curva) & (feridos)]

In [ ]:
df14_sd_reta_curva_feridos.tracado_via.unique()

In [ ]:
np.sort(df14_sd_reta_curva_feridos.feridos.unique())

In [ ]:
counts, bins = np.histogram(df14_sd_reta_curva_feridos['Hora'], bins=range(0, 25, 1))
acidentes14 = np.array(counts)/sum(np.array(counts))
fluxo14 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa14 = (acidentes14/fluxo14)/sum(acidentes14/fluxo14)

In [ ]:
graphics_plot_prf(y=[taxa14, fluxo14, acidentes14],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a straight or curve section of the road with at least one person injured (2014) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_dupla_reta_curva_feridos_2014')

**2015**

In [ ]:
df15_simples_dupla.tracado_via.unique()

In [ ]:
np.sort(df15_simples_dupla.feridos.unique())

In [ ]:
# Filtragem 'Reta' ou 'Curva' + pelo menos 1 ferido
reta_curva = df15_simples_dupla.tracado_via.isin(['Reta', 'Curva'])

feridos = df15_simples_dupla.feridos > 0

df15_sd_reta_curva_feridos = df15_simples_dupla[(reta_curva) & (feridos)]

In [ ]:
df15_sd_reta_curva_feridos.tracado_via.unique()

In [ ]:
np.sort(df15_sd_reta_curva_feridos.feridos.unique())

In [ ]:
counts, bins = np.histogram(df15_sd_reta_curva_feridos['Hora'], bins=range(0, 25, 1))
acidentes15 = np.array(counts)/sum(np.array(counts))
fluxo15 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa15 = (acidentes15/fluxo15)/sum(acidentes15/fluxo15)

In [ ]:
graphics_plot_prf(y=[taxa15, fluxo15, acidentes15],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a straight or curve section of the road with at least one person injured (2015) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_dupla_reta_curva_feridos_2015')

**2016**

In [ ]:
df16_simples_dupla.tracado_via.unique()

In [ ]:
np.sort(df16_simples_dupla.feridos.unique())

In [ ]:
# Filtragem 'Reta' ou 'Curva' + pelo menos 1 ferido
reta_curva = df16_simples_dupla.tracado_via.isin(['Reta', 'Curva'])

feridos = df16_simples_dupla.feridos > 0

df16_sd_reta_curva_feridos = df16_simples_dupla[(reta_curva) & (feridos)]

In [ ]:
df16_sd_reta_curva_feridos.tracado_via.unique()

In [ ]:
np.sort(df16_sd_reta_curva_feridos.feridos.unique())

In [ ]:
counts, bins = np.histogram(df16_sd_reta_curva_feridos['Hora'], bins=range(0, 25, 1))
acidentes16 = np.array(counts)/sum(np.array(counts))
fluxo16 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa16 = (acidentes16/fluxo16)/sum(acidentes16/fluxo16)

In [ ]:
graphics_plot_prf(y=[taxa16, fluxo16, acidentes16],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a straight or curve section of the road with at least one person injured (2016) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_dupla_reta_curva_feridos_2016')

**2017**

In [ ]:
df17_simples_dupla.tracado_via.unique()

In [ ]:
np.sort(df17_simples_dupla.feridos.unique())

In [ ]:
# Filtragem 'Reta' ou 'Curva' + pelo menos 1 ferido
reta_curva = df17_simples_dupla.tracado_via.isin(['Reta', 'Curva'])

feridos = df17_simples_dupla.feridos > 0

df17_sd_reta_curva_feridos = df17_simples_dupla[(reta_curva) & (feridos)]

In [ ]:
df17_sd_reta_curva_feridos.tracado_via.unique()

In [ ]:
np.sort(df17_sd_reta_curva_feridos.feridos.unique())

In [ ]:
counts, bins = np.histogram(df17_sd_reta_curva_feridos['Hora'], bins=range(0, 25, 1))
acidentes17 = np.array(counts)/sum(np.array(counts))
fluxo17 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa17 = (acidentes17/fluxo17)/sum(acidentes17/fluxo17)

In [ ]:
graphics_plot_prf(y=[taxa17, fluxo17, acidentes17],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a straight or curve section of the road with at least one person injured (2017) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_dupla_reta_curva_feridos_2017')

In [ ]:
# Salvando as taxas
df_taxas['simples_dupla_reta_curva_feridos_13'] = taxa13
df_taxas['simples_dupla_reta_curva_feridos_14'] = taxa14
df_taxas['simples_dupla_reta_curva_feridos_15'] = taxa15
df_taxas['simples_dupla_reta_curva_feridos_16'] = taxa16
df_taxas['simples_dupla_reta_curva_feridos_17'] = taxa17

## **Estrada simples**

### **Um veículo + Reta**

**2013**

In [ ]:
df13_simples.tracado_via.unique()

In [ ]:
# Filtragem 'Reta'
reta = df13_simples.tracado_via == 'Reta'

df13s_reta = df13_simples[(reta)]

In [ ]:
df13s_reta.tracado_via.unique()

In [ ]:
counts, bins = np.histogram(df13s_reta['Hora'], bins=range(0, 25, 1))
acidentes13 = np.array(counts)/sum(np.array(counts))
fluxo13 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa13 = (acidentes13/fluxo13)/sum(acidentes13/fluxo13)

In [ ]:
graphics_plot_prf(y=[taxa13, fluxo13, acidentes13],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a straight section of the road (2013) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_reta_2013')

**2014**

In [ ]:
df14_simples.tracado_via.unique()

In [ ]:
# Filtragem 'Reta'
reta = df14_simples.tracado_via == 'Reta'

df14s_reta = df14_simples[(reta)]

In [ ]:
df14s_reta.tracado_via.unique()

In [ ]:
counts, bins = np.histogram(df14s_reta['Hora'], bins=range(0, 25, 1))
acidentes14 = np.array(counts)/sum(np.array(counts))
fluxo14 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa14 = (acidentes14/fluxo14)/sum(acidentes14/fluxo14)

In [ ]:
graphics_plot_prf(y=[taxa14, fluxo14, acidentes14],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a straight section of the road (2014) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_reta_2014')

**2015**

In [ ]:
df15_simples.tracado_via.unique()

In [ ]:
# Filtragem 'Reta'
reta = df15_simples.tracado_via == 'Reta'

df15s_reta = df15_simples[(reta)]

In [ ]:
df15s_reta.tracado_via.unique()

In [ ]:
counts, bins = np.histogram(df15s_reta['Hora'], bins=range(0, 25, 1))
acidentes15 = np.array(counts)/sum(np.array(counts))
fluxo15 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa15 = (acidentes15/fluxo15)/sum(acidentes15/fluxo15)

In [ ]:
graphics_plot_prf(y=[taxa15, fluxo15, acidentes15],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a straight section of the road (2015) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_reta_2015')

**2016**

In [ ]:
df16_simples.tracado_via.unique()

In [ ]:
# Filtragem 'Reta'
reta = df16_simples.tracado_via == 'Reta'

df16s_reta = df16_simples[(reta)]

In [ ]:
df16s_reta.tracado_via.unique()

In [ ]:
counts, bins = np.histogram(df16s_reta['Hora'], bins=range(0, 25, 1))
acidentes16 = np.array(counts)/sum(np.array(counts))
fluxo16 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa16 = (acidentes16/fluxo16)/sum(acidentes16/fluxo16)

In [ ]:
graphics_plot_prf(y=[taxa16, fluxo16, acidentes16],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a straight section of the road (2016) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_reta_2016')

**2017**

In [ ]:
df17_simples.tracado_via.unique()

In [ ]:
# Filtragem 'Reta'
reta = df17_simples.tracado_via == 'Reta'

df17s_reta = df17_simples[(reta)]

In [ ]:
df17s_reta.tracado_via.unique()

In [ ]:
counts, bins = np.histogram(df17s_reta['Hora'], bins=range(0, 25, 1))
acidentes17 = np.array(counts)/sum(np.array(counts))
fluxo17 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa17 = (acidentes17/fluxo17)/sum(acidentes17/fluxo17)

In [ ]:
graphics_plot_prf(y=[taxa17, fluxo17, acidentes17],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a straight section of the road (2017) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_reta_2017')

In [ ]:
# Salvando as taxas
df_taxas['simples_reta_13'] = taxa13
df_taxas['simples_reta_14'] = taxa14
df_taxas['simples_reta_15'] = taxa15
df_taxas['simples_reta_16'] = taxa16
df_taxas['simples_reta_17'] = taxa17

### **Um veículo + Reta + feridos**

**2013**

In [ ]:
df13_simples.tracado_via.unique()

In [ ]:
np.sort(df13_simples.feridos.unique())

In [ ]:
# Filtragem 'Reta' + pelo menos 1 ferido
reta = df13_simples.tracado_via == 'Reta'

feridos = df13_simples.feridos > 0

df13s_reta_feridos = df13_simples[(reta) & (feridos)]

In [ ]:
df13s_reta_feridos.tracado_via.unique()

In [ ]:
np.sort(df13s_reta_feridos.feridos.unique())

In [ ]:
counts, bins = np.histogram(df13s_reta_feridos['Hora'], bins=range(0, 25, 1))
acidentes13 = np.array(counts)/sum(np.array(counts))
fluxo13 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa13 = (acidentes13/fluxo13)/sum(acidentes13/fluxo13)

In [ ]:
graphics_plot_prf(y=[taxa13, fluxo13, acidentes13],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a straight section of the road with at least one person injured (2013) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_reta_feridos_2013')

**2014**

In [ ]:
df14_simples.tracado_via.unique()

In [ ]:
np.sort(df14_simples.feridos.unique())

In [ ]:
# Filtragem 'Reta' + pelo menos 1 ferido
reta = df14_simples.tracado_via == 'Reta'

feridos = df14_simples.feridos > 0

df14s_reta_feridos = df14_simples[(reta) & (feridos)]

In [ ]:
df14s_reta_feridos.tracado_via.unique()

In [ ]:
np.sort(df14s_reta_feridos.feridos.unique())

In [ ]:
counts, bins = np.histogram(df14s_reta_feridos['Hora'], bins=range(0, 25, 1))
acidentes14 = np.array(counts)/sum(np.array(counts))
fluxo14 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa14 = (acidentes14/fluxo14)/sum(acidentes14/fluxo14)

In [ ]:
graphics_plot_prf(y=[taxa14, fluxo14, acidentes14],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a straight section of the road with at least one person injured (2014) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_reta_feridos_2014')

**2015**

In [ ]:
df15_simples.tracado_via.unique()

In [ ]:
np.sort(df15_simples.feridos.unique())

In [ ]:
# Filtragem 'Reta' + pelo menos 1 ferido
reta = df15_simples.tracado_via == 'Reta'

feridos = df15_simples.feridos > 0

df15s_reta_feridos = df15_simples[(reta) & (feridos)]

In [ ]:
df15s_reta_feridos.tracado_via.unique()

In [ ]:
np.sort(df15s_reta_feridos.feridos.unique())

In [ ]:
counts, bins = np.histogram(df15s_reta_feridos['Hora'], bins=range(0, 25, 1))
acidentes15 = np.array(counts)/sum(np.array(counts))
fluxo15 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa15 = (acidentes15/fluxo15)/sum(acidentes15/fluxo15)

In [ ]:
graphics_plot_prf(y=[taxa15, fluxo15, acidentes15],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a straight section of the road with at least one person injured (2015) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_reta_feridos_2015')

**2016**

In [ ]:
df16_simples.tracado_via.unique()

In [ ]:
np.sort(df16_simples.feridos.unique())

In [ ]:
# Filtragem 'Reta' + pelo menos 1 ferido
reta = df16_simples.tracado_via == 'Reta'

feridos = df16_simples.feridos > 0

df16s_reta_feridos = df16_simples[(reta) & (feridos)]

In [ ]:
df16s_reta_feridos.tracado_via.unique()

In [ ]:
np.sort(df16s_reta_feridos.feridos.unique())

In [ ]:
counts, bins = np.histogram(df16s_reta_feridos['Hora'], bins=range(0, 25, 1))
acidentes16 = np.array(counts)/sum(np.array(counts))
fluxo16 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa16 = (acidentes16/fluxo16)/sum(acidentes16/fluxo16)

In [ ]:
graphics_plot_prf(y=[taxa16, fluxo16, acidentes16],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a straight section of the road with at least one person injured (2016) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_reta_feridos_2016')

**2017**

In [ ]:
df17_simples.tracado_via.unique()

In [ ]:
np.sort(df17_simples.feridos.unique())

In [ ]:
# Filtragem 'Reta' + pelo menos 1 ferido
reta = df17_simples.tracado_via == 'Reta'

feridos = df17_simples.feridos > 0

df17s_reta_feridos = df17_simples[(reta) & (feridos)]

In [ ]:
df17s_reta_feridos.tracado_via.unique()

In [ ]:
np.sort(df17s_reta_feridos.feridos.unique())

In [ ]:
counts, bins = np.histogram(df17s_reta_feridos['Hora'], bins=range(0, 25, 1))
acidentes17 = np.array(counts)/sum(np.array(counts))
fluxo17 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa17 = (acidentes17/fluxo17)/sum(acidentes17/fluxo17)

In [ ]:
graphics_plot_prf(y=[taxa17, fluxo17, acidentes17],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a straight section of the road with at least one person injured (2017) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_reta_feridos_2017')

In [ ]:
# Salvando as taxas
df_taxas['simples_reta_feridos_13'] = taxa13
df_taxas['simples_reta_feridos_14'] = taxa14
df_taxas['simples_reta_feridos_15'] = taxa15
df_taxas['simples_reta_feridos_16'] = taxa16
df_taxas['simples_reta_feridos_17'] = taxa17

### **Um veículo + Curva**

**2013**

In [ ]:
df13_simples.tracado_via.unique()

In [ ]:
# Filtragem 'Curva'
curva = df13_simples.tracado_via == 'Curva'

df13s_curva = df13_simples[(curva)]

In [ ]:
df13s_curva.tracado_via.unique()

In [ ]:
counts, bins = np.histogram(df13s_curva['Hora'], bins=range(0, 25, 1))
acidentes13 = np.array(counts)/sum(np.array(counts))
fluxo13 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa13 = (acidentes13/fluxo13)/sum(acidentes13/fluxo13)

In [ ]:
graphics_plot_prf(y=[taxa13, fluxo13, acidentes13],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a curve section of the road (2013) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_curva_2013')

**2014**

In [ ]:
df14_simples.tracado_via.unique()

In [ ]:
# Filtragem 'Curva'
curva = df14_simples.tracado_via == 'Curva'

df14s_curva = df14_simples[(curva)]

In [ ]:
df14s_curva.tracado_via.unique()

In [ ]:
counts, bins = np.histogram(df14s_curva['Hora'], bins=range(0, 25, 1))
acidentes14 = np.array(counts)/sum(np.array(counts))
fluxo14 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa14 = (acidentes14/fluxo14)/sum(acidentes14/fluxo14)

In [ ]:
graphics_plot_prf(y=[taxa14, fluxo14, acidentes14],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a curve section of the road (2014) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_curva_2014')

**2015**

In [ ]:
df15_simples.tracado_via.unique()

In [ ]:
# Filtragem 'Curva'
curva = df15_simples.tracado_via == 'Curva'

df15s_curva = df15_simples[(curva)]

In [ ]:
df15s_curva.tracado_via.unique()

In [ ]:
counts, bins = np.histogram(df15s_curva['Hora'], bins=range(0, 25, 1))
acidentes15 = np.array(counts)/sum(np.array(counts))
fluxo15 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa15 = (acidentes15/fluxo15)/sum(acidentes15/fluxo15)

In [ ]:
graphics_plot_prf(y=[taxa15, fluxo15, acidentes15],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a curve section of the road (2015) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_curva_2015')

**2016**

In [ ]:
df16_simples.tracado_via.unique()

In [ ]:
# Filtragem 'Curva'
curva = df16_simples.tracado_via == 'Curva'

df16s_curva = df16_simples[(curva)]

In [ ]:
df16s_curva.tracado_via.unique()

In [ ]:
counts, bins = np.histogram(df16s_curva['Hora'], bins=range(0, 25, 1))
acidentes16 = np.array(counts)/sum(np.array(counts))
fluxo16 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa16 = (acidentes16/fluxo16)/sum(acidentes16/fluxo16)

In [ ]:
graphics_plot_prf(y=[taxa16, fluxo16, acidentes16],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a curve section of the road (2016) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_curva_2016')

**2017**

In [ ]:
df17_simples.tracado_via.unique()

In [ ]:
# Filtragem 'Curva'
curva = df17_simples.tracado_via == 'Curva'

df17s_curva = df17_simples[(curva)]

In [ ]:
df17s_curva.tracado_via.unique()

In [ ]:
counts, bins = np.histogram(df17s_curva['Hora'], bins=range(0, 25, 1))
acidentes17 = np.array(counts)/sum(np.array(counts))
fluxo17 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa17 = (acidentes17/fluxo17)/sum(acidentes17/fluxo17)

In [ ]:
graphics_plot_prf(y=[taxa17, fluxo17, acidentes17],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a curve section of the road (2017) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_curva_2017')

In [ ]:
# Salvando as taxas
df_taxas['simples_curva_13'] = taxa13
df_taxas['simples_curva_14'] = taxa14
df_taxas['simples_curva_15'] = taxa15
df_taxas['simples_curva_16'] = taxa16
df_taxas['simples_curva_17'] = taxa17

### **Um veículo + Curva + feridos**

**2013**

In [ ]:
df13_simples.tracado_via.unique()

In [ ]:
np.sort(df13_simples.feridos.unique())

In [ ]:
# Filtragem 'Curva' + pelo menos 1 ferido
curva = df13_simples.tracado_via == 'Curva'

feridos = df13_simples.feridos > 0

df13s_curva_feridos = df13_simples[(curva) & (feridos)]

In [ ]:
df13s_curva_feridos.tracado_via.unique()

In [ ]:
np.sort(df13s_curva_feridos.feridos.unique())

In [ ]:
counts, bins = np.histogram(df13s_curva_feridos['Hora'], bins=range(0, 25, 1))
acidentes13 = np.array(counts)/sum(np.array(counts))
fluxo13 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa13 = (acidentes13/fluxo13)/sum(acidentes13/fluxo13)

In [ ]:
graphics_plot_prf(y=[taxa13, fluxo13, acidentes13],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a curve section of the road with at least one person injured (2013) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_curva_feridos_2013')

**2014**

In [ ]:
df14_simples.tracado_via.unique()

In [ ]:
np.sort(df14_simples.feridos.unique())

In [ ]:
# Filtragem 'Curva' + pelo menos 1 ferido
curva = df14_simples.tracado_via == 'Curva'

feridos = df14_simples.feridos > 0

df14s_curva_feridos = df14_simples[(curva) & (feridos)]

In [ ]:
df14s_curva_feridos.tracado_via.unique()

In [ ]:
np.sort(df14s_curva_feridos.feridos.unique())

In [ ]:
counts, bins = np.histogram(df14s_curva_feridos['Hora'], bins=range(0, 25, 1))
acidentes14 = np.array(counts)/sum(np.array(counts))
fluxo14 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa14 = (acidentes14/fluxo14)/sum(acidentes14/fluxo14)

In [ ]:
graphics_plot_prf(y=[taxa14, fluxo14, acidentes14],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a curve section of the road with at least one person injured (2014) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_curva_feridos_2014')

**2015**

In [ ]:
df15_simples.tracado_via.unique()

In [ ]:
np.sort(df15_simples.feridos.unique())

In [ ]:
# Filtragem 'Curva' + pelo menos 1 ferido
curva = df15_simples.tracado_via == 'Curva'

feridos = df15_simples.feridos > 0

df15s_curva_feridos = df15_simples[(curva) & (feridos)]

In [ ]:
df15s_curva_feridos.tracado_via.unique()

In [ ]:
np.sort(df15s_curva_feridos.feridos.unique())

In [ ]:
counts, bins = np.histogram(df15s_curva_feridos['Hora'], bins=range(0, 25, 1))
acidentes15 = np.array(counts)/sum(np.array(counts))
fluxo15 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa15 = (acidentes15/fluxo15)/sum(acidentes15/fluxo15)

In [ ]:
graphics_plot_prf(y=[taxa15, fluxo15, acidentes15],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a curve section of the road with at least one person injured (2015) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_curva_feridos_2015')

**2016**

In [ ]:
df16_simples.tracado_via.unique()

In [ ]:
np.sort(df16_simples.feridos.unique())

In [ ]:
# Filtragem 'Curva' + pelo menos 1 ferido
curva = df16_simples.tracado_via == 'Curva'

feridos = df16_simples.feridos > 0

df16s_curva_feridos = df16_simples[(curva) & (feridos)]

In [ ]:
df16s_curva_feridos.tracado_via.unique()

In [ ]:
np.sort(df16s_curva_feridos.feridos.unique())

In [ ]:
counts, bins = np.histogram(df16s_curva_feridos['Hora'], bins=range(0, 25, 1))
acidentes16 = np.array(counts)/sum(np.array(counts))
fluxo16 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa16 = (acidentes16/fluxo16)/sum(acidentes16/fluxo16)

In [ ]:
graphics_plot_prf(y=[taxa16, fluxo16, acidentes16],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a curve section of the road with at least one person injured (2016) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_curva_feridos_2016')

**2017**

In [ ]:
df17_simples.tracado_via.unique()

In [ ]:
np.sort(df17_simples.feridos.unique())

In [ ]:
# Filtragem 'Curva' + pelo menos 1 ferido
curva = df17_simples.tracado_via == 'Curva'

feridos = df17_simples.feridos > 0

df17s_curva_feridos = df17_simples[(curva) & (feridos)]

In [ ]:
df17s_curva_feridos.tracado_via.unique()

In [ ]:
np.sort(df17s_curva_feridos.feridos.unique())

In [ ]:
counts, bins = np.histogram(df17s_curva_feridos['Hora'], bins=range(0, 25, 1))
acidentes17 = np.array(counts)/sum(np.array(counts))
fluxo17 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa17 = (acidentes17/fluxo17)/sum(acidentes17/fluxo17)

In [ ]:
graphics_plot_prf(y=[taxa17, fluxo17, acidentes17],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a curve section of the road with at least one person injured (2017) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_curva_feridos_2017')

In [ ]:
# Salvando as taxas
df_taxas['simples_curva_feridos_13'] = taxa13
df_taxas['simples_curva_feridos_14'] = taxa14
df_taxas['simples_curva_feridos_15'] = taxa15
df_taxas['simples_curva_feridos_16'] = taxa16
df_taxas['simples_curva_feridos_17'] = taxa17

### **Um veículo + Reta + Curva**

**2013**

In [ ]:
df13_simples.tracado_via.unique()

In [ ]:
# Filtragem 'Reta' ou 'Curva'
reta_curva = df13_simples.tracado_via.isin(['Reta', 'Curva'])

df13s_reta_curva = df13_simples[(reta_curva)]

In [ ]:
df13s_reta_curva.tracado_via.unique()

In [ ]:
counts, bins = np.histogram(df13s_reta_curva['Hora'], bins=range(0, 25, 1))
acidentes13 = np.array(counts)/sum(np.array(counts))
fluxo13 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa13 = (acidentes13/fluxo13)/sum(acidentes13/fluxo13)

In [ ]:
graphics_plot_prf(y=[taxa13, fluxo13, acidentes13],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a straight or curve section of the road (2013) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_reta_curva_2013')

**2014**

In [ ]:
df14_simples.tracado_via.unique()

In [ ]:
# Filtragem 'Reta' ou 'Curva'
reta_curva = df14_simples.tracado_via.isin(['Reta', 'Curva'])

df14s_reta_curva = df14_simples[(reta_curva)]

In [ ]:
df14s_reta_curva.tracado_via.unique()

In [ ]:
counts, bins = np.histogram(df14s_reta_curva['Hora'], bins=range(0, 25, 1))
acidentes14 = np.array(counts)/sum(np.array(counts))
fluxo14 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa14 = (acidentes14/fluxo14)/sum(acidentes14/fluxo14)

In [ ]:
graphics_plot_prf(y=[taxa14, fluxo14, acidentes14],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a straight or curve section of the road (2014) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_reta_curva_2014')

**2015**

In [ ]:
df15_simples.tracado_via.unique()

In [ ]:
# Filtragem 'Reta' ou 'Curva'
reta_curva = df15_simples.tracado_via.isin(['Reta', 'Curva'])

df15s_reta_curva = df15_simples[(reta_curva)]

In [ ]:
df15s_reta_curva.tracado_via.unique()

In [ ]:
counts, bins = np.histogram(df15s_reta_curva['Hora'], bins=range(0, 25, 1))
acidentes15 = np.array(counts)/sum(np.array(counts))
fluxo15 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa15 = (acidentes15/fluxo15)/sum(acidentes15/fluxo15)

In [ ]:
graphics_plot_prf(y=[taxa15, fluxo15, acidentes15],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a straight or curve section of the road (2015) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_reta_curva_2015')

**2016**

In [ ]:
df16_simples.tracado_via.unique()

In [ ]:
# Filtragem 'Reta' ou 'Curva'
reta_curva = df16_simples.tracado_via.isin(['Reta', 'Curva'])

df16s_reta_curva = df16_simples[(reta_curva)]

In [ ]:
df16s_reta_curva.tracado_via.unique()

In [ ]:
counts, bins = np.histogram(df16s_reta_curva['Hora'], bins=range(0, 25, 1))
acidentes16 = np.array(counts)/sum(np.array(counts))
fluxo16 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa16 = (acidentes16/fluxo16)/sum(acidentes16/fluxo16)

In [ ]:
graphics_plot_prf(y=[taxa16, fluxo16, acidentes16],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a straight or curve section of the road (2016) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_reta_curva_2016')

**2017**

In [ ]:
df17_simples.tracado_via.unique()

In [ ]:
# Filtragem 'Reta' ou 'Curva'
reta_curva = df17_simples.tracado_via.isin(['Reta', 'Curva'])

df17s_reta_curva = df17_simples[(reta_curva)]

In [ ]:
df17s_reta_curva.tracado_via.unique()

In [ ]:
counts, bins = np.histogram(df17s_reta_curva['Hora'], bins=range(0, 25, 1))
acidentes17 = np.array(counts)/sum(np.array(counts))
fluxo17 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa17 = (acidentes17/fluxo17)/sum(acidentes17/fluxo17)

In [ ]:
graphics_plot_prf(y=[taxa17, fluxo17, acidentes17],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a straight or curve section of the road (2017) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_reta_curva_2017')

In [ ]:
# Salvando as taxas
df_taxas['simples_reta_curva_13'] = taxa13
df_taxas['simples_reta_curva_14'] = taxa14
df_taxas['simples_reta_curva_15'] = taxa15
df_taxas['simples_reta_curva_16'] = taxa16
df_taxas['simples_reta_curva_17'] = taxa17

### **Um veículo + Reta + Curva + feridos**

**2013**

In [ ]:
df13_simples.tracado_via.unique()

In [ ]:
np.sort(df13_simples.feridos.unique())

In [ ]:
# Filtragem 'Reta' ou 'Curva' + pelo menos 1 ferido
reta_curva = df13_simples.tracado_via.isin(['Reta', 'Curva'])

feridos = df13_simples.feridos > 0

df13s_reta_curva_feridos = df13_simples[(reta_curva) & (feridos)]

In [ ]:
df13s_reta_curva_feridos.tracado_via.unique()

In [ ]:
np.sort(df13s_reta_curva_feridos.feridos.unique())

In [ ]:
counts, bins = np.histogram(df13s_reta_curva_feridos['Hora'], bins=range(0, 25, 1))
acidentes13 = np.array(counts)/sum(np.array(counts))
fluxo13 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa13 = (acidentes13/fluxo13)/sum(acidentes13/fluxo13)

In [ ]:
graphics_plot_prf(y=[taxa13, fluxo13, acidentes13],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a straight or curve section of the road with at least one person injured (2013) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_reta_curva_feridos_2013')

**2014**

In [ ]:
df14_simples.tracado_via.unique()

In [ ]:
np.sort(df14_simples.feridos.unique())

In [ ]:
# Filtragem 'Reta' ou 'Curva' + pelo menos 1 ferido
reta_curva = df14_simples.tracado_via.isin(['Reta', 'Curva'])

feridos = df14_simples.feridos > 0

df14s_reta_curva_feridos = df14_simples[(reta_curva) & (feridos)]

In [ ]:
df14s_reta_curva_feridos.tracado_via.unique()

In [ ]:
np.sort(df14s_reta_curva_feridos.feridos.unique())

In [ ]:
counts, bins = np.histogram(df14s_reta_curva_feridos['Hora'], bins=range(0, 25, 1))
acidentes14 = np.array(counts)/sum(np.array(counts))
fluxo14 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa14 = (acidentes14/fluxo14)/sum(acidentes14/fluxo14)

In [ ]:
graphics_plot_prf(y=[taxa14, fluxo14, acidentes14],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a straight or curve section of the road with at least one person injured (2014) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_reta_curva_feridos_2014')

**2015**

In [ ]:
df15_simples.tracado_via.unique()

In [ ]:
np.sort(df15_simples.feridos.unique())

In [ ]:
# Filtragem 'Reta' ou 'Curva' + pelo menos 1 ferido
reta_curva = df15_simples.tracado_via.isin(['Reta', 'Curva'])

feridos = df15_simples.feridos > 0

df15s_reta_curva_feridos = df15_simples[(reta_curva) & (feridos)]

In [ ]:
df15s_reta_curva_feridos.tracado_via.unique()

In [ ]:
np.sort(df15s_reta_curva_feridos.feridos.unique())

In [ ]:
counts, bins = np.histogram(df15s_reta_curva_feridos['Hora'], bins=range(0, 25, 1))
acidentes15 = np.array(counts)/sum(np.array(counts))
fluxo15 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa15 = (acidentes15/fluxo15)/sum(acidentes15/fluxo15)

In [ ]:
graphics_plot_prf(y=[taxa15, fluxo15, acidentes15],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a straight or curve section of the road with at least one person injured (2015) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_reta_curva_feridos_2015')

**2016**

In [ ]:
df16_simples.tracado_via.unique()

In [ ]:
np.sort(df16_simples.feridos.unique())

In [ ]:
# Filtragem 'Reta' ou 'Curva' + pelo menos 1 ferido
reta_curva = df16_simples.tracado_via.isin(['Reta', 'Curva'])

feridos = df16_simples.feridos > 0

df16s_reta_curva_feridos = df16_simples[(reta_curva) & (feridos)]

In [ ]:
df16s_reta_curva_feridos.tracado_via.unique()

In [ ]:
np.sort(df16s_reta_curva_feridos.feridos.unique())

In [ ]:
counts, bins = np.histogram(df16s_reta_curva_feridos['Hora'], bins=range(0, 25, 1))
acidentes16 = np.array(counts)/sum(np.array(counts))
fluxo16 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa16 = (acidentes16/fluxo16)/sum(acidentes16/fluxo16)

In [ ]:
graphics_plot_prf(y=[taxa16, fluxo16, acidentes16],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a straight or curve section of the road with at least one person injured (2016) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_reta_curva_feridos_2016')

**2017**

In [ ]:
df17_simples.tracado_via.unique()

In [ ]:
np.sort(df17_simples.feridos.unique())

In [ ]:
# Filtragem 'Reta' ou 'Curva' + pelo menos 1 ferido
reta_curva = df17_simples.tracado_via.isin(['Reta', 'Curva'])

feridos = df17_simples.feridos > 0

df17s_reta_curva_feridos = df17_simples[(reta_curva) & (feridos)]

In [ ]:
df17s_reta_curva_feridos.tracado_via.unique()

In [ ]:
np.sort(df17s_reta_curva_feridos.feridos.unique())

In [ ]:
counts, bins = np.histogram(df17s_reta_curva_feridos['Hora'], bins=range(0, 25, 1))
acidentes17 = np.array(counts)/sum(np.array(counts))
fluxo17 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa17 = (acidentes17/fluxo17)/sum(acidentes17/fluxo17)

In [ ]:
graphics_plot_prf(y=[taxa17, fluxo17, acidentes17],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a straight or curve section of the road with at least one person injured (2017) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_reta_curva_feridos_2017')

In [ ]:
# Salvando as taxas
df_taxas['simples_reta_curva_feridos_13'] = taxa13
df_taxas['simples_reta_curva_feridos_14'] = taxa14
df_taxas['simples_reta_curva_feridos_15'] = taxa15
df_taxas['simples_reta_curva_feridos_16'] = taxa16
df_taxas['simples_reta_curva_feridos_17'] = taxa17

## **Estrada dupla**

### **Um veículo + Reta**

**2013**

In [ ]:
df13_dupla.tracado_via.unique()

In [ ]:
# Filtragem 'Reta'
reta = df13_dupla.tracado_via == 'Reta'

df13d_reta = df13_dupla[(reta)]

In [ ]:
df13d_reta.tracado_via.unique()

In [ ]:
counts, bins = np.histogram(df13d_reta['Hora'], bins=range(0, 25, 1))
acidentes13 = np.array(counts)/sum(np.array(counts))
fluxo13 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa13 = (acidentes13/fluxo13)/sum(acidentes13/fluxo13)

In [ ]:
graphics_plot_prf(y=[taxa13, fluxo13, acidentes13],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a straight section of the road (2013) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='dupla_reta_2013')

**2014**

In [ ]:
df14_dupla.tracado_via.unique()

In [ ]:
# Filtragem 'Reta'
reta = df14_dupla.tracado_via == 'Reta'

df14d_reta = df14_dupla[(reta)]

In [ ]:
df14d_reta.tracado_via.unique()

In [ ]:
counts, bins = np.histogram(df14d_reta['Hora'], bins=range(0, 25, 1))
acidentes14 = np.array(counts)/sum(np.array(counts))
fluxo14 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa14 = (acidentes14/fluxo14)/sum(acidentes14/fluxo14)

In [ ]:
graphics_plot_prf(y=[taxa14, fluxo14, acidentes14],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a straight section of the road (2014) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='dupla_reta_2014')

**2015**

In [ ]:
df15_dupla.tracado_via.unique()

In [ ]:
# Filtragem 'Reta'
reta = df15_dupla.tracado_via == 'Reta'

df15d_reta = df15_dupla[(reta)]

In [ ]:
df15d_reta.tracado_via.unique()

In [ ]:
counts, bins = np.histogram(df15d_reta['Hora'], bins=range(0, 25, 1))
acidentes15 = np.array(counts)/sum(np.array(counts))
fluxo15 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa15 = (acidentes15/fluxo15)/sum(acidentes15/fluxo15)

In [ ]:
graphics_plot_prf(y=[taxa15, fluxo15, acidentes15],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a straight section of the road (2015) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='dupla_reta_2015')

**2016**

In [ ]:
df16_dupla.tracado_via.unique()

In [ ]:
# Filtragem 'Reta'
reta = df16_dupla.tracado_via == 'Reta'

df16d_reta = df16_dupla[(reta)]

In [ ]:
df16d_reta.tracado_via.unique()

In [ ]:
counts, bins = np.histogram(df16d_reta['Hora'], bins=range(0, 25, 1))
acidentes16 = np.array(counts)/sum(np.array(counts))
fluxo16 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa16 = (acidentes16/fluxo16)/sum(acidentes16/fluxo16)

In [ ]:
graphics_plot_prf(y=[taxa16, fluxo16, acidentes16],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a straight section of the road (2016) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='dupla_reta_2016')

**2017**

In [ ]:
df17_dupla.tracado_via.unique()

In [ ]:
# Filtragem 'Reta'
reta = df17_dupla.tracado_via == 'Reta'

df17d_reta = df17_dupla[(reta)]

In [ ]:
df17d_reta.tracado_via.unique()

In [ ]:
counts, bins = np.histogram(df17d_reta['Hora'], bins=range(0, 25, 1))
acidentes17 = np.array(counts)/sum(np.array(counts))
fluxo17 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa17 = (acidentes17/fluxo17)/sum(acidentes17/fluxo17)

In [ ]:
graphics_plot_prf(y=[taxa17, fluxo17, acidentes17],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a straight section of the road (2017) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='dupla_reta_2017')

In [ ]:
# Salvando as taxas
df_taxas['dupla_reta_13'] = taxa13
df_taxas['dupla_reta_14'] = taxa14
df_taxas['dupla_reta_15'] = taxa15
df_taxas['dupla_reta_16'] = taxa16
df_taxas['dupla_reta_17'] = taxa17

### **Um veículo + Reta + feridos**

**2013**

In [ ]:
df13_dupla.tracado_via.unique()

In [ ]:
np.sort(df13_dupla.feridos.unique())

In [ ]:
# Filtragem 'Reta' + pelo menos 1 ferido
reta = df13_dupla.tracado_via == 'Reta'

feridos = df13_dupla.feridos > 0

df13d_reta_feridos = df13_dupla[(reta) & (feridos)]

In [ ]:
df13d_reta_feridos.tracado_via.unique()

In [ ]:
np.sort(df13d_reta_feridos.feridos.unique())

In [ ]:
counts, bins = np.histogram(df13d_reta_feridos['Hora'], bins=range(0, 25, 1))
acidentes13 = np.array(counts)/sum(np.array(counts))
fluxo13 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa13 = (acidentes13/fluxo13)/sum(acidentes13/fluxo13)

In [ ]:
graphics_plot_prf(y=[taxa13, fluxo13, acidentes13],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a straight section of the road with at least one person injured (2013) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='dupla_reta_feridos_2013')

**2014**

In [ ]:
df14_dupla.tracado_via.unique()

In [ ]:
np.sort(df14_dupla.feridos.unique())

In [ ]:
# Filtragem 'Reta' + pelo menos 1 ferido
reta = df14_dupla.tracado_via == 'Reta'

feridos = df14_dupla.feridos > 0

df14d_reta_feridos = df14_dupla[(reta) & (feridos)]

In [ ]:
df14d_reta_feridos.tracado_via.unique()

In [ ]:
np.sort(df14d_reta_feridos.feridos.unique())

In [ ]:
counts, bins = np.histogram(df14d_reta_feridos['Hora'], bins=range(0, 25, 1))
acidentes14 = np.array(counts)/sum(np.array(counts))
fluxo14 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa14 = (acidentes14/fluxo14)/sum(acidentes14/fluxo14)

In [ ]:
graphics_plot_prf(y=[taxa14, fluxo14, acidentes14],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a straight section of the road with at least one person injured (2014) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='dupla_reta_feridos_2014')

**2015**

In [ ]:
df15_dupla.tracado_via.unique()

In [ ]:
np.sort(df15_dupla.feridos.unique())

In [ ]:
# Filtragem 'Reta' + pelo menos 1 ferido
reta = df15_dupla.tracado_via == 'Reta'

feridos = df15_dupla.feridos > 0

df15d_reta_feridos = df15_dupla[(reta) & (feridos)]

In [ ]:
df15d_reta_feridos.tracado_via.unique()

In [ ]:
np.sort(df15d_reta_feridos.feridos.unique())

In [ ]:
counts, bins = np.histogram(df15d_reta_feridos['Hora'], bins=range(0, 25, 1))
acidentes15 = np.array(counts)/sum(np.array(counts))
fluxo15 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa15 = (acidentes15/fluxo15)/sum(acidentes15/fluxo15)

In [ ]:
graphics_plot_prf(y=[taxa15, fluxo15, acidentes15],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a straight section of the road with at least one person injured (2015) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='dupla_reta_feridos_2015')

**2016**

In [ ]:
df16_dupla.tracado_via.unique()

In [ ]:
np.sort(df16_dupla.feridos.unique())

In [ ]:
# Filtragem 'Reta' + pelo menos 1 ferido
reta = df16_dupla.tracado_via == 'Reta'

feridos = df16_dupla.feridos > 0

df16d_reta_feridos = df16_dupla[(reta) & (feridos)]

In [ ]:
df16d_reta_feridos.tracado_via.unique()

In [ ]:
np.sort(df16d_reta_feridos.feridos.unique())

In [ ]:
counts, bins = np.histogram(df16d_reta_feridos['Hora'], bins=range(0, 25, 1))
acidentes16 = np.array(counts)/sum(np.array(counts))
fluxo16 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa16 = (acidentes16/fluxo16)/sum(acidentes16/fluxo16)

In [ ]:
graphics_plot_prf(y=[taxa16, fluxo16, acidentes16],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a straight section of the road with at least one person injured (2016) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='dupla_reta_feridos_2016')

**2017**

In [ ]:
df17_dupla.tracado_via.unique()

In [ ]:
np.sort(df17_dupla.feridos.unique())

In [ ]:
# Filtragem 'Reta' + pelo menos 1 ferido
reta = df17_dupla.tracado_via == 'Reta'

feridos = df17_dupla.feridos > 0

df17d_reta_feridos = df17_dupla[(reta) & (feridos)]

In [ ]:
df17d_reta_feridos.tracado_via.unique()

In [ ]:
np.sort(df17d_reta_feridos.feridos.unique())

In [ ]:
counts, bins = np.histogram(df17d_reta_feridos['Hora'], bins=range(0, 25, 1))
acidentes17 = np.array(counts)/sum(np.array(counts))
fluxo17 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa17 = (acidentes17/fluxo17)/sum(acidentes17/fluxo17)

In [ ]:
graphics_plot_prf(y=[taxa17, fluxo17, acidentes17],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a straight section of the road with at least one person injured (2017) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='dupla_reta_feridos_2017')

In [ ]:
# Salvando as taxas
df_taxas['dupla_reta_feridos_13'] = taxa13
df_taxas['dupla_reta_feridos_14'] = taxa14
df_taxas['dupla_reta_feridos_15'] = taxa15
df_taxas['dupla_reta_feridos_16'] = taxa16
df_taxas['dupla_reta_feridos_17'] = taxa17

### **Um veículo + Curva**

**2013**

In [ ]:
df13_dupla.tracado_via.unique()

In [ ]:
# Filtragem 'Curva'
curva = df13_dupla.tracado_via == 'Curva'

df13d_curva = df13_dupla[(curva)]

In [ ]:
df13d_curva.tracado_via.unique()

In [ ]:
counts, bins = np.histogram(df13d_curva['Hora'], bins=range(0, 25, 1))
acidentes13 = np.array(counts)/sum(np.array(counts))
fluxo13 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa13 = (acidentes13/fluxo13)/sum(acidentes13/fluxo13)

In [ ]:
graphics_plot_prf(y=[taxa13, fluxo13, acidentes13],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a curve section of the road (2013) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='dupla_curva_2013')

**2014**

In [ ]:
df14_dupla.tracado_via.unique()

In [ ]:
# Filtragem 'Curva'
curva = df14_dupla.tracado_via == 'Curva'

df14d_curva = df14_dupla[(curva)]

In [ ]:
df14d_curva.tracado_via.unique()

In [ ]:
counts, bins = np.histogram(df14d_curva['Hora'], bins=range(0, 25, 1))
acidentes14 = np.array(counts)/sum(np.array(counts))
fluxo14 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa14 = (acidentes14/fluxo14)/sum(acidentes14/fluxo14)

In [ ]:
graphics_plot_prf(y=[taxa14, fluxo14, acidentes14],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a curve section of the road (2014) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='dupla_curva_2014')

**2015**

In [ ]:
df15_dupla.tracado_via.unique()

In [ ]:
# Filtragem 'Curva'
curva = df15_dupla.tracado_via == 'Curva'

df15d_curva = df15_dupla[(curva)]

In [ ]:
df15d_curva.tracado_via.unique()

In [ ]:
counts, bins = np.histogram(df15d_curva['Hora'], bins=range(0, 25, 1))
acidentes15 = np.array(counts)/sum(np.array(counts))
fluxo15 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa15 = (acidentes15/fluxo15)/sum(acidentes15/fluxo15)

In [ ]:
graphics_plot_prf(y=[taxa15, fluxo15, acidentes15],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a curve section of the road (2015) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='dupla_curva_2015')

**2016**

In [ ]:
df16_dupla.tracado_via.unique()

In [ ]:
# Filtragem 'Curva'
curva = df16_dupla.tracado_via == 'Curva'

df16d_curva = df16_dupla[(curva)]

In [ ]:
df16d_curva.tracado_via.unique()

In [ ]:
counts, bins = np.histogram(df16d_curva['Hora'], bins=range(0, 25, 1))
acidentes16 = np.array(counts)/sum(np.array(counts))
fluxo16 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa16 = (acidentes16/fluxo16)/sum(acidentes16/fluxo16)

In [ ]:
graphics_plot_prf(y=[taxa16, fluxo16, acidentes16],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a curve section of the road (2016) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='dupla_curva_2016')

**2017**

In [ ]:
df17_dupla.tracado_via.unique()

In [ ]:
# Filtragem 'Curva'
curva = df17_dupla.tracado_via == 'Curva'

df17d_curva = df17_dupla[(curva)]

In [ ]:
df17d_curva.tracado_via.unique()

In [ ]:
counts, bins = np.histogram(df17d_curva['Hora'], bins=range(0, 25, 1))
acidentes17 = np.array(counts)/sum(np.array(counts))
fluxo17 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa17 = (acidentes17/fluxo17)/sum(acidentes17/fluxo17)

In [ ]:
graphics_plot_prf(y=[taxa17, fluxo17, acidentes17],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a curve section of the road (2017) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='dupla_curva_2017')

In [ ]:
# Salvando as taxas
df_taxas['dupla_curva_13'] = taxa13
df_taxas['dupla_curva_14'] = taxa14
df_taxas['dupla_curva_15'] = taxa15
df_taxas['dupla_curva_16'] = taxa16
df_taxas['dupla_curva_17'] = taxa17

### **Um veículo + Curva + feridos**

**2013**

In [ ]:
df13_dupla.tracado_via.unique()

In [ ]:
np.sort(df13_dupla.feridos.unique())

In [ ]:
# Filtragem 'Curva' + pelo menos 1 ferido
curva = df13_dupla.tracado_via == 'Curva'

feridos = df13_dupla.feridos > 0

df13d_curva_feridos = df13_dupla[(curva) & (feridos)]

In [ ]:
df13d_curva_feridos.tracado_via.unique()

In [ ]:
np.sort(df13d_curva_feridos.feridos.unique())

In [ ]:
counts, bins = np.histogram(df13d_curva_feridos['Hora'], bins=range(0, 25, 1))
acidentes13 = np.array(counts)/sum(np.array(counts))
fluxo13 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa13 = (acidentes13/fluxo13)/sum(acidentes13/fluxo13)

In [ ]:
graphics_plot_prf(y=[taxa13, fluxo13, acidentes13],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a curve section of the road with at least one person injured (2013) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='dupla_curva_feridos_2013')

**2014**

In [ ]:
df14_dupla.tracado_via.unique()

In [ ]:
np.sort(df14_dupla.feridos.unique())

In [ ]:
# Filtragem 'Curva' + pelo menos 1 ferido
curva = df14_dupla.tracado_via == 'Curva'

feridos = df14_dupla.feridos > 0

df14d_curva_feridos = df14_dupla[(curva) & (feridos)]

In [ ]:
df14d_curva_feridos.tracado_via.unique()

In [ ]:
np.sort(df14d_curva_feridos.feridos.unique())

In [ ]:
counts, bins = np.histogram(df14d_curva_feridos['Hora'], bins=range(0, 25, 1))
acidentes14 = np.array(counts)/sum(np.array(counts))
fluxo14 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa14 = (acidentes14/fluxo14)/sum(acidentes14/fluxo14)

In [ ]:
graphics_plot_prf(y=[taxa14, fluxo14, acidentes14],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a curve section of the road with at least one person injured (2014) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='dupla_curva_feridos_2014')

**2015**

In [ ]:
df15_dupla.tracado_via.unique()

In [ ]:
np.sort(df15_dupla.feridos.unique())

In [ ]:
# Filtragem 'Curva' + pelo menos 1 ferido
curva = df15_dupla.tracado_via == 'Curva'

feridos = df15_dupla.feridos > 0

df15d_curva_feridos = df15_dupla[(curva) & (feridos)]

In [ ]:
df15d_curva_feridos.tracado_via.unique()

In [ ]:
np.sort(df15d_curva_feridos.feridos.unique())

In [ ]:
counts, bins = np.histogram(df15d_curva_feridos['Hora'], bins=range(0, 25, 1))
acidentes15 = np.array(counts)/sum(np.array(counts))
fluxo15 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa15 = (acidentes15/fluxo15)/sum(acidentes15/fluxo15)

In [ ]:
graphics_plot_prf(y=[taxa15, fluxo15, acidentes15],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a curve section of the road with at least one person injured (2015) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='dupla_curva_feridos_2015')

**2016**

In [ ]:
df16_dupla.tracado_via.unique()

In [ ]:
np.sort(df16_dupla.feridos.unique())

In [ ]:
# Filtragem 'Curva' + pelo menos 1 ferido
curva = df16_dupla.tracado_via == 'Curva'

feridos = df16_dupla.feridos > 0

df16d_curva_feridos = df16_dupla[(curva) & (feridos)]

In [ ]:
df16d_curva_feridos.tracado_via.unique()

In [ ]:
np.sort(df16d_curva_feridos.feridos.unique())

In [ ]:
counts, bins = np.histogram(df16d_curva_feridos['Hora'], bins=range(0, 25, 1))
acidentes16 = np.array(counts)/sum(np.array(counts))
fluxo16 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa16 = (acidentes16/fluxo16)/sum(acidentes16/fluxo16)

In [ ]:
graphics_plot_prf(y=[taxa16, fluxo16, acidentes16],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a curve section of the road with at least one person injured (2016) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='dupla_curva_feridos_2016')

**2017**

In [ ]:
df17_dupla.tracado_via.unique()

In [ ]:
np.sort(df17_dupla.feridos.unique())

In [ ]:
# Filtragem 'Curva' + pelo menos 1 ferido
curva = df17_dupla.tracado_via == 'Curva'

feridos = df17_dupla.feridos > 0

df17d_curva_feridos = df17_dupla[(curva) & (feridos)]

In [ ]:
df17d_curva_feridos.tracado_via.unique()

In [ ]:
np.sort(df17d_curva_feridos.feridos.unique())

In [ ]:
counts, bins = np.histogram(df17d_curva_feridos['Hora'], bins=range(0, 25, 1))
acidentes17 = np.array(counts)/sum(np.array(counts))
fluxo17 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa17 = (acidentes17/fluxo17)/sum(acidentes17/fluxo17)

In [ ]:
graphics_plot_prf(y=[taxa17, fluxo17, acidentes17],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a curve section of the road with at least one person injured (2017) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='dupla_curva_feridos_2017')

In [ ]:
# Salvando as taxas
df_taxas['dupla_curva_feridos_13'] = taxa13
df_taxas['dupla_curva_feridos_14'] = taxa14
df_taxas['dupla_curva_feridos_15'] = taxa15
df_taxas['dupla_curva_feridos_16'] = taxa16
df_taxas['dupla_curva_feridos_17'] = taxa17

### **Um veículo + Reta + Curva**

**2013**

In [ ]:
df13_dupla.tracado_via.unique()

In [ ]:
# Filtragem 'Reta' ou 'Curva'
reta_curva = df13_dupla.tracado_via.isin(['Reta', 'Curva'])

df13d_reta_curva = df13_dupla[(reta_curva)]

In [ ]:
df13d_reta_curva.tracado_via.unique()

In [ ]:
counts, bins = np.histogram(df13d_reta_curva['Hora'], bins=range(0, 25, 1))
acidentes13 = np.array(counts)/sum(np.array(counts))
fluxo13 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa13 = (acidentes13/fluxo13)/sum(acidentes13/fluxo13)

In [ ]:
graphics_plot_prf(y=[taxa13, fluxo13, acidentes13],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a straight or curve section of the road (2013) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='dupla_reta_curva_2013')

**2014**

In [ ]:
df14_dupla.tracado_via.unique()

In [ ]:
# Filtragem 'Reta' ou 'Curva'
reta_curva = df14_dupla.tracado_via.isin(['Reta', 'Curva'])

df14d_reta_curva = df14_dupla[(reta_curva)]

In [ ]:
df14d_reta_curva.tracado_via.unique()

In [ ]:
counts, bins = np.histogram(df14d_reta_curva['Hora'], bins=range(0, 25, 1))
acidentes14 = np.array(counts)/sum(np.array(counts))
fluxo14 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa14 = (acidentes14/fluxo14)/sum(acidentes14/fluxo14)

In [ ]:
graphics_plot_prf(y=[taxa14, fluxo14, acidentes14],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a straight or curve section of the road (2014) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='dupla_reta_curva_2014')

**2015**

In [ ]:
df15_dupla.tracado_via.unique()

In [ ]:
# Filtragem 'Reta' ou 'Curva'
reta_curva = df15_dupla.tracado_via.isin(['Reta', 'Curva'])

df15d_reta_curva = df15_dupla[(reta_curva)]

In [ ]:
df15d_reta_curva.tracado_via.unique()

In [ ]:
counts, bins = np.histogram(df15d_reta_curva['Hora'], bins=range(0, 25, 1))
acidentes15 = np.array(counts)/sum(np.array(counts))
fluxo15 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa15 = (acidentes15/fluxo15)/sum(acidentes15/fluxo15)

In [ ]:
graphics_plot_prf(y=[taxa15, fluxo15, acidentes15],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a straight or curve section of the road (2015) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='dupla_reta_curva_2015')

**2016**

In [ ]:
df16_dupla.tracado_via.unique()

In [ ]:
# Filtragem 'Reta' ou 'Curva'
reta_curva = df16_dupla.tracado_via.isin(['Reta', 'Curva'])

df16d_reta_curva = df16_dupla[(reta_curva)]

In [ ]:
df16d_reta_curva.tracado_via.unique()

In [ ]:
counts, bins = np.histogram(df16d_reta_curva['Hora'], bins=range(0, 25, 1))
acidentes16 = np.array(counts)/sum(np.array(counts))
fluxo16 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa16 = (acidentes16/fluxo16)/sum(acidentes16/fluxo16)

In [ ]:
graphics_plot_prf(y=[taxa16, fluxo16, acidentes16],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a straight or curve section of the road (2016) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='dupla_reta_curva_2016')

**2017**

In [ ]:
df17_dupla.tracado_via.unique()

In [ ]:
# Filtragem 'Reta' ou 'Curva'
reta_curva = df17_dupla.tracado_via.isin(['Reta', 'Curva'])

df17d_reta_curva = df17_dupla[(reta_curva)]

In [ ]:
df17d_reta_curva.tracado_via.unique()

In [ ]:
counts, bins = np.histogram(df17d_reta_curva['Hora'], bins=range(0, 25, 1))
acidentes17 = np.array(counts)/sum(np.array(counts))
fluxo17 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa17 = (acidentes17/fluxo17)/sum(acidentes17/fluxo17)

In [ ]:
graphics_plot_prf(y=[taxa17, fluxo17, acidentes17],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a straight or curve section of the road (2017) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='dupla_reta_curva_2017')

In [ ]:
# Salvando as taxas
df_taxas['dupla_reta_curva_13'] = taxa13
df_taxas['dupla_reta_curva_14'] = taxa14
df_taxas['dupla_reta_curva_15'] = taxa15
df_taxas['dupla_reta_curva_16'] = taxa16
df_taxas['dupla_reta_curva_17'] = taxa17

### **Um veículo + Reta + Curva + feridos**

**2013**

In [ ]:
df13_dupla.tracado_via.unique()

In [ ]:
np.sort(df13_dupla.feridos.unique())

In [ ]:
# Filtragem 'Reta' ou 'Curva' + pelo menos 1 ferido
reta_curva = df13_dupla.tracado_via.isin(['Reta', 'Curva'])

feridos = df13_dupla.feridos > 0

df13d_reta_curva_feridos = df13_dupla[(reta_curva) & (feridos)]

In [ ]:
df13d_reta_curva_feridos.tracado_via.unique()

In [ ]:
np.sort(df13d_reta_curva_feridos.feridos.unique())

In [ ]:
counts, bins = np.histogram(df13d_reta_curva_feridos['Hora'], bins=range(0, 25, 1))
acidentes13 = np.array(counts)/sum(np.array(counts))
fluxo13 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa13 = (acidentes13/fluxo13)/sum(acidentes13/fluxo13)

In [ ]:
graphics_plot_prf(y=[taxa13, fluxo13, acidentes13],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a straight or curve section of the road with at least one person injured (2013) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='dupla_reta_curva_feridos_2013')

**2014**

In [ ]:
df14_dupla.tracado_via.unique()

In [ ]:
np.sort(df14_dupla.feridos.unique())

In [ ]:
# Filtragem 'Reta' ou 'Curva' + pelo menos 1 ferido
reta_curva = df14_dupla.tracado_via.isin(['Reta', 'Curva'])

feridos = df14_dupla.feridos > 0

df14d_reta_curva_feridos = df14_dupla[(reta_curva) & (feridos)]

In [ ]:
df14d_reta_curva_feridos.tracado_via.unique()

In [ ]:
np.sort(df14d_reta_curva_feridos.feridos.unique())

In [ ]:
counts, bins = np.histogram(df14d_reta_curva_feridos['Hora'], bins=range(0, 25, 1))
acidentes14 = np.array(counts)/sum(np.array(counts))
fluxo14 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa14 = (acidentes14/fluxo14)/sum(acidentes14/fluxo14)

In [ ]:
graphics_plot_prf(y=[taxa14, fluxo14, acidentes14],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a straight or curve section of the road with at least one person injured (2014) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='dupla_reta_curva_feridos_2014')

**2015**

In [ ]:
df15_dupla.tracado_via.unique()

In [ ]:
np.sort(df15_dupla.feridos.unique())

In [ ]:
# Filtragem 'Reta' ou 'Curva' + pelo menos 1 ferido
reta_curva = df15_dupla.tracado_via.isin(['Reta', 'Curva'])

feridos = df15_dupla.feridos > 0

df15d_reta_curva_feridos = df15_dupla[(reta_curva) & (feridos)]

In [ ]:
df15d_reta_curva_feridos.tracado_via.unique()

In [ ]:
np.sort(df15d_reta_curva_feridos.feridos.unique())

In [ ]:
counts, bins = np.histogram(df15d_reta_curva_feridos['Hora'], bins=range(0, 25, 1))
acidentes15 = np.array(counts)/sum(np.array(counts))
fluxo15 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa15 = (acidentes15/fluxo15)/sum(acidentes15/fluxo15)

In [ ]:
graphics_plot_prf(y=[taxa15, fluxo15, acidentes15],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a straight or curve section of the road with at least one person injured (2015) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='dupla_reta_curva_feridos_2015')

**2016**

In [ ]:
df16_dupla.tracado_via.unique()

In [ ]:
np.sort(df16_dupla.feridos.unique())

In [ ]:
# Filtragem 'Reta' ou 'Curva' + pelo menos 1 ferido
reta_curva = df16_dupla.tracado_via.isin(['Reta', 'Curva'])

feridos = df16_dupla.feridos > 0

df16d_reta_curva_feridos = df16_dupla[(reta_curva) & (feridos)]

In [ ]:
df16d_reta_curva_feridos.tracado_via.unique()

In [ ]:
np.sort(df16d_reta_curva_feridos.feridos.unique())

In [ ]:
counts, bins = np.histogram(df16d_reta_curva_feridos['Hora'], bins=range(0, 25, 1))
acidentes16 = np.array(counts)/sum(np.array(counts))
fluxo16 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa16 = (acidentes16/fluxo16)/sum(acidentes16/fluxo16)

In [ ]:
graphics_plot_prf(y=[taxa16, fluxo16, acidentes16],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a straight or curve section of the road with at least one person injured (2016) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='dupla_reta_curva_feridos_2016')

**2017**

In [ ]:
df17_dupla.tracado_via.unique()

In [ ]:
np.sort(df17_dupla.feridos.unique())

In [ ]:
# Filtragem 'Reta' ou 'Curva' + pelo menos 1 ferido
reta_curva = df17_dupla.tracado_via.isin(['Reta', 'Curva'])

feridos = df17_dupla.feridos > 0

df17d_reta_curva_feridos = df17_dupla[(reta_curva) & (feridos)]

In [ ]:
df17d_reta_curva_feridos.tracado_via.unique()

In [ ]:
np.sort(df17d_reta_curva_feridos.feridos.unique())

In [ ]:
counts, bins = np.histogram(df17d_reta_curva_feridos['Hora'], bins=range(0, 25, 1))
acidentes17 = np.array(counts)/sum(np.array(counts))
fluxo17 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa17 = (acidentes17/fluxo17)/sum(acidentes17/fluxo17)

In [ ]:
graphics_plot_prf(y=[taxa17, fluxo17, acidentes17],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a straight or curve section of the road with at least one person injured (2017) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='dupla_reta_curva_feridos_2017')

In [ ]:
# Salvando as taxas
df_taxas['dupla_reta_curva_feridos_13'] = taxa13
df_taxas['dupla_reta_curva_feridos_14'] = taxa14
df_taxas['dupla_reta_curva_feridos_15'] = taxa15
df_taxas['dupla_reta_curva_feridos_16'] = taxa16
df_taxas['dupla_reta_curva_feridos_17'] = taxa17

## **Reta + 1 veículo + falta de atenção + dormindo**

**2013**

In [ ]:
prf13.tracado_via.unique()

In [ ]:
prf13.causa_acidente.unique()

In [ ]:
# Filtragem 'Reta' + 'Falta de atenção' ou 'Dormindo'
reta = prf13.tracado_via == 'Reta'

list_causa = ['Falta de atenção', 'Dormindo']
causa = prf13.causa_acidente.isin(list_causa)

df13_reta_causa = prf13[(reta) & (causa)]

In [ ]:
df13_reta_causa.tracado_via.unique()

In [ ]:
df13_reta_causa.causa_acidente.unique()

In [ ]:
counts, bins = np.histogram(df13_reta_causa['Hora'], bins=range(0, 25, 1))
acidentes13 = np.array(counts)/sum(np.array(counts))
fluxo13 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa13 = (acidentes13/fluxo13)/sum(acidentes13/fluxo13)

In [ ]:
graphics_plot_prf(y=[taxa13, fluxo13, acidentes13],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a straight section of the road, sleeping or lack of attention (2013) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='reta_dorm_atencao_2013')

**2014**

In [ ]:
prf14.tracado_via.unique()

In [ ]:
prf14.causa_acidente.unique()

In [ ]:
# Filtragem 'Reta' + 'Falta de atenção' ou 'Dormindo'
reta = prf14.tracado_via == 'Reta'

list_causa = ['Falta de atenção', 'Dormindo']
causa = prf14.causa_acidente.isin(list_causa)

df14_reta_causa = prf14[(reta) & (causa)]

In [ ]:
df14_reta_causa.tracado_via.unique()

In [ ]:
df14_reta_causa.causa_acidente.unique()

In [ ]:
counts, bins = np.histogram(df14_reta_causa['Hora'], bins=range(0, 25, 1))
acidentes14 = np.array(counts)/sum(np.array(counts))
fluxo14 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa14 = (acidentes14/fluxo14)/sum(acidentes14/fluxo14)

In [ ]:
graphics_plot_prf(y=[taxa14, fluxo14, acidentes14],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a straight section of the road, sleeping or lack of attention (2014) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='reta_dorm_atencao_2014')

**2015**

In [ ]:
prf15.tracado_via.unique()

In [ ]:
prf15.causa_acidente.unique()

In [ ]:
# Filtragem 'Reta' + 'Falta de atenção' ou 'Dormindo'
reta = prf15.tracado_via == 'Reta'

list_causa = ['Falta de atenção', 'Dormindo']
causa = prf15.causa_acidente.isin(list_causa)

df15_reta_causa = prf15[(reta) & (causa)]

In [ ]:
df15_reta_causa.tracado_via.unique()

In [ ]:
df15_reta_causa.causa_acidente.unique()

In [ ]:
counts, bins = np.histogram(df15_reta_causa['Hora'], bins=range(0, 25, 1))
acidentes15 = np.array(counts)/sum(np.array(counts))
fluxo15 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa15 = (acidentes15/fluxo15)/sum(acidentes15/fluxo15)

In [ ]:
graphics_plot_prf(y=[taxa15, fluxo15, acidentes15],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a straight section of the road, sleeping or lack of attention (2015) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='reta_dorm_atencao_2015')

**2016**

In [ ]:
prf16.tracado_via.unique()

In [ ]:
prf16.causa_acidente.unique()

In [ ]:
# Filtragem 'Reta' + 'Falta de atenção' ou 'Dormindo'
reta = prf16.tracado_via == 'Reta'

list_causa = ['Falta de atenção', 'Dormindo']
causa = prf16.causa_acidente.isin(list_causa)

df16_reta_causa = prf16[(reta) & (causa)]

In [ ]:
df16_reta_causa.tracado_via.unique()

In [ ]:
df16_reta_causa.causa_acidente.unique()

In [ ]:
counts, bins = np.histogram(df16_reta_causa['Hora'], bins=range(0, 25, 1))
acidentes16 = np.array(counts)/sum(np.array(counts))
fluxo16 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa16 = (acidentes16/fluxo16)/sum(acidentes16/fluxo16)

In [ ]:
graphics_plot_prf(y=[taxa16, fluxo16, acidentes16],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a straight section of the road, sleeping or lack of attention (2016) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='reta_dorm_atencao_2016')

**2017**

In [ ]:
prf17.tracado_via.unique()

In [ ]:
prf17.causa_acidente.unique()

In [ ]:
# Filtragem 'Reta' + 'Falta de Atenção à Condução' ou 'Condutor Dormindo'
reta = prf17.tracado_via == 'Reta'

list_causa = ['Falta de Atenção à Condução', 'Condutor Dormindo']
causa = prf17.causa_acidente.isin(list_causa)

df17_reta_causa = prf17[(reta) & (causa)]

In [ ]:
df17_reta_causa.tracado_via.unique()

In [ ]:
df17_reta_causa.causa_acidente.unique()

In [ ]:
counts, bins = np.histogram(df17_reta_causa['Hora'], bins=range(0, 25, 1))
acidentes17 = np.array(counts)/sum(np.array(counts))
fluxo17 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa17 = (acidentes17/fluxo17)/sum(acidentes17/fluxo17)

In [ ]:
graphics_plot_prf(y=[taxa17, fluxo17, acidentes17],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a straight section of the road, sleeping or lack of attention (2017) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='reta_dorm_atencao_2017')

In [ ]:
# Salvando as taxas
df_taxas['reta_dorm_atencao_13'] = taxa13
df_taxas['reta_dorm_atencao_14'] = taxa14
df_taxas['reta_dorm_atencao_15'] = taxa15
df_taxas['reta_dorm_atencao_16'] = taxa16
df_taxas['reta_dorm_atencao_17'] = taxa17

## **Curva + 1 veículo + falta de atenção + dormindo**

**2013**

In [ ]:
prf13.tracado_via.unique()

In [ ]:
prf13.causa_acidente.unique()

In [ ]:
# Filtragem 'Curva' + 'Falta de atenção' ou 'Dormindo'
curva = prf13.tracado_via == 'Curva'

list_causa = ['Falta de atenção', 'Dormindo']
causa = prf13.causa_acidente.isin(list_causa)

df13_curva_causa = prf13[(curva) & (causa)]

In [ ]:
df13_curva_causa.tracado_via.unique()

In [ ]:
df13_curva_causa.causa_acidente.unique()

In [ ]:
counts, bins = np.histogram(df13_curva_causa['Hora'], bins=range(0, 25, 1))
acidentes13 = np.array(counts)/sum(np.array(counts))
fluxo13 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa13 = (acidentes13/fluxo13)/sum(acidentes13/fluxo13)

In [ ]:
graphics_plot_prf(y=[taxa13, fluxo13, acidentes13],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a curve section of the road, sleeping or lack of attention (2013) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='curva_dorm_atencao_2013')

**2014**

In [ ]:
prf14.tracado_via.unique()

In [ ]:
prf14.causa_acidente.unique()

In [ ]:
# Filtragem 'Curva' + 'Falta de atenção' ou 'Dormindo'
curva = prf14.tracado_via == 'Curva'

list_causa = ['Falta de atenção', 'Dormindo']
causa = prf14.causa_acidente.isin(list_causa)

df14_curva_causa = prf14[(curva) & (causa)]

In [ ]:
df14_curva_causa.tracado_via.unique()

In [ ]:
df14_curva_causa.causa_acidente.unique()

In [ ]:
counts, bins = np.histogram(df14_curva_causa['Hora'], bins=range(0, 25, 1))
acidentes14 = np.array(counts)/sum(np.array(counts))
fluxo14 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa14 = (acidentes14/fluxo14)/sum(acidentes14/fluxo14)

In [ ]:
graphics_plot_prf(y=[taxa14, fluxo14, acidentes14],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a curve section of the road, sleeping or lack of attention (2014) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='curva_dorm_atencao_2014')

**2015**

In [ ]:
prf15.tracado_via.unique()

In [ ]:
prf15.causa_acidente.unique()

In [ ]:
# Filtragem 'Curva' + 'Falta de atenção' ou 'Dormindo'
curva = prf15.tracado_via == 'Curva'

list_causa = ['Falta de atenção', 'Dormindo']
causa = prf15.causa_acidente.isin(list_causa)

df15_curva_causa = prf15[(curva) & (causa)]

In [ ]:
df15_curva_causa.tracado_via.unique()

In [ ]:
df15_curva_causa.causa_acidente.unique()

In [ ]:
counts, bins = np.histogram(df15_curva_causa['Hora'], bins=range(0, 25, 1))
acidentes15 = np.array(counts)/sum(np.array(counts))
fluxo15 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa15 = (acidentes15/fluxo15)/sum(acidentes15/fluxo15)

In [ ]:
graphics_plot_prf(y=[taxa15, fluxo15, acidentes15],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a curve section of the road, sleeping or lack of attention (2015) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='curva_dorm_atencao_2015')

**2016**

In [ ]:
prf16.tracado_via.unique()

In [ ]:
prf16.causa_acidente.unique()

In [ ]:
# Filtragem 'Curva' + 'Falta de atenção' ou 'Dormindo'
curva = prf16.tracado_via == 'Curva'

list_causa = ['Falta de atenção', 'Dormindo']
causa = prf16.causa_acidente.isin(list_causa)

df16_curva_causa = prf16[(curva) & (causa)]

In [ ]:
df16_curva_causa.tracado_via.unique()

In [ ]:
df16_curva_causa.causa_acidente.unique()

In [ ]:
counts, bins = np.histogram(df16_curva_causa['Hora'], bins=range(0, 25, 1))
acidentes16 = np.array(counts)/sum(np.array(counts))
fluxo16 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa16 = (acidentes16/fluxo16)/sum(acidentes16/fluxo16)

In [ ]:
graphics_plot_prf(y=[taxa16, fluxo16, acidentes16],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a curve section of the road, sleeping or lack of attention (2016) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='curva_dorm_atencao_2016')

**2017**

In [ ]:
prf17.tracado_via.unique()

In [ ]:
prf17.causa_acidente.unique()

In [ ]:
# Filtragem 'Curva' + 'Falta de Atenção à Condução' ou 'Condutor Dormindo'
curva = prf17.tracado_via == 'Curva'

list_causa = ['Falta de Atenção à Condução', 'Condutor Dormindo']
causa = prf17.causa_acidente.isin(list_causa)

df17_curva_causa = prf17[(curva) & (causa)]

In [ ]:
df17_curva_causa.tracado_via.unique()

In [ ]:
df17_curva_causa.causa_acidente.unique()

In [ ]:
counts, bins = np.histogram(df17_curva_causa['Hora'], bins=range(0, 25, 1))
acidentes17 = np.array(counts)/sum(np.array(counts))
fluxo17 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa17 = (acidentes17/fluxo17)/sum(acidentes17/fluxo17)

In [ ]:
graphics_plot_prf(y=[taxa17, fluxo17, acidentes17],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a curve section of the road, sleeping or lack of attention (2017) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='curva_dorm_atencao_2017')

In [ ]:
# Salvando as taxas
df_taxas['curva_dorm_atencao_13'] = taxa13
df_taxas['curva_dorm_atencao_14'] = taxa14
df_taxas['curva_dorm_atencao_15'] = taxa15
df_taxas['curva_dorm_atencao_16'] = taxa16
df_taxas['curva_dorm_atencao_17'] = taxa17

## **Curva + Reta + 1 veículo + falta de atenção + dormindo**

**2013**

In [ ]:
prf13.tracado_via.unique()

In [ ]:
prf13.causa_acidente.unique()

In [ ]:
# Filtragem 'Reta' ou 'Curva' + 'Falta de atenção' ou 'Dormindo'
reta_curva = prf13.tracado_via.isin(['Reta', 'Curva'])

list_causa = ['Falta de atenção', 'Dormindo']
causa = prf13.causa_acidente.isin(list_causa)

df13_reta_curva_causa = prf13[(reta_curva) & (causa)]

In [ ]:
df13_reta_curva_causa.tracado_via.unique()

In [ ]:
df13_reta_curva_causa.causa_acidente.unique()

In [ ]:
counts, bins = np.histogram(df13_reta_curva_causa['Hora'], bins=range(0, 25, 1))
acidentes13 = np.array(counts)/sum(np.array(counts))
fluxo13 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa13 = (acidentes13/fluxo13)/sum(acidentes13/fluxo13)

In [ ]:
graphics_plot_prf(y=[taxa13, fluxo13, acidentes13],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a straight or curve section of the road, sleeping or lack of attention (2013) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='reta_curva_dorm_atencao_2013')

**2014**

In [ ]:
prf14.tracado_via.unique()

In [ ]:
prf14.causa_acidente.unique()

In [ ]:
# Filtragem 'Reta' ou 'Curva' + 'Falta de atenção' ou 'Dormindo'
reta_curva = prf14.tracado_via.isin(['Reta', 'Curva'])

list_causa = ['Falta de atenção', 'Dormindo']
causa = prf14.causa_acidente.isin(list_causa)

df14_reta_curva_causa = prf14[(reta_curva) & (causa)]

In [ ]:
df14_reta_curva_causa.tracado_via.unique()

In [ ]:
df14_reta_curva_causa.causa_acidente.unique()

In [ ]:
counts, bins = np.histogram(df14_reta_curva_causa['Hora'], bins=range(0, 25, 1))
acidentes14 = np.array(counts)/sum(np.array(counts))
fluxo14 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa14 = (acidentes14/fluxo14)/sum(acidentes14/fluxo14)

In [ ]:
graphics_plot_prf(y=[taxa14, fluxo14, acidentes14],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a straight or curve section of the road, sleeping or lack of attention (2014) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='reta_curva_dorm_atencao_2014')

**2015**

In [ ]:
prf15.tracado_via.unique()

In [ ]:
prf15.causa_acidente.unique()

In [ ]:
# Filtragem 'Reta' ou 'Curva' + 'Falta de atenção' ou 'Dormindo'
reta_curva = prf15.tracado_via.isin(['Reta', 'Curva'])

list_causa = ['Falta de atenção', 'Dormindo']
causa = prf15.causa_acidente.isin(list_causa)

df15_reta_curva_causa = prf15[(reta_curva) & (causa)]

In [ ]:
df15_reta_curva_causa.tracado_via.unique()

In [ ]:
df15_reta_curva_causa.causa_acidente.unique()

In [ ]:
counts, bins = np.histogram(df15_reta_curva_causa['Hora'], bins=range(0, 25, 1))
acidentes15 = np.array(counts)/sum(np.array(counts))
fluxo15 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa15 = (acidentes15/fluxo15)/sum(acidentes15/fluxo15)

In [ ]:
graphics_plot_prf(y=[taxa15, fluxo15, acidentes15],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a straight or curve section of the road, sleeping or lack of attention (2015) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='reta_curva_dorm_atencao_2015')

**2016**

In [ ]:
prf16.tracado_via.unique()

In [ ]:
prf16.causa_acidente.unique()

In [ ]:
# Filtragem 'Reta' ou 'Curva' + 'Falta de atenção' ou 'Dormindo'
reta_curva = prf16.tracado_via.isin(['Reta', 'Curva'])

list_causa = ['Falta de atenção', 'Dormindo']
causa = prf16.causa_acidente.isin(list_causa)

df16_reta_curva_causa = prf16[(reta_curva) & (causa)]

In [ ]:
df16_reta_curva_causa.tracado_via.unique()

In [ ]:
df16_reta_curva_causa.causa_acidente.unique()

In [ ]:
counts, bins = np.histogram(df16_reta_curva_causa['Hora'], bins=range(0, 25, 1))
acidentes16 = np.array(counts)/sum(np.array(counts))
fluxo16 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa16 = (acidentes16/fluxo16)/sum(acidentes16/fluxo16)

In [ ]:
graphics_plot_prf(y=[taxa16, fluxo16, acidentes16],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a straight or curve section of the road, sleeping or lack of attention (2016) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='reta_curva_dorm_atencao_2016')

**2017**

In [ ]:
prf17.tracado_via.unique()

In [ ]:
prf17.causa_acidente.unique()

In [ ]:
# Filtragem 'Reta' ou 'Curva' + 'Falta de Atenção à Condução' ou 'Condutor Dormindo'
reta_curva = prf17.tracado_via.isin(['Reta', 'Curva'])

list_causa = ['Falta de Atenção à Condução', 'Condutor Dormindo']
causa = prf17.causa_acidente.isin(list_causa)

df17_reta_curva_causa = prf17[(reta_curva) & (causa)]

In [ ]:
df17_reta_curva_causa.tracado_via.unique()

In [ ]:
df17_reta_curva_causa.causa_acidente.unique()

In [ ]:
counts, bins = np.histogram(df17_reta_curva_causa['Hora'], bins=range(0, 25, 1))
acidentes17 = np.array(counts)/sum(np.array(counts))
fluxo17 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa17 = (acidentes17/fluxo17)/sum(acidentes17/fluxo17)

In [ ]:
graphics_plot_prf(y=[taxa17, fluxo17, acidentes17],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks on a straight or curve section of the road, sleeping or lack of attention (2017) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='reta_curva_dorm_atencao_2017')

In [ ]:
# Salvando as taxas
df_taxas['reta_curva_dorm_atencao_13'] = taxa13
df_taxas['reta_curva_dorm_atencao_14'] = taxa14
df_taxas['reta_curva_dorm_atencao_15'] = taxa15
df_taxas['reta_curva_dorm_atencao_16'] = taxa16
df_taxas['reta_curva_dorm_atencao_17'] = taxa17

## **DataFrame taxas**

In [ ]:
# df_taxas.to_csv('taxas.csv', index=False)

In [ ]:
!gdown 1C3J0MBLPZj4n9a_SvObcD3pR9cUh9kCZ

In [ ]:
df_taxas = pd.read_csv('taxas.csv')
print(df_taxas.shape)
df_taxas.head(3)

In [ ]:
df_max = df_taxas.copy()
cols = df_max.iloc[:,1:].columns

for col in cols:
    df_max[col] = [1 if x == df_max[col].max() else 0 for x in df_max[col]]

df_max['max'] = df_max.iloc[:,1:].sum(axis=1)
df_max.head(3)

In [ ]:
df_max['max']

In [ ]:
import plotly.express as px

fig = px.histogram(df_max, x='hora', y='max', nbins=24,
                   title='Accident rate peak per hour')
fig.update_layout(xaxis=dict(tickvals=df_max.hora),
                  xaxis_title='Hour',
                  yaxis_title='Occurrences')
fig.show()

In [ ]:
df_max2 = df_taxas.copy()
cols = df_max2.iloc[:,1:].columns

for col in cols:
    df_max2[col] = [1 if x == df_max2[col].nlargest(2).values[-1] else 0 for x in df_max2[col]]

df_max2['max2'] = df_max2.iloc[:,1:].sum(axis=1)
df_max2.head(3)

In [ ]:
df_max2['max2']

In [ ]:
import plotly.express as px

fig = px.histogram(df_max2, x='hora', y='max2', nbins=24,
                   title='Accident rate second peak per hour')
fig.update_layout(xaxis = dict(tickvals=df_max2.hora),
                  xaxis_title='Hour',
                  yaxis_title='Occurrences')
fig.show()

In [ ]:
df_max3 = df_taxas.copy()
cols = df_max3.iloc[:,1:].columns

for col in cols:
    df_max3[col] = [1 if x == df_max3[col].nlargest(3).values[-1] else 0 for x in df_max3[col]]

df_max3['max3'] = df_max3.iloc[:,1:].sum(axis=1)
df_max3.head(3)

In [ ]:
df_max3['max3']

In [ ]:
import plotly.express as px

fig = px.histogram(df_max3, x='hora', y='max3', nbins=24,
                   title='Accident rate third peak per hour')
fig.update_layout(xaxis=dict(tickvals=df_max3.hora),
                  xaxis_title='Hour',
                  yaxis_title='Occurrences')
fig.show()

In [ ]:
df_taxas_transp = df_taxas.set_index('hora').T
df_taxas_transp.head(2)

In [ ]:
mean_7am_7pm = df_taxas_transp.iloc[:,6:19].values.mean()
mean_7am_7pm

In [ ]:
import plotly.graph_objects as go

mean_list = []

fig = go.Figure()
for col in range(1, 25):
    fig.add_trace(go.Box(y=df_taxas_transp[col]/mean_7am_7pm, name=col))
    mean_list.append(df_taxas_transp[col].mean()/mean_7am_7pm)

fig.add_trace(go.Scatter(x=df_taxas_transp.columns.values, y=mean_list,
                         line = dict(color='gray', dash='dot')))
fig.update_layout(showlegend=False,
                  xaxis_title='Hour',
                  yaxis_title='Rate/mean_7am_7pm',
                  title='Accidents rate, divided by rate mean between 7am and 7pm, per hour')
fig.show()

In [ ]:
# Download images folder
# !zip -r /content/images.zip /content/images

# from google.colab import files
# files.download('/content/images.zip')

# **Outras análises**

1.   Todos os veículos, em linha reta, um único veículo, tombamento
2.   Todos os veículos, em curva, um único veículo, tombamento
3.   Todos os veículos, em linha reta, um único veículo, capotamento
4.   Todos os veículos, em curva, um único veículo, capotamento

In [ ]:
def filtro_tracado_acidente(df, tipo_tracado, tipo_acidente):

    via = df.tracado_via.isin([tipo_tracado])
    acid = df.tipo_acidente.isin([tipo_acidente])

    return df[(via) & (acid)]

## **Dados**

In [ ]:
%%capture

# Fluxo
!gdown 1Kru8e3M6AjhEDuVXH7Tqx36wZjz4-Xc0

dfFluxo = pd.read_csv('DNIT.csv')
dfDNIT = pd.DataFrame(dfFluxo['avgtotal'].loc[2::3].values.reshape(4,24)).T

In [ ]:
%%capture
# !gdown 1rLrFckRbr_oScvziSCodjC2_UxoEOAMk # PRF 2013 - 1 veículo
# !gdown 1--JAT8gfd9CBtIlNBBj5yB_C5ipq9p0u # PRF 2014 - 1 veículo
!gdown 1-4vkJOgGW9W__P9zFhbVQkT3Uarwmbbv # PRF 2015 - 1 veículo
!gdown 1-6cj1H9JsUtdIZtO78x3HhdSFjpsqPj_ # PRF 2016 - 1 veículo
!gdown 1-6tc2J-omr-39k9n3FWApTqnJrZFIbnp # PRF 2017 - 1 veículo

# Datasets já filtrados para veículo único
# df_prf13 = pd.read_csv('prf_2013_1veiculo.csv')
# df_prf13['Hora'] = pd.to_datetime(df_prf13['horario'], format='%H:%M:%S').dt.hour
# df_prf14 = pd.read_csv('prf_2014_1veiculo.csv')
# df_prf14['Hora'] = pd.to_datetime(df_prf14['horario'], format='%H:%M:%S').dt.hour
df_prf15 = pd.read_csv('prf_2015_1veiculo.csv')
df_prf15['Hora'] = pd.to_datetime(df_prf15['horario'], format='%H:%M:%S').dt.hour
df_prf16 = pd.read_csv('prf_2016_1veiculo.csv')
df_prf16['Hora'] = pd.to_datetime(df_prf16['horario'], format='%H:%M:%S').dt.hour
df_prf17 = pd.read_csv('prf_2017_1veiculo.csv')
df_prf17['Hora'] = pd.to_datetime(df_prf17['horario'], format='%H:%M:%S').dt.hour

## **1. Todos os veículos, em linha reta, um único veículo, tombamento**

**2013**

In [ ]:
# Filtragem 'Reta' + 'Tombamento'
df13_reta_tomb = filtro_tracado_acidente(df_prf13, 'Reta', 'Tombamento')

In [ ]:
df13_reta_tomb.tracado_via.unique()

array(['Reta'], dtype=object)

In [ ]:
df13_reta_tomb.tipo_acidente.unique()

array(['Tombamento'], dtype=object)

In [ ]:
counts, bins = np.histogram(df13_reta_tomb['Hora'], bins=range(0, 25, 1))
acidentes13 = np.array(counts)/sum(np.array(counts))
fluxo13 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa13 = (acidentes13/fluxo13)/sum(acidentes13/fluxo13)

In [ ]:
graphics_plot_prf(y=[taxa13, fluxo13, acidentes13],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Overturning of one vehicle on a straight section of the road (2013) - Accidents, traffic and accidents / traffic',
                  x_tick=4, save=True, save_name='reta_tomb_2013')

**2014**

In [ ]:
# Filtragem 'Reta' + 'Tombamento'
df14_reta_tomb = filtro_tracado_acidente(df_prf14, 'Reta', 'Tombamento')

In [ ]:
df14_reta_tomb.tracado_via.unique()

array(['Reta'], dtype=object)

In [ ]:
df14_reta_tomb.tipo_acidente.unique()

array(['Tombamento'], dtype=object)

In [ ]:
counts, bins = np.histogram(df14_reta_tomb['Hora'], bins=range(0, 25, 1))
acidentes14 = np.array(counts)/sum(np.array(counts))
fluxo14 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa14 = (acidentes14/fluxo14)/sum(acidentes14/fluxo14)

In [ ]:
graphics_plot_prf(y=[taxa14, fluxo14, acidentes14],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Overturning of one vehicle on a straight section of the road (2014) - Accidents, traffic and accidents / traffic',
                  x_tick=4, save=True, save_name='reta_tomb_2014')

**2015**

In [ ]:
# Filtragem 'Reta' + 'Tombamento'
df15_reta_tomb = filtro_tracado_acidente(df_prf15, 'Reta', 'Tombamento')

In [ ]:
df15_reta_tomb.tracado_via.unique()

array(['Reta'], dtype=object)

In [ ]:
df15_reta_tomb.tipo_acidente.unique()

array(['Tombamento'], dtype=object)

In [ ]:
counts, bins = np.histogram(df15_reta_tomb['Hora'], bins=range(0, 25, 1))
acidentes15 = np.array(counts)/sum(np.array(counts))
fluxo15 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa15 = (acidentes15/fluxo15)/sum(acidentes15/fluxo15)

In [ ]:
graphics_plot_prf(y=[taxa15, fluxo15, acidentes15],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Overturning of one vehicle on a straight section of the road (2015) - Accidents, traffic and accidents / traffic',
                  x_tick=4, save=True, save_name='reta_tomb_2015')

**2016**

In [ ]:
# Filtragem 'Reta' + 'Tombamento'
df16_reta_tomb = filtro_tracado_acidente(df_prf16, 'Reta', 'Tombamento')

In [ ]:
df16_reta_tomb.tracado_via.unique()

array(['Reta'], dtype=object)

In [ ]:
df16_reta_tomb.tipo_acidente.unique()

array(['Tombamento'], dtype=object)

In [ ]:
counts, bins = np.histogram(df16_reta_tomb['Hora'], bins=range(0, 25, 1))
acidentes16 = np.array(counts)/sum(np.array(counts))
fluxo16 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa16 = (acidentes16/fluxo16)/sum(acidentes16/fluxo16)

In [ ]:
graphics_plot_prf(y=[taxa16, fluxo16, acidentes16],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Overturning of one vehicle on a straight section of the road (2016) - Accidents, traffic and accidents / traffic',
                  x_tick=4, save=True, save_name='reta_tomb_2016')

**2017**

In [ ]:
# Filtragem 'Reta' + 'Tombamento'
df17_reta_tomb = filtro_tracado_acidente(df_prf17, 'Reta', 'Tombamento')

In [ ]:
df17_reta_tomb.tracado_via.unique()

array(['Reta'], dtype=object)

In [ ]:
df17_reta_tomb.tipo_acidente.unique()

array(['Tombamento'], dtype=object)

In [ ]:
counts, bins = np.histogram(df17_reta_tomb['Hora'], bins=range(0, 25, 1))
acidentes17 = np.array(counts)/sum(np.array(counts))
fluxo17= np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa17 = (acidentes17/fluxo17)/sum(acidentes17/fluxo17)

In [ ]:
graphics_plot_prf(y=[taxa17, fluxo17, acidentes17],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Overturning of one vehicle on a straight section of the road (2017) - Accidents, traffic and accidents / traffic',
                  x_tick=4, save=True, save_name='reta_tomb_2017')

In [ ]:
df_novas_taxas = pd.DataFrame(data=range(1,25), columns=['hora'])
# df_novas_taxas['reta_tomb_13'] = taxa13
# df_novas_taxas['reta_tomb_14'] = taxa14
df_novas_taxas['reta_tomb_15'] = taxa15
df_novas_taxas['reta_tomb_16'] = taxa16
df_novas_taxas['reta_tomb_17'] = taxa17

df_novas_taxas.head(3)

,hora,reta_tomb_15,reta_tomb_16,reta_tomb_17
0,1,0.061583,0.066493,0.071589
1,2,0.079076,0.090659,0.071237
2,3,0.086799,0.114448,0.081742


### **Prinicipais razões de acidentes por hora**

**2013**

In [ ]:
df13_reta_tomb.causa_acidente.unique()

array(['Defeito mecânico em veículo', 'Velocidade incompatível',
       'Animais na Pista', 'Outras', 'Falta de atenção',
       'Não guardar distância de segurança', 'Defeito na via', 'Dormindo',
       'Ultrapassagem indevida', 'Desobediência à sinalização',
       'Ingestão de álcool'], dtype=object)

In [ ]:
df13_reta_tomb_acids = pd.DataFrame()
for hora in range(24):
    top5 = df13_reta_tomb[df13_reta_tomb.Hora == hora]['causa_acidente'].value_counts(normalize=True).head(5)
    df13_reta_tomb_acids[f'causa_reta_tomb_{hora}h'] = top5.index
    df13_reta_tomb_acids[f'%_reta_tomb_{hora}h'] = top5.values

df13_reta_tomb_acids

,causa_reta_tomb_0h,%_reta_tomb_0h,causa_reta_tomb_1h,%_reta_tomb_1h,causa_reta_tomb_2h,%_reta_tomb_2h,causa_reta_tomb_3h,%_reta_tomb_3h,causa_reta_tomb_4h,%_reta_tomb_4h,...,causa_reta_tomb_19h,%_reta_tomb_19h,causa_reta_tomb_20h,%_reta_tomb_20h,causa_reta_tomb_21h,%_reta_tomb_21h,causa_reta_tomb_22h,%_reta_tomb_22h,causa_reta_tomb_23h,%_reta_tomb_23h
0,Outras,0.352941,Outras,0.368421,Outras,0.355263,Outras,0.359551,Outras,0.259615,...,Outras,0.388060,Outras,0.365854,Outras,0.381356,Outras,0.412281,Outras,0.357143
1,Dormindo,0.141176,Dormindo,0.171053,Falta de atenção,0.236842,Dormindo,0.224719,Dormindo,0.240385,...,Falta de atenção,0.208955,Falta de atenção,0.186992,Falta de atenção,0.152542,Falta de atenção,0.157895,Falta de atenção,0.193878
2,Ingestão de álcool,0.129412,Falta de atenção,0.131579,Dormindo,0.184211,Falta de atenção,0.179775,Falta de atenção,0.192308,...,Defeito na via,0.104478,Defeito na via,0.146341,Velocidade incompatível,0.118644,Defeito mecânico em veículo,0.105263,Defeito na via,0.102041
3,Falta de atenção,0.129412,Velocidade incompatível,0.105263,Velocidade incompatível,0.092105,Ingestão de álcool,0.056180,Velocidade incompatível,0.096154,...,Velocidade incompatível,0.097015,Defeito mecânico em veículo,0.121951,Defeito mecânico em veículo,0.101695,Dormindo,0.078947,Dormindo,0.081633
4,Defeito mecânico em veículo,0.105882,Defeito mecânico em veículo,0.092105,Defeito mecânico em veículo,0.065789,Defeito na via,0.056180,Defeito mecânico em veículo,0.076923,...,Ingestão de álcool,0.074627,Velocidade incompatível,0.048780,Ingestão de álcool,0.076271,Defeito na via,0.070175,Velocidade incompatível,0.081633


In [ ]:
# df13_reta_tomb_acids.to_csv('df13_reta_tomb_acids.csv', index=False,
#                             encoding='ISO-8859-1', sep=';')

**2014**

In [ ]:
df14_reta_tomb.causa_acidente.unique()

array(['Defeito mecânico em veículo', 'Dormindo', 'Outras',
       'Defeito na via', 'Velocidade incompatível', 'Falta de atenção',
       'Ingestão de álcool', 'Desobediência à sinalização',
       'Não guardar distância de segurança', 'Ultrapassagem indevida',
       'Animais na Pista'], dtype=object)

In [ ]:
df14_reta_tomb_acids = pd.DataFrame()
for hora in range(24):
    top5 = df14_reta_tomb[df14_reta_tomb.Hora == hora]['causa_acidente'].value_counts(normalize=True).head(5)
    df14_reta_tomb_acids[f'causa_reta_tomb_{hora}h'] = top5.index
    df14_reta_tomb_acids[f'%_reta_tomb_{hora}h'] = top5.values

df14_reta_tomb_acids

,causa_reta_tomb_0h,%_reta_tomb_0h,causa_reta_tomb_1h,%_reta_tomb_1h,causa_reta_tomb_2h,%_reta_tomb_2h,causa_reta_tomb_3h,%_reta_tomb_3h,causa_reta_tomb_4h,%_reta_tomb_4h,...,causa_reta_tomb_19h,%_reta_tomb_19h,causa_reta_tomb_20h,%_reta_tomb_20h,causa_reta_tomb_21h,%_reta_tomb_21h,causa_reta_tomb_22h,%_reta_tomb_22h,causa_reta_tomb_23h,%_reta_tomb_23h
0,Dormindo,0.243590,Outras,0.290698,Outras,0.315789,Dormindo,0.375000,Outras,0.376471,...,Outras,0.385621,Outras,0.352941,Outras,0.323810,Outras,0.373832,Outras,0.366337
1,Outras,0.230769,Dormindo,0.232558,Defeito mecânico em veículo,0.157895,Outras,0.339286,Falta de atenção,0.164706,...,Falta de atenção,0.196078,Falta de atenção,0.134454,Falta de atenção,0.152381,Falta de atenção,0.168224,Dormindo,0.168317
2,Falta de atenção,0.205128,Falta de atenção,0.139535,Dormindo,0.140351,Falta de atenção,0.160714,Dormindo,0.141176,...,Defeito na via,0.156863,Defeito na via,0.126050,Velocidade incompatível,0.114286,Dormindo,0.121495,Falta de atenção,0.118812
3,Defeito na via,0.102564,Defeito mecânico em veículo,0.104651,Falta de atenção,0.122807,Velocidade incompatível,0.035714,Defeito na via,0.082353,...,Defeito mecânico em veículo,0.117647,Defeito mecânico em veículo,0.117647,Defeito mecânico em veículo,0.104762,Defeito mecânico em veículo,0.102804,Animais na Pista,0.089109
4,Defeito mecânico em veículo,0.076923,Ingestão de álcool,0.069767,Velocidade incompatível,0.105263,Ingestão de álcool,0.035714,Ingestão de álcool,0.058824,...,Velocidade incompatível,0.058824,Velocidade incompatível,0.084034,Ingestão de álcool,0.085714,Defeito na via,0.084112,Defeito na via,0.079208


In [ ]:
# df14_reta_tomb_acids.to_csv('df14_reta_tomb_acids.csv', index=False,
#                             encoding='ISO-8859-1', sep=';')

**2015**

In [ ]:
df15_reta_tomb.causa_acidente.unique()

array(['Outras', 'Defeito mecânico em veículo', 'Ingestão de álcool',
       'Dormindo', 'Velocidade incompatível', 'Defeito na via',
       'Falta de atenção', 'Ultrapassagem indevida',
       'Desobediência à sinalização',
       'Não guardar distância de segurança', 'Animais na Pista'],
      dtype=object)

In [ ]:
df15_reta_tomb_acids = pd.DataFrame()
for hora in range(24):
    top5 = df15_reta_tomb[df15_reta_tomb.Hora == hora]['causa_acidente'].value_counts(normalize=True).head(5)
    df15_reta_tomb_acids[f'causa_reta_tomb_{hora}h'] = top5.index
    df15_reta_tomb_acids[f'%_reta_tomb_{hora}h'] = top5.values

df15_reta_tomb_acids

,causa_reta_tomb_0h,%_reta_tomb_0h,causa_reta_tomb_1h,%_reta_tomb_1h,causa_reta_tomb_2h,%_reta_tomb_2h,causa_reta_tomb_3h,%_reta_tomb_3h,causa_reta_tomb_4h,%_reta_tomb_4h,...,causa_reta_tomb_19h,%_reta_tomb_19h,causa_reta_tomb_20h,%_reta_tomb_20h,causa_reta_tomb_21h,%_reta_tomb_21h,causa_reta_tomb_22h,%_reta_tomb_22h,causa_reta_tomb_23h,%_reta_tomb_23h
0,Dormindo,0.194030,Outras,0.281250,Outras,0.344828,Outras,0.289855,Outras,0.406977,...,Outras,0.333333,Outras,0.329545,Outras,0.336634,Outras,0.280899,Outras,0.303371
1,Falta de atenção,0.194030,Dormindo,0.265625,Dormindo,0.224138,Dormindo,0.246377,Dormindo,0.232558,...,Falta de atenção,0.192982,Falta de atenção,0.261364,Defeito mecânico em veículo,0.188119,Falta de atenção,0.213483,Falta de atenção,0.191011
2,Outras,0.164179,Falta de atenção,0.171875,Falta de atenção,0.172414,Falta de atenção,0.130435,Falta de atenção,0.116279,...,Defeito mecânico em veículo,0.131579,Defeito na via,0.147727,Falta de atenção,0.158416,Dormindo,0.123596,Dormindo,0.179775
3,Defeito mecânico em veículo,0.119403,Defeito mecânico em veículo,0.078125,Defeito mecânico em veículo,0.086207,Velocidade incompatível,0.101449,Defeito na via,0.093023,...,Velocidade incompatível,0.087719,Defeito mecânico em veículo,0.136364,Velocidade incompatível,0.128713,Defeito mecânico em veículo,0.112360,Velocidade incompatível,0.112360
4,Ingestão de álcool,0.119403,Velocidade incompatível,0.078125,Velocidade incompatível,0.051724,Defeito mecânico em veículo,0.086957,Velocidade incompatível,0.069767,...,Defeito na via,0.078947,Velocidade incompatível,0.068182,Defeito na via,0.059406,Velocidade incompatível,0.101124,Defeito na via,0.067416


In [ ]:
# df15_reta_tomb_acids.to_csv('df15_reta_tomb_acids.csv', index=False,
#                             encoding='ISO-8859-1', sep=';')

**2016**

In [ ]:
df16_reta_tomb.causa_acidente.unique()

array(['Outras', 'Velocidade incompatível', 'Dormindo',
       'Falta de atenção', 'Defeito mecânico em veículo',
       'Defeito na via', 'Ingestão de álcool',
       'Não guardar distância de segurança',
       'Desobediência à sinalização', 'Animais na Pista',
       'Ultrapassagem indevida'], dtype=object)

In [ ]:
df16_reta_tomb_acids = pd.DataFrame()
for hora in range(24):
    top5 = df16_reta_tomb[df16_reta_tomb.Hora == hora]['causa_acidente'].value_counts(normalize=True).head(5)
    df16_reta_tomb_acids[f'causa_reta_tomb_{hora}h'] = top5.index
    df16_reta_tomb_acids[f'%_reta_tomb_{hora}h'] = top5.values

df16_reta_tomb_acids

,causa_reta_tomb_0h,%_reta_tomb_0h,causa_reta_tomb_1h,%_reta_tomb_1h,causa_reta_tomb_2h,%_reta_tomb_2h,causa_reta_tomb_3h,%_reta_tomb_3h,causa_reta_tomb_4h,%_reta_tomb_4h,...,causa_reta_tomb_19h,%_reta_tomb_19h,causa_reta_tomb_20h,%_reta_tomb_20h,causa_reta_tomb_21h,%_reta_tomb_21h,causa_reta_tomb_22h,%_reta_tomb_22h,causa_reta_tomb_23h,%_reta_tomb_23h
0,Outras,0.300000,Outras,0.338028,Dormindo,0.351351,Outras,0.354839,Dormindo,0.3500,...,Outras,0.412371,Outras,0.292683,Outras,0.325301,Outras,0.304878,Outras,0.361702
1,Dormindo,0.228571,Dormindo,0.183099,Outras,0.175676,Dormindo,0.225806,Outras,0.3375,...,Falta de atenção,0.206186,Falta de atenção,0.195122,Velocidade incompatível,0.156627,Dormindo,0.182927,Falta de atenção,0.191489
2,Falta de atenção,0.185714,Falta de atenção,0.140845,Falta de atenção,0.162162,Falta de atenção,0.129032,Velocidade incompatível,0.1125,...,Defeito mecânico em veículo,0.113402,Defeito mecânico em veículo,0.170732,Defeito mecânico em veículo,0.132530,Falta de atenção,0.170732,Dormindo,0.148936
3,Velocidade incompatível,0.128571,Velocidade incompatível,0.098592,Velocidade incompatível,0.081081,Velocidade incompatível,0.080645,Falta de atenção,0.0750,...,Defeito na via,0.103093,Defeito na via,0.109756,Ingestão de álcool,0.096386,Ingestão de álcool,0.097561,Ingestão de álcool,0.074468
4,Defeito na via,0.057143,Ingestão de álcool,0.098592,Ingestão de álcool,0.081081,Defeito mecânico em veículo,0.080645,Defeito mecânico em veículo,0.0375,...,Velocidade incompatível,0.072165,Velocidade incompatível,0.085366,Falta de atenção,0.096386,Velocidade incompatível,0.085366,Defeito mecânico em veículo,0.074468


In [ ]:
# df16_reta_tomb_acids.to_csv('df16_reta_tomb_acids.csv', index=False,
#                             encoding='ISO-8859-1', sep=';')

**2017**

In [ ]:
df17_reta_tomb.causa_acidente.unique()

array(['Defeito na Via', 'Condutor Dormindo',
       'Carga excessiva e/ou mal acondicionada',
       'Falta de Atenção à Condução',
       'Avarias e/ou desgaste excessivo no pneu',
       'Defeito Mecânico no Veículo', 'Velocidade Incompatível',
       'Ultrapassagem Indevida', 'Pista Escorregadia', 'Mal Súbito',
       'Ingestão de Álcool', 'Animais na Pista',
       'Restrição de Visibilidade', 'Não guardar distância de segurança',
       'Fenômenos da Natureza',
       'Desobediência às normas de trânsito pelo condutor',
       'Sinalização da via insuficiente ou inadequada',
       'Objeto estático sobre o leito carroçável',
       'Falta de Atenção do Pedestre',
       'Ingestão de Substâncias Psicoativas',
       'Deficiência ou não Acionamento do Sistema de Iluminação/Sinalização do Veículo'],
      dtype=object)

In [ ]:
df17_reta_tomb_acids = pd.DataFrame()
for hora in range(24):
    top5 = df17_reta_tomb[df17_reta_tomb.Hora == hora]['causa_acidente'].value_counts(normalize=True).head(5)
    df17_reta_tomb_acids[f'causa_reta_tomb_{hora}h'] = top5.index
    df17_reta_tomb_acids[f'%_reta_tomb_{hora}h'] = top5.values

df17_reta_tomb_acids

,causa_reta_tomb_0h,%_reta_tomb_0h,causa_reta_tomb_1h,%_reta_tomb_1h,causa_reta_tomb_2h,%_reta_tomb_2h,causa_reta_tomb_3h,%_reta_tomb_3h,causa_reta_tomb_4h,%_reta_tomb_4h,...,causa_reta_tomb_19h,%_reta_tomb_19h,causa_reta_tomb_20h,%_reta_tomb_20h,causa_reta_tomb_21h,%_reta_tomb_21h,causa_reta_tomb_22h,%_reta_tomb_22h,causa_reta_tomb_23h,%_reta_tomb_23h
0,Falta de Atenção à Condução,0.324675,Condutor Dormindo,0.245614,Falta de Atenção à Condução,0.351852,Falta de Atenção à Condução,0.318841,Falta de Atenção à Condução,0.265823,...,Falta de Atenção à Condução,0.381679,Falta de Atenção à Condução,0.42,Falta de Atenção à Condução,0.269663,Falta de Atenção à Condução,0.333333,Falta de Atenção à Condução,0.3625
1,Condutor Dormindo,0.181818,Falta de Atenção à Condução,0.228070,Condutor Dormindo,0.277778,Condutor Dormindo,0.289855,Condutor Dormindo,0.253165,...,Avarias e/ou desgaste excessivo no pneu,0.122137,Defeito Mecânico no Veículo,0.12,Defeito Mecânico no Veículo,0.179775,Defeito Mecânico no Veículo,0.161290,Condutor Dormindo,0.1250
2,Velocidade Incompatível,0.116883,Ingestão de Álcool,0.122807,Ingestão de Álcool,0.148148,Ingestão de Álcool,0.072464,Ingestão de Álcool,0.151899,...,Defeito na Via,0.099237,Ingestão de Álcool,0.10,Mal Súbito,0.078652,Condutor Dormindo,0.086022,Defeito Mecânico no Veículo,0.1125
3,Defeito Mecânico no Veículo,0.090909,Animais na Pista,0.087719,Defeito Mecânico no Veículo,0.092593,Ultrapassagem Indevida,0.043478,Defeito Mecânico no Veículo,0.075949,...,Pista Escorregadia,0.076336,Pista Escorregadia,0.05,Pista Escorregadia,0.078652,Ingestão de Álcool,0.075269,Ingestão de Álcool,0.1000
4,Ingestão de Álcool,0.090909,Velocidade Incompatível,0.070175,Defeito na Via,0.037037,Defeito Mecânico no Veículo,0.043478,Animais na Pista,0.050633,...,Defeito Mecânico no Veículo,0.068702,Defeito na Via,0.05,Condutor Dormindo,0.067416,Pista Escorregadia,0.075269,Avarias e/ou desgaste excessivo no pneu,0.0875


In [ ]:
# df17_reta_tomb_acids.to_csv('df17_reta_tomb_acids.csv', index=False,
#                             encoding='ISO-8859-1', sep=';')

## **2. Todos os veículos, em curva, um único veículo, tombamento**

**2013**

In [ ]:
# Filtragem 'Curva' + 'Tombamento'
df13_curva_tomb = filtro_tracado_acidente(df_prf13, 'Curva', 'Tombamento')

In [ ]:
df13_curva_tomb.tracado_via.unique()

array(['Curva'], dtype=object)

In [ ]:
df13_curva_tomb.tipo_acidente.unique()

array(['Tombamento'], dtype=object)

In [ ]:
counts, bins = np.histogram(df13_curva_tomb['Hora'], bins=range(0, 25, 1))
acidentes13 = np.array(counts)/sum(np.array(counts))
fluxo13 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa13 = (acidentes13/fluxo13)/sum(acidentes13/fluxo13)

In [ ]:
graphics_plot_prf(y=[taxa13, fluxo13, acidentes13],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Overturning of one vehicle on a curve section of the road (2013) - Accidents, traffic and accidents / traffic',
                  x_tick=4, save=True, save_name='curva_tomb_2013')

**2014**

In [ ]:
# Filtragem 'Curva' + 'Tombamento'
df14_curva_tomb = filtro_tracado_acidente(df_prf14, 'Curva', 'Tombamento')

In [ ]:
df14_curva_tomb.tracado_via.unique()

array(['Curva'], dtype=object)

In [ ]:
df14_curva_tomb.tipo_acidente.unique()

array(['Tombamento'], dtype=object)

In [ ]:
counts, bins = np.histogram(df14_curva_tomb['Hora'], bins=range(0, 25, 1))
acidentes14 = np.array(counts)/sum(np.array(counts))
fluxo14 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa14 = (acidentes14/fluxo14)/sum(acidentes14/fluxo14)

In [ ]:
graphics_plot_prf(y=[taxa14, fluxo14, acidentes14],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Overturning of one vehicle on a curve section of the road (2014) - Accidents, traffic and accidents / traffic',
                  x_tick=4, save=True, save_name='curva_tomb_2014')

**2015**

In [ ]:
# Filtragem 'Curva' + 'Tombamento'
df15_curva_tomb = filtro_tracado_acidente(df_prf15, 'Curva', 'Tombamento')

In [ ]:
df15_curva_tomb.tracado_via.unique()

array(['Curva'], dtype=object)

In [ ]:
df15_curva_tomb.tipo_acidente.unique()

array(['Tombamento'], dtype=object)

In [ ]:
counts, bins = np.histogram(df15_curva_tomb['Hora'], bins=range(0, 25, 1))
acidentes15 = np.array(counts)/sum(np.array(counts))
fluxo15 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa15 = (acidentes15/fluxo15)/sum(acidentes15/fluxo15)

In [ ]:
graphics_plot_prf(y=[taxa15, fluxo15, acidentes15],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Overturning of one vehicle on a curve section of the road (2015) - Accidents, traffic and accidents / traffic',
                  x_tick=4, save=True, save_name='curva_tomb_2015')

**2016**

In [ ]:
# Filtragem 'Curva' + 'Tombamento'
df16_curva_tomb = filtro_tracado_acidente(df_prf16, 'Curva', 'Tombamento')

In [ ]:
df16_curva_tomb.tracado_via.unique()

array(['Curva'], dtype=object)

In [ ]:
df16_curva_tomb.tipo_acidente.unique()

array(['Tombamento'], dtype=object)

In [ ]:
counts, bins = np.histogram(df16_curva_tomb['Hora'], bins=range(0, 25, 1))
acidentes16 = np.array(counts)/sum(np.array(counts))
fluxo16 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa16 = (acidentes16/fluxo16)/sum(acidentes16/fluxo16)

In [ ]:
graphics_plot_prf(y=[taxa16, fluxo16, acidentes16],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Overturning of one vehicle on a curve section of the road (2016) - Accidents, traffic and accidents / traffic',
                  x_tick=4, save=True, save_name='curva_tomb_2016')

**2017**

In [ ]:
# Filtragem 'Curva' + 'Tombamento'
df17_curva_tomb = filtro_tracado_acidente(df_prf17, 'Curva', 'Tombamento')

In [ ]:
df17_curva_tomb.tracado_via.unique()

array(['Curva'], dtype=object)

In [ ]:
df17_curva_tomb.tipo_acidente.unique()

array(['Tombamento'], dtype=object)

In [ ]:
counts, bins = np.histogram(df17_curva_tomb['Hora'], bins=range(0, 25, 1))
acidentes17 = np.array(counts)/sum(np.array(counts))
fluxo17= np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa17 = (acidentes17/fluxo17)/sum(acidentes17/fluxo17)

In [ ]:
graphics_plot_prf(y=[taxa17, fluxo17, acidentes17],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Overturning of one vehicle on a curve section of the road (2017) - Accidents, traffic and accidents / traffic',
                  x_tick=4, save=True, save_name='curva_tomb_2017')

In [ ]:
# df_novas_taxas['curva_tomb_13'] = taxa13
# df_novas_taxas['curva_tomb_14'] = taxa14
df_novas_taxas['curva_tomb_15'] = taxa15
df_novas_taxas['curva_tomb_16'] = taxa16
df_novas_taxas['curva_tomb_17'] = taxa17

df_novas_taxas.head(3)

,hora,reta_tomb_15,reta_tomb_16,reta_tomb_17,curva_tomb_15,curva_tomb_16,curva_tomb_17
0,1,0.061583,0.066493,0.071589,0.050382,0.056333,0.050344
1,2,0.079076,0.090659,0.071237,0.081005,0.075725,0.067674
2,3,0.086799,0.114448,0.081742,0.070771,0.077045,0.088994


### **Prinicipais razões de acidentes por hora**

**2013**

In [ ]:
df13_curva_tomb.causa_acidente.unique()

array(['Outras', 'Velocidade incompatível', 'Falta de atenção',
       'Dormindo', 'Defeito mecânico em veículo', 'Defeito na via',
       'Ingestão de álcool', 'Ultrapassagem indevida',
       'Desobediência à sinalização', 'Animais na Pista',
       'Não guardar distância de segurança'], dtype=object)

In [ ]:
df13_curva_tomb_acids = pd.DataFrame()
for hora in range(24):
    top5 = df13_curva_tomb[df13_curva_tomb.Hora == hora]['causa_acidente'].value_counts(normalize=True).head(5)
    df13_curva_tomb_acids[f'causa_curva_tomb_{hora}h'] = top5.index
    df13_curva_tomb_acids[f'%_curva_tomb_{hora}h'] = top5.values

df13_curva_tomb_acids

,causa_curva_tomb_0h,%_curva_tomb_0h,causa_curva_tomb_1h,%_curva_tomb_1h,causa_curva_tomb_2h,%_curva_tomb_2h,causa_curva_tomb_3h,%_curva_tomb_3h,causa_curva_tomb_4h,%_curva_tomb_4h,...,causa_curva_tomb_19h,%_curva_tomb_19h,causa_curva_tomb_20h,%_curva_tomb_20h,causa_curva_tomb_21h,%_curva_tomb_21h,causa_curva_tomb_22h,%_curva_tomb_22h,causa_curva_tomb_23h,%_curva_tomb_23h
0,Velocidade incompatível,0.382353,Velocidade incompatível,0.338235,Velocidade incompatível,0.345679,Velocidade incompatível,0.433962,Velocidade incompatível,0.337209,...,Outras,0.348315,Velocidade incompatível,0.372340,Velocidade incompatível,0.391753,Velocidade incompatível,0.433333,Velocidade incompatível,0.340426
1,Outras,0.250000,Outras,0.338235,Outras,0.296296,Outras,0.339623,Outras,0.325581,...,Velocidade incompatível,0.337079,Outras,0.276596,Outras,0.226804,Outras,0.266667,Outras,0.329787
2,Dormindo,0.117647,Falta de atenção,0.132353,Falta de atenção,0.148148,Falta de atenção,0.075472,Dormindo,0.139535,...,Defeito mecânico em veículo,0.112360,Falta de atenção,0.127660,Falta de atenção,0.185567,Falta de atenção,0.100000,Falta de atenção,0.106383
3,Falta de atenção,0.117647,Dormindo,0.088235,Dormindo,0.123457,Defeito mecânico em veículo,0.075472,Defeito mecânico em veículo,0.081395,...,Falta de atenção,0.101124,Defeito mecânico em veículo,0.063830,Defeito mecânico em veículo,0.092784,Dormindo,0.077778,Defeito mecânico em veículo,0.085106
4,Ingestão de álcool,0.044118,Defeito mecânico em veículo,0.058824,Ingestão de álcool,0.037037,Dormindo,0.056604,Falta de atenção,0.046512,...,Defeito na via,0.067416,Ingestão de álcool,0.053191,Ingestão de álcool,0.041237,Defeito mecânico em veículo,0.066667,Dormindo,0.074468


In [ ]:
# df13_curva_tomb_acids.to_csv('df13_curva_tomb_acids.csv', index=False,
#                              encoding='ISO-8859-1', sep=';')

**2014**

In [ ]:
df14_curva_tomb.causa_acidente.unique()

array(['Velocidade incompatível', 'Falta de atenção', 'Outras',
       'Dormindo', 'Defeito mecânico em veículo',
       'Não guardar distância de segurança', 'Defeito na via',
       'Ingestão de álcool', 'Ultrapassagem indevida', 'Animais na Pista',
       'Desobediência à sinalização'], dtype=object)

In [ ]:
df14_curva_tomb_acids = pd.DataFrame()
for hora in range(24):
    top5 = df14_curva_tomb[df14_curva_tomb.Hora == hora]['causa_acidente'].value_counts(normalize=True).head(5)
    df14_curva_tomb_acids[f'causa_curva_tomb_{hora}h'] = top5.index
    df14_curva_tomb_acids[f'%_curva_tomb_{hora}h'] = top5.values

df14_curva_tomb_acids

,causa_curva_tomb_0h,%_curva_tomb_0h,causa_curva_tomb_1h,%_curva_tomb_1h,causa_curva_tomb_2h,%_curva_tomb_2h,causa_curva_tomb_3h,%_curva_tomb_3h,causa_curva_tomb_4h,%_curva_tomb_4h,...,causa_curva_tomb_19h,%_curva_tomb_19h,causa_curva_tomb_20h,%_curva_tomb_20h,causa_curva_tomb_21h,%_curva_tomb_21h,causa_curva_tomb_22h,%_curva_tomb_22h,causa_curva_tomb_23h,%_curva_tomb_23h
0,Velocidade incompatível,0.500000,Velocidade incompatível,0.383562,Velocidade incompatível,0.380282,Velocidade incompatível,0.353846,Velocidade incompatível,0.376471,...,Velocidade incompatível,0.383178,Velocidade incompatível,0.390244,Velocidade incompatível,0.393258,Velocidade incompatível,0.473684,Velocidade incompatível,0.424528
1,Outras,0.305556,Dormindo,0.219178,Outras,0.197183,Outras,0.215385,Outras,0.223529,...,Outras,0.364486,Outras,0.341463,Outras,0.292135,Outras,0.228070,Outras,0.273585
2,Falta de atenção,0.097222,Outras,0.178082,Falta de atenção,0.169014,Falta de atenção,0.169231,Dormindo,0.152941,...,Defeito mecânico em veículo,0.084112,Falta de atenção,0.097561,Falta de atenção,0.146067,Falta de atenção,0.096491,Falta de atenção,0.122642
3,Dormindo,0.069444,Falta de atenção,0.109589,Dormindo,0.154930,Dormindo,0.107692,Falta de atenção,0.105882,...,Falta de atenção,0.065421,Defeito mecânico em veículo,0.085366,Ingestão de álcool,0.067416,Defeito mecânico em veículo,0.070175,Defeito mecânico em veículo,0.066038
4,Defeito mecânico em veículo,0.027778,Defeito mecânico em veículo,0.068493,Ingestão de álcool,0.042254,Defeito mecânico em veículo,0.092308,Defeito mecânico em veículo,0.047059,...,Ingestão de álcool,0.037383,Ingestão de álcool,0.048780,Defeito mecânico em veículo,0.056180,Dormindo,0.052632,Ingestão de álcool,0.037736


In [ ]:
# df14_curva_tomb_acids.to_csv('df14_curva_tomb_acids.csv', index=False,
#                              encoding='ISO-8859-1', sep=';')

**2015**

In [ ]:
df15_curva_tomb.causa_acidente.unique()

array(['Defeito mecânico em veículo', 'Outras', 'Falta de atenção',
       'Velocidade incompatível', 'Defeito na via', 'Dormindo',
       'Ingestão de álcool', 'Ultrapassagem indevida',
       'Não guardar distância de segurança',
       'Desobediência à sinalização', 'Animais na Pista'], dtype=object)

In [ ]:
df15_curva_tomb_acids = pd.DataFrame()
for hora in range(24):
    top5 = df15_curva_tomb[df15_curva_tomb.Hora == hora]['causa_acidente'].value_counts(normalize=True).head(5)
    df15_curva_tomb_acids[f'causa_curva_tomb_{hora}h'] = top5.index
    df15_curva_tomb_acids[f'%_curva_tomb_{hora}h'] = top5.values

df15_curva_tomb_acids

,causa_curva_tomb_0h,%_curva_tomb_0h,causa_curva_tomb_1h,%_curva_tomb_1h,causa_curva_tomb_2h,%_curva_tomb_2h,causa_curva_tomb_3h,%_curva_tomb_3h,causa_curva_tomb_4h,%_curva_tomb_4h,...,causa_curva_tomb_19h,%_curva_tomb_19h,causa_curva_tomb_20h,%_curva_tomb_20h,causa_curva_tomb_21h,%_curva_tomb_21h,causa_curva_tomb_22h,%_curva_tomb_22h,causa_curva_tomb_23h,%_curva_tomb_23h
0,Velocidade incompatível,0.333333,Velocidade incompatível,0.393443,Velocidade incompatível,0.522727,Velocidade incompatível,0.333333,Outras,0.294118,...,Velocidade incompatível,0.483146,Velocidade incompatível,0.464789,Velocidade incompatível,0.388235,Velocidade incompatível,0.418919,Velocidade incompatível,0.422222
1,Outras,0.294118,Outras,0.213115,Outras,0.204545,Outras,0.235294,Velocidade incompatível,0.282353,...,Outras,0.247191,Outras,0.239437,Outras,0.305882,Outras,0.189189,Outras,0.177778
2,Dormindo,0.176471,Dormindo,0.163934,Dormindo,0.159091,Dormindo,0.137255,Dormindo,0.200000,...,Defeito mecânico em veículo,0.101124,Falta de atenção,0.112676,Defeito mecânico em veículo,0.105882,Falta de atenção,0.148649,Falta de atenção,0.133333
3,Falta de atenção,0.098039,Falta de atenção,0.098361,Falta de atenção,0.068182,Defeito mecânico em veículo,0.137255,Falta de atenção,0.094118,...,Falta de atenção,0.089888,Defeito mecânico em veículo,0.098592,Falta de atenção,0.082353,Defeito na via,0.081081,Dormindo,0.100000
4,Ingestão de álcool,0.039216,Defeito na via,0.049180,Ingestão de álcool,0.022727,Falta de atenção,0.098039,Defeito mecânico em veículo,0.070588,...,Ingestão de álcool,0.033708,Ingestão de álcool,0.028169,Defeito na via,0.047059,Defeito mecânico em veículo,0.067568,Defeito mecânico em veículo,0.055556


In [ ]:
# df15_curva_tomb_acids.to_csv('df15_curva_tomb_acids.csv', index=False,
#                              encoding='ISO-8859-1', sep=';')

**2016**

In [ ]:
df16_curva_tomb.causa_acidente.unique()

array(['Dormindo', 'Outras', 'Velocidade incompatível',
       'Defeito mecânico em veículo', 'Falta de atenção',
       'Defeito na via', 'Ingestão de álcool',
       'Não guardar distância de segurança', 'Ultrapassagem indevida',
       'Desobediência à sinalização', 'Animais na Pista'], dtype=object)

In [ ]:
df16_curva_tomb_acids = pd.DataFrame()
for hora in range(24):
    top5 = df16_curva_tomb[df16_curva_tomb.Hora == hora]['causa_acidente'].value_counts(normalize=True).head(5)
    df16_curva_tomb_acids[f'causa_curva_tomb_{hora}h'] = top5.index
    df16_curva_tomb_acids[f'%_curva_tomb_{hora}h'] = top5.values

df16_curva_tomb_acids

,causa_curva_tomb_0h,%_curva_tomb_0h,causa_curva_tomb_1h,%_curva_tomb_1h,causa_curva_tomb_2h,%_curva_tomb_2h,causa_curva_tomb_3h,%_curva_tomb_3h,causa_curva_tomb_4h,%_curva_tomb_4h,...,causa_curva_tomb_19h,%_curva_tomb_19h,causa_curva_tomb_20h,%_curva_tomb_20h,causa_curva_tomb_21h,%_curva_tomb_21h,causa_curva_tomb_22h,%_curva_tomb_22h,causa_curva_tomb_23h,%_curva_tomb_23h
0,Velocidade incompatível,0.42,Velocidade incompatível,0.44,Velocidade incompatível,0.452381,Velocidade incompatível,0.458333,Outras,0.303571,...,Velocidade incompatível,0.441860,Velocidade incompatível,0.472973,Velocidade incompatível,0.477612,Velocidade incompatível,0.454545,Velocidade incompatível,0.380282
1,Outras,0.24,Outras,0.22,Outras,0.190476,Outras,0.229167,Velocidade incompatível,0.303571,...,Outras,0.255814,Outras,0.297297,Outras,0.298507,Outras,0.194805,Outras,0.197183
2,Dormindo,0.10,Falta de atenção,0.12,Falta de atenção,0.095238,Falta de atenção,0.187500,Falta de atenção,0.178571,...,Falta de atenção,0.116279,Falta de atenção,0.094595,Falta de atenção,0.104478,Falta de atenção,0.129870,Defeito mecânico em veículo,0.112676
3,Falta de atenção,0.10,Dormindo,0.12,Dormindo,0.095238,Dormindo,0.041667,Dormindo,0.125000,...,Defeito mecânico em veículo,0.104651,Defeito mecânico em veículo,0.040541,Defeito na via,0.044776,Defeito na via,0.064935,Falta de atenção,0.098592
4,Defeito mecânico em veículo,0.08,Ingestão de álcool,0.04,Defeito mecânico em veículo,0.071429,Defeito mecânico em veículo,0.020833,Ingestão de álcool,0.053571,...,Ingestão de álcool,0.046512,Dormindo,0.027027,Ingestão de álcool,0.044776,Defeito mecânico em veículo,0.051948,Dormindo,0.084507


In [ ]:
# df16_curva_tomb_acids.to_csv('df16_curva_tomb_acids.csv', index=False,
#                              encoding='ISO-8859-1', sep=';')

**2017**

In [ ]:
df17_curva_tomb.causa_acidente.unique()

array(['Velocidade Incompatível', 'Mal Súbito', 'Pista Escorregadia',
       'Falta de Atenção à Condução', 'Defeito Mecânico no Veículo',
       'Ultrapassagem Indevida', 'Condutor Dormindo',
       'Carga excessiva e/ou mal acondicionada', 'Animais na Pista',
       'Defeito na Via', 'Sinalização da via insuficiente ou inadequada',
       'Avarias e/ou desgaste excessivo no pneu',
       'Restrição de Visibilidade', 'Não guardar distância de segurança',
       'Objeto estático sobre o leito carroçável',
       'Desobediência às normas de trânsito pelo condutor',
       'Ingestão de Álcool', 'Falta de Atenção do Pedestre',
       'Deficiência ou não Acionamento do Sistema de Iluminação/Sinalização do Veículo',
       'Fenômenos da Natureza'], dtype=object)

In [ ]:
df17_curva_tomb_acids = pd.DataFrame()
for hora in range(24):
    top5 = df17_curva_tomb[df17_curva_tomb.Hora == hora]['causa_acidente'].value_counts(normalize=True).head(5)
    df17_curva_tomb_acids[f'causa_curva_tomb_{hora}h'] = top5.index
    df17_curva_tomb_acids[f'%_curva_tomb_{hora}h'] = top5.values

df17_curva_tomb_acids

,causa_curva_tomb_0h,%_curva_tomb_0h,causa_curva_tomb_1h,%_curva_tomb_1h,causa_curva_tomb_2h,%_curva_tomb_2h,causa_curva_tomb_3h,%_curva_tomb_3h,causa_curva_tomb_4h,%_curva_tomb_4h,...,causa_curva_tomb_19h,%_curva_tomb_19h,causa_curva_tomb_20h,%_curva_tomb_20h,causa_curva_tomb_21h,%_curva_tomb_21h,causa_curva_tomb_22h,%_curva_tomb_22h,causa_curva_tomb_23h,%_curva_tomb_23h
0,Velocidade Incompatível,0.371429,Velocidade Incompatível,0.342857,Velocidade Incompatível,0.315789,Velocidade Incompatível,0.333333,Velocidade Incompatível,0.315789,...,Falta de Atenção à Condução,0.310345,Velocidade Incompatível,0.270833,Velocidade Incompatível,0.333333,Falta de Atenção à Condução,0.365385,Velocidade Incompatível,0.333333
1,Falta de Atenção à Condução,0.314286,Falta de Atenção à Condução,0.200000,Falta de Atenção à Condução,0.263158,Falta de Atenção à Condução,0.250000,Falta de Atenção à Condução,0.289474,...,Velocidade Incompatível,0.258621,Falta de Atenção à Condução,0.250000,Falta de Atenção à Condução,0.263158,Velocidade Incompatível,0.250000,Falta de Atenção à Condução,0.270833
2,Defeito Mecânico no Veículo,0.085714,Condutor Dormindo,0.142857,Condutor Dormindo,0.157895,Condutor Dormindo,0.062500,Defeito Mecânico no Veículo,0.105263,...,Defeito Mecânico no Veículo,0.189655,Defeito Mecânico no Veículo,0.083333,Defeito Mecânico no Veículo,0.122807,Defeito Mecânico no Veículo,0.115385,Defeito Mecânico no Veículo,0.104167
3,Ingestão de Álcool,0.085714,Carga excessiva e/ou mal acondicionada,0.057143,Pista Escorregadia,0.131579,Carga excessiva e/ou mal acondicionada,0.062500,Carga excessiva e/ou mal acondicionada,0.052632,...,Pista Escorregadia,0.051724,Carga excessiva e/ou mal acondicionada,0.083333,Defeito na Via,0.052632,Ingestão de Álcool,0.076923,Condutor Dormindo,0.083333
4,Defeito na Via,0.057143,Defeito Mecânico no Veículo,0.057143,Avarias e/ou desgaste excessivo no pneu,0.026316,Animais na Pista,0.062500,Pista Escorregadia,0.052632,...,Desobediência às normas de trânsito pelo condutor,0.051724,Pista Escorregadia,0.083333,Avarias e/ou desgaste excessivo no pneu,0.052632,Restrição de Visibilidade,0.038462,Pista Escorregadia,0.062500


In [ ]:
# df17_curva_tomb_acids.to_csv('df17_curva_tomb_acids.csv', index=False,
#                              encoding='ISO-8859-1', sep=';')

## **3. Todos os veículos, em linha reta, um único veículo, capotamento**

**2013**

In [ ]:
# Filtragem 'Reta' + 'Capotamento'
df13_reta_capot = filtro_tracado_acidente(df_prf13, 'Reta', 'Capotamento')

In [ ]:
df13_reta_capot.tracado_via.unique()

array(['Reta'], dtype=object)

In [ ]:
df13_reta_capot.tipo_acidente.unique()

array(['Capotamento'], dtype=object)

In [ ]:
counts, bins = np.histogram(df13_reta_capot['Hora'], bins=range(0, 25, 1))
acidentes13 = np.array(counts)/sum(np.array(counts))
fluxo13 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa13 = (acidentes13/fluxo13)/sum(acidentes13/fluxo13)

In [ ]:
graphics_plot_prf(y=[taxa13, fluxo13, acidentes13],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Rollover of one vehicle on a straight section of the road (2013) - Accidents, traffic and accidents / traffic',
                  x_tick=4, save=True, save_name='reta_capot_2013')

**2014**

In [ ]:
# Filtragem 'Reta' + 'Capotamento'
df14_reta_capot = filtro_tracado_acidente(df_prf14, 'Reta', 'Capotamento')

In [ ]:
df14_reta_capot.tracado_via.unique()

array(['Reta'], dtype=object)

In [ ]:
df14_reta_capot.tipo_acidente.unique()

array(['Capotamento'], dtype=object)

In [ ]:
counts, bins = np.histogram(df14_reta_capot['Hora'], bins=range(0, 25, 1))
acidentes14 = np.array(counts)/sum(np.array(counts))
fluxo14 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa14 = (acidentes14/fluxo14)/sum(acidentes14/fluxo14)

In [ ]:
graphics_plot_prf(y=[taxa14, fluxo14, acidentes14],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Rollover of one vehicle on a straight section of the road (2014) - Accidents, traffic and accidents / traffic',
                  x_tick=4, save=True, save_name='reta_capot_2014')

**2015**

In [ ]:
# Filtragem 'Reta' + 'Capotamento'
df15_reta_capot = filtro_tracado_acidente(df_prf15, 'Reta', 'Capotamento')

In [ ]:
df15_reta_capot.tracado_via.unique()

array(['Reta'], dtype=object)

In [ ]:
df15_reta_capot.tipo_acidente.unique()

array(['Capotamento'], dtype=object)

In [ ]:
counts, bins = np.histogram(df15_reta_capot['Hora'], bins=range(0, 25, 1))
acidentes15 = np.array(counts)/sum(np.array(counts))
fluxo15 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa15 = (acidentes15/fluxo15)/sum(acidentes15/fluxo15)

In [ ]:
graphics_plot_prf(y=[taxa15, fluxo15, acidentes15],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Rollover of one vehicle on a straight section of the road (2015) - Accidents, traffic and accidents / traffic',
                  x_tick=4, save=True, save_name='reta_capot_2015')

**2016**

In [ ]:
# Filtragem 'Reta' + 'Capotamento'
df16_reta_capot = filtro_tracado_acidente(df_prf16, 'Reta', 'Capotamento')

In [ ]:
df16_reta_capot.tracado_via.unique()

array(['Reta'], dtype=object)

In [ ]:
df16_reta_capot.tipo_acidente.unique()

array(['Capotamento'], dtype=object)

In [ ]:
counts, bins = np.histogram(df16_reta_capot['Hora'], bins=range(0, 25, 1))
acidentes16 = np.array(counts)/sum(np.array(counts))
fluxo16 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa16 = (acidentes16/fluxo16)/sum(acidentes16/fluxo16)

In [ ]:
graphics_plot_prf(y=[taxa16, fluxo16, acidentes16],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Rollover of one vehicle on a straight section of the road (2016) - Accidents, traffic and accidents / traffic',
                  x_tick=4, save=True, save_name='reta_capot_2016')

**2017**

In [ ]:
# Filtragem 'Reta' + 'Capotamento'
df17_reta_capot = filtro_tracado_acidente(df_prf17, 'Reta', 'Capotamento')

In [ ]:
df17_reta_capot.tracado_via.unique()

array(['Reta'], dtype=object)

In [ ]:
df17_reta_capot.tipo_acidente.unique()

array(['Capotamento'], dtype=object)

In [ ]:
counts, bins = np.histogram(df17_reta_capot['Hora'], bins=range(0, 25, 1))
acidentes17 = np.array(counts)/sum(np.array(counts))
fluxo17= np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa17 = (acidentes17/fluxo17)/sum(acidentes17/fluxo17)

In [ ]:
graphics_plot_prf(y=[taxa17, fluxo17, acidentes17],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Rollover of one vehicle on a straight section of the road (2017) - Accidents, traffic and accidents / traffic',
                  x_tick=4, save=True, save_name='reta_capot_2017')

In [ ]:
# df_novas_taxas['reta_capot_13'] = taxa13
# df_novas_taxas['reta_capot_14'] = taxa14
df_novas_taxas['reta_capot_15'] = taxa15
df_novas_taxas['reta_capot_16'] = taxa16
df_novas_taxas['reta_capot_17'] = taxa17

df_novas_taxas.head(3)

,hora,reta_tomb_15,reta_tomb_16,reta_tomb_17,curva_tomb_15,curva_tomb_16,curva_tomb_17,reta_capot_15,reta_capot_16,reta_capot_17
0,1,0.061583,0.066493,0.071589,0.050382,0.056333,0.050344,0.058884,0.080476,0.072178
1,2,0.079076,0.090659,0.071237,0.081005,0.075725,0.067674,0.098942,0.090150,0.100490
2,3,0.086799,0.114448,0.081742,0.070771,0.077045,0.088994,0.087157,0.090473,0.110523


### **Prinicipais razões de acidentes por hora**

**2013**

In [ ]:
df13_reta_capot.causa_acidente.unique()

array(['Ingestão de álcool', 'Dormindo', 'Falta de atenção',
       'Velocidade incompatível', 'Defeito na via', 'Outras',
       'Desobediência à sinalização', 'Ultrapassagem indevida',
       'Defeito mecânico em veículo', 'Animais na Pista',
       'Não guardar distância de segurança'], dtype=object)

In [ ]:
df13_reta_capot_acids = pd.DataFrame()
for hora in range(24):
    top5 = df13_reta_capot[df13_reta_capot.Hora == hora]['causa_acidente'].value_counts(normalize=True).head(5)
    df13_reta_capot_acids[f'causa_reta_capot_{hora}h'] = top5.index
    df13_reta_capot_acids[f'%_reta_capot_{hora}h'] = top5.values

df13_reta_capot_acids

,causa_reta_capot_0h,%_reta_capot_0h,causa_reta_capot_1h,%_reta_capot_1h,causa_reta_capot_2h,%_reta_capot_2h,causa_reta_capot_3h,%_reta_capot_3h,causa_reta_capot_4h,%_reta_capot_4h,...,causa_reta_capot_19h,%_reta_capot_19h,causa_reta_capot_20h,%_reta_capot_20h,causa_reta_capot_21h,%_reta_capot_21h,causa_reta_capot_22h,%_reta_capot_22h,causa_reta_capot_23h,%_reta_capot_23h
0,Outras,0.392523,Outras,0.373832,Outras,0.264706,Outras,0.298851,Outras,0.366412,...,Outras,0.376543,Outras,0.518519,Outras,0.425532,Outras,0.401460,Outras,0.387324
1,Falta de atenção,0.205607,Velocidade incompatível,0.196262,Velocidade incompatível,0.196078,Ingestão de álcool,0.183908,Dormindo,0.145038,...,Velocidade incompatível,0.191358,Velocidade incompatível,0.160494,Velocidade incompatível,0.184397,Velocidade incompatível,0.167883,Velocidade incompatível,0.161972
2,Velocidade incompatível,0.130841,Ingestão de álcool,0.140187,Falta de atenção,0.176471,Dormindo,0.160920,Falta de atenção,0.137405,...,Falta de atenção,0.160494,Falta de atenção,0.104938,Falta de atenção,0.163121,Falta de atenção,0.102190,Falta de atenção,0.147887
3,Dormindo,0.074766,Dormindo,0.112150,Ingestão de álcool,0.166667,Velocidade incompatível,0.149425,Ingestão de álcool,0.137405,...,Ingestão de álcool,0.092593,Animais na Pista,0.055556,Ingestão de álcool,0.092199,Ingestão de álcool,0.087591,Ingestão de álcool,0.112676
4,Defeito mecânico em veículo,0.056075,Falta de atenção,0.112150,Dormindo,0.127451,Falta de atenção,0.126437,Velocidade incompatível,0.083969,...,Defeito mecânico em veículo,0.061728,Defeito na via,0.055556,Animais na Pista,0.042553,Defeito na via,0.072993,Dormindo,0.091549


In [ ]:
# df13_reta_capot_acids.to_csv('df13_reta_capot_acids.csv', index=False,
#                              encoding='ISO-8859-1', sep=';')

**2014**

In [ ]:
df14_reta_capot.causa_acidente.unique()

array(['Outras', 'Ingestão de álcool', 'Ultrapassagem indevida',
       'Dormindo', 'Defeito mecânico em veículo', 'Falta de atenção',
       'Animais na Pista', 'Velocidade incompatível', 'Defeito na via',
       'Não guardar distância de segurança',
       'Desobediência à sinalização'], dtype=object)

In [ ]:
df14_reta_capot_acids = pd.DataFrame()
for hora in range(24):
    top5 = df14_reta_capot[df14_reta_capot.Hora == hora]['causa_acidente'].value_counts(normalize=True).head(5)
    df14_reta_capot_acids[f'causa_reta_capot_{hora}h'] = top5.index
    df14_reta_capot_acids[f'%_reta_capot_{hora}h'] = top5.values

df14_reta_capot_acids

,causa_reta_capot_0h,%_reta_capot_0h,causa_reta_capot_1h,%_reta_capot_1h,causa_reta_capot_2h,%_reta_capot_2h,causa_reta_capot_3h,%_reta_capot_3h,causa_reta_capot_4h,%_reta_capot_4h,...,causa_reta_capot_19h,%_reta_capot_19h,causa_reta_capot_20h,%_reta_capot_20h,causa_reta_capot_21h,%_reta_capot_21h,causa_reta_capot_22h,%_reta_capot_22h,causa_reta_capot_23h,%_reta_capot_23h
0,Outras,0.360360,Outras,0.30,Outras,0.287037,Outras,0.363636,Outras,0.244275,...,Outras,0.380645,Outras,0.452229,Outras,0.400000,Outras,0.369863,Outras,0.349206
1,Falta de atenção,0.153153,Velocidade incompatível,0.17,Velocidade incompatível,0.203704,Dormindo,0.172727,Dormindo,0.206107,...,Velocidade incompatível,0.193548,Falta de atenção,0.165605,Velocidade incompatível,0.185185,Velocidade incompatível,0.219178,Velocidade incompatível,0.206349
2,Velocidade incompatível,0.153153,Falta de atenção,0.16,Dormindo,0.138889,Ingestão de álcool,0.163636,Velocidade incompatível,0.160305,...,Falta de atenção,0.180645,Velocidade incompatível,0.133758,Falta de atenção,0.177778,Falta de atenção,0.178082,Falta de atenção,0.134921
3,Dormindo,0.099099,Ingestão de álcool,0.13,Ingestão de álcool,0.129630,Velocidade incompatível,0.136364,Falta de atenção,0.122137,...,Ingestão de álcool,0.083871,Defeito mecânico em veículo,0.063694,Ingestão de álcool,0.125926,Ingestão de álcool,0.075342,Ingestão de álcool,0.095238
4,Ingestão de álcool,0.090090,Dormindo,0.12,Falta de atenção,0.111111,Falta de atenção,0.109091,Ingestão de álcool,0.106870,...,Animais na Pista,0.064516,Ingestão de álcool,0.057325,Defeito mecânico em veículo,0.044444,Animais na Pista,0.054795,Defeito na via,0.063492


In [ ]:
# df14_reta_capot_acids.to_csv('df14_reta_capot_acids.csv', index=False,
#                              encoding='ISO-8859-1', sep=';')

**2015**

In [ ]:
df15_reta_capot.causa_acidente.unique()

array(['Outras', 'Ingestão de álcool', 'Dormindo', 'Falta de atenção',
       'Defeito mecânico em veículo', 'Defeito na via',
       'Velocidade incompatível', 'Animais na Pista',
       'Ultrapassagem indevida', 'Não guardar distância de segurança',
       'Desobediência à sinalização'], dtype=object)

In [ ]:
df15_reta_capot_acids = pd.DataFrame()
for hora in range(24):
    top5 = df15_reta_capot[df15_reta_capot.Hora == hora]['causa_acidente'].value_counts(normalize=True).head(5)
    df15_reta_capot_acids[f'causa_reta_capot_{hora}h'] = top5.index
    df15_reta_capot_acids[f'%_reta_capot_{hora}h'] = top5.values

df15_reta_capot_acids

,causa_reta_capot_0h,%_reta_capot_0h,causa_reta_capot_1h,%_reta_capot_1h,causa_reta_capot_2h,%_reta_capot_2h,causa_reta_capot_3h,%_reta_capot_3h,causa_reta_capot_4h,%_reta_capot_4h,...,causa_reta_capot_19h,%_reta_capot_19h,causa_reta_capot_20h,%_reta_capot_20h,causa_reta_capot_21h,%_reta_capot_21h,causa_reta_capot_22h,%_reta_capot_22h,causa_reta_capot_23h,%_reta_capot_23h
0,Outras,0.386364,Outras,0.363636,Outras,0.3000,Outras,0.313253,Outras,0.382609,...,Outras,0.386740,Outras,0.312500,Outras,0.431655,Outras,0.450450,Outras,0.448819
1,Velocidade incompatível,0.181818,Velocidade incompatível,0.218182,Dormindo,0.2125,Dormindo,0.216867,Velocidade incompatível,0.165217,...,Velocidade incompatível,0.187845,Velocidade incompatível,0.205357,Velocidade incompatível,0.187050,Velocidade incompatível,0.171171,Velocidade incompatível,0.181102
2,Ingestão de álcool,0.159091,Ingestão de álcool,0.145455,Falta de atenção,0.1500,Ingestão de álcool,0.180723,Dormindo,0.147826,...,Falta de atenção,0.171271,Falta de atenção,0.142857,Falta de atenção,0.136691,Falta de atenção,0.117117,Ingestão de álcool,0.110236
3,Falta de atenção,0.125000,Dormindo,0.136364,Ingestão de álcool,0.1375,Falta de atenção,0.108434,Ingestão de álcool,0.113043,...,Ingestão de álcool,0.099448,Ingestão de álcool,0.107143,Ingestão de álcool,0.136691,Animais na Pista,0.054054,Falta de atenção,0.102362
4,Dormindo,0.068182,Falta de atenção,0.090909,Velocidade incompatível,0.1250,Velocidade incompatível,0.096386,Falta de atenção,0.078261,...,Defeito mecânico em veículo,0.038674,Defeito mecânico em veículo,0.053571,Dormindo,0.028777,Ingestão de álcool,0.054054,Dormindo,0.062992


In [ ]:
# df15_reta_capot_acids.to_csv('df15_reta_capot_acids.csv', index=False,
#                              encoding='ISO-8859-1', sep=';')

**2016**

In [ ]:
df16_reta_capot.causa_acidente.unique()

array(['Outras', 'Dormindo', 'Defeito mecânico em veículo',
       'Falta de atenção', 'Ultrapassagem indevida', 'Defeito na via',
       'Velocidade incompatível', 'Animais na Pista',
       'Ingestão de álcool', 'Não guardar distância de segurança',
       'Desobediência à sinalização'], dtype=object)

In [ ]:
df16_reta_capot_acids = pd.DataFrame()
for hora in range(24):
    top5 = df16_reta_capot[df16_reta_capot.Hora == hora]['causa_acidente'].value_counts(normalize=True).head(5)
    df16_reta_capot_acids[f'causa_reta_capot_{hora}h'] = top5.index
    df16_reta_capot_acids[f'%_reta_capot_{hora}h'] = top5.values

df16_reta_capot_acids

,causa_reta_capot_0h,%_reta_capot_0h,causa_reta_capot_1h,%_reta_capot_1h,causa_reta_capot_2h,%_reta_capot_2h,causa_reta_capot_3h,%_reta_capot_3h,causa_reta_capot_4h,%_reta_capot_4h,...,causa_reta_capot_19h,%_reta_capot_19h,causa_reta_capot_20h,%_reta_capot_20h,causa_reta_capot_21h,%_reta_capot_21h,causa_reta_capot_22h,%_reta_capot_22h,causa_reta_capot_23h,%_reta_capot_23h
0,Outras,0.333333,Outras,0.428571,Outras,0.413793,Outras,0.277228,Outras,0.300000,...,Outras,0.386667,Outras,0.452055,Outras,0.410853,Outras,0.344262,Outras,0.296610
1,Velocidade incompatível,0.190476,Velocidade incompatível,0.161905,Velocidade incompatível,0.218391,Dormindo,0.247525,Dormindo,0.183333,...,Velocidade incompatível,0.233333,Velocidade incompatível,0.178082,Velocidade incompatível,0.170543,Velocidade incompatível,0.188525,Velocidade incompatível,0.220339
2,Ingestão de álcool,0.150794,Dormindo,0.142857,Falta de atenção,0.103448,Ingestão de álcool,0.158416,Ingestão de álcool,0.175000,...,Falta de atenção,0.186667,Falta de atenção,0.130137,Falta de atenção,0.155039,Falta de atenção,0.122951,Falta de atenção,0.135593
3,Falta de atenção,0.134921,Falta de atenção,0.114286,Ingestão de álcool,0.103448,Falta de atenção,0.118812,Falta de atenção,0.150000,...,Ingestão de álcool,0.053333,Ingestão de álcool,0.068493,Ingestão de álcool,0.100775,Ingestão de álcool,0.122951,Dormindo,0.135593
4,Dormindo,0.095238,Ingestão de álcool,0.085714,Dormindo,0.080460,Velocidade incompatível,0.108911,Velocidade incompatível,0.125000,...,Defeito mecânico em veículo,0.046667,Defeito na via,0.054795,Animais na Pista,0.054264,Animais na Pista,0.073770,Ingestão de álcool,0.076271


In [ ]:
# df16_reta_capot_acids.to_csv('df16_reta_capot_acids.csv', index=False,
#                              encoding='ISO-8859-1', sep=';')

**2017**

In [ ]:
df17_reta_capot.causa_acidente.unique()

array(['Defeito Mecânico no Veículo', 'Falta de Atenção à Condução',
       'Animais na Pista', 'Condutor Dormindo', 'Pista Escorregadia',
       'Velocidade Incompatível', 'Ultrapassagem Indevida',
       'Defeito na Via', 'Avarias e/ou desgaste excessivo no pneu',
       'Carga excessiva e/ou mal acondicionada',
       'Não guardar distância de segurança',
       'Objeto estático sobre o leito carroçável', 'Mal Súbito',
       'Sinalização da via insuficiente ou inadequada',
       'Ingestão de Álcool',
       'Desobediência às normas de trânsito pelo condutor',
       'Falta de Atenção do Pedestre', 'Fenômenos da Natureza',
       'Restrição de Visibilidade',
       'Deficiência ou não Acionamento do Sistema de Iluminação/Sinalização do Veículo',
       'Ingestão de Substâncias Psicoativas'], dtype=object)

In [ ]:
df17_reta_capot_acids = pd.DataFrame()
for hora in range(24):
    top5 = df17_reta_capot[df17_reta_capot.Hora == hora]['causa_acidente'].value_counts(normalize=True).head(5)
    df17_reta_capot_acids[f'causa_reta_capot_{hora}h'] = top5.index
    df17_reta_capot_acids[f'%_reta_capot_{hora}h'] = top5.values

df17_reta_capot_acids

,causa_reta_capot_0h,%_reta_capot_0h,causa_reta_capot_1h,%_reta_capot_1h,causa_reta_capot_2h,%_reta_capot_2h,causa_reta_capot_3h,%_reta_capot_3h,causa_reta_capot_4h,%_reta_capot_4h,...,causa_reta_capot_19h,%_reta_capot_19h,causa_reta_capot_20h,%_reta_capot_20h,causa_reta_capot_21h,%_reta_capot_21h,causa_reta_capot_22h,%_reta_capot_22h,causa_reta_capot_23h,%_reta_capot_23h
0,Falta de Atenção à Condução,0.297619,Falta de Atenção à Condução,0.298851,Falta de Atenção à Condução,0.253165,Falta de Atenção à Condução,0.250000,Falta de Atenção à Condução,0.240506,...,Falta de Atenção à Condução,0.306931,Falta de Atenção à Condução,0.371134,Falta de Atenção à Condução,0.351064,Falta de Atenção à Condução,0.378378,Falta de Atenção à Condução,0.277778
1,Velocidade Incompatível,0.238095,Condutor Dormindo,0.206897,Ingestão de Álcool,0.215190,Ingestão de Álcool,0.228261,Condutor Dormindo,0.227848,...,Velocidade Incompatível,0.198020,Velocidade Incompatível,0.206186,Velocidade Incompatível,0.170213,Velocidade Incompatível,0.162162,Velocidade Incompatível,0.180556
2,Ingestão de Álcool,0.166667,Velocidade Incompatível,0.195402,Condutor Dormindo,0.202532,Condutor Dormindo,0.206522,Velocidade Incompatível,0.177215,...,Pista Escorregadia,0.099010,Ingestão de Álcool,0.082474,Ingestão de Álcool,0.127660,Ingestão de Álcool,0.162162,Condutor Dormindo,0.166667
3,Condutor Dormindo,0.130952,Ingestão de Álcool,0.091954,Velocidade Incompatível,0.151899,Velocidade Incompatível,0.108696,Ingestão de Álcool,0.126582,...,Ingestão de Álcool,0.089109,Objeto estático sobre o leito carroçável,0.051546,Animais na Pista,0.106383,Condutor Dormindo,0.081081,Ingestão de Álcool,0.125000
4,Animais na Pista,0.059524,Avarias e/ou desgaste excessivo no pneu,0.045977,Animais na Pista,0.037975,Animais na Pista,0.065217,Animais na Pista,0.063291,...,Defeito Mecânico no Veículo,0.059406,Pista Escorregadia,0.051546,Defeito Mecânico no Veículo,0.063830,Ultrapassagem Indevida,0.054054,Animais na Pista,0.111111


In [ ]:
# df17_reta_capot_acids.to_csv('df17_reta_capot_acids.csv', index=False,
#                             encoding='ISO-8859-1', sep=';')

## **4. Todos os veículos, em curva, um único veículo, capotamento**

**2013**

In [ ]:
# Filtragem 'Curva' + 'Capotamento'
df13_curva_capot = filtro_tracado_acidente(df_prf13, 'Curva', 'Capotamento')

In [ ]:
df13_curva_capot.tracado_via.unique()

array(['Curva'], dtype=object)

In [ ]:
df13_curva_capot.tipo_acidente.unique()

array(['Capotamento'], dtype=object)

In [ ]:
counts, bins = np.histogram(df13_curva_capot['Hora'], bins=range(0, 25, 1))
acidentes13 = np.array(counts)/sum(np.array(counts))
fluxo13 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa13 = (acidentes13/fluxo13)/sum(acidentes13/fluxo13)

In [ ]:
graphics_plot_prf(y=[taxa13, fluxo13, acidentes13],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Rollover of one vehicle on a curve section of the road (2013) - Accidents, traffic and accidents / traffic',
                  x_tick=4, save=True, save_name='curva_capot_2013')

**2014**

In [ ]:
# Filtragem 'Curva' + 'Capotamento'
df14_curva_capot = filtro_tracado_acidente(df_prf14, 'Curva', 'Capotamento')

In [ ]:
df14_curva_capot.tracado_via.unique()

array(['Curva'], dtype=object)

In [ ]:
df14_curva_capot.tipo_acidente.unique()

array(['Capotamento'], dtype=object)

In [ ]:
counts, bins = np.histogram(df14_curva_capot['Hora'], bins=range(0, 25, 1))
acidentes14 = np.array(counts)/sum(np.array(counts))
fluxo14 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa14 = (acidentes14/fluxo14)/sum(acidentes14/fluxo14)

In [ ]:
graphics_plot_prf(y=[taxa14, fluxo14, acidentes14],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Rollover of one vehicle on a curve section of the road (2014) - Accidents, traffic and accidents / traffic',
                  x_tick=4, save=True, save_name='curva_capot_2014')

**2015**

In [ ]:
# Filtragem 'Curva' + 'Capotamento'
df15_curva_capot = filtro_tracado_acidente(df_prf15, 'Curva', 'Capotamento')

In [ ]:
df15_curva_capot.tracado_via.unique()

array(['Curva'], dtype=object)

In [ ]:
df15_curva_capot.tipo_acidente.unique()

array(['Capotamento'], dtype=object)

In [ ]:
counts, bins = np.histogram(df15_curva_capot['Hora'], bins=range(0, 25, 1))
acidentes15 = np.array(counts)/sum(np.array(counts))
fluxo15 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa15 = (acidentes15/fluxo15)/sum(acidentes15/fluxo15)

In [ ]:
graphics_plot_prf(y=[taxa15, fluxo15, acidentes15],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Rollover of one vehicle on a curve section of the road (2015) - Accidents, traffic and accidents / traffic',
                  x_tick=4, save=True, save_name='curva_capot_2015')

**2016**

In [ ]:
# Filtragem 'Curva' + 'Capotamento'
df16_curva_capot = filtro_tracado_acidente(df_prf16, 'Curva', 'Capotamento')

In [ ]:
df16_curva_capot.tracado_via.unique()

array(['Curva'], dtype=object)

In [ ]:
df16_curva_capot.tipo_acidente.unique()

array(['Capotamento'], dtype=object)

In [ ]:
counts, bins = np.histogram(df16_curva_capot['Hora'], bins=range(0, 25, 1))
acidentes16 = np.array(counts)/sum(np.array(counts))
fluxo16 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa16 = (acidentes16/fluxo16)/sum(acidentes16/fluxo16)

In [ ]:
graphics_plot_prf(y=[taxa16, fluxo16, acidentes16],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Rollover of one vehicle on a curve section of the road (2016) - Accidents, traffic and accidents / traffic',
                  x_tick=4, save=True, save_name='curva_capot_2016')

**2017**

In [ ]:
# Filtragem 'Curva' + 'Capotamento'
df17_curva_capot = filtro_tracado_acidente(df_prf17, 'Curva', 'Capotamento')

In [ ]:
df17_curva_capot.tracado_via.unique()

array(['Curva'], dtype=object)

In [ ]:
df17_curva_capot.tipo_acidente.unique()

array(['Capotamento'], dtype=object)

In [ ]:
counts, bins = np.histogram(df17_curva_capot['Hora'], bins=range(0, 25, 1))
acidentes17 = np.array(counts)/sum(np.array(counts))
fluxo17= np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa17 = (acidentes17/fluxo17)/sum(acidentes17/fluxo17)

In [ ]:
graphics_plot_prf(y=[taxa17, fluxo17, acidentes17],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Rollover of one vehicle on a curve section of the road (2017) - Accidents, traffic and accidents / traffic',
                  x_tick=4, save=True, save_name='curva_capot_2017')

In [ ]:
# df_novas_taxas['curva_capot_13'] = taxa13
# df_novas_taxas['curva_capot_14'] = taxa14
df_novas_taxas['curva_capot_15'] = taxa15
df_novas_taxas['curva_capot_16'] = taxa16
df_novas_taxas['curva_capot_17'] = taxa17

df_novas_taxas.head(3)

,hora,reta_tomb_15,reta_tomb_16,reta_tomb_17,curva_tomb_15,curva_tomb_16,curva_tomb_17,reta_capot_15,reta_capot_16,reta_capot_17,curva_capot_15,curva_capot_16,curva_capot_17
0,1,0.061583,0.066493,0.071589,0.050382,0.056333,0.050344,0.058884,0.080476,0.072178,0.062854,0.058616,0.051969
1,2,0.079076,0.090659,0.071237,0.081005,0.075725,0.067674,0.098942,0.090150,0.100490,0.088331,0.080006,0.073185
2,3,0.086799,0.114448,0.081742,0.070771,0.077045,0.088994,0.087157,0.090473,0.110523,0.083730,0.074881,0.088643


### **Prinicipais razões de acidentes por hora**

**2013**

In [ ]:
df13_curva_capot.causa_acidente.unique()

array(['Dormindo', 'Falta de atenção', 'Velocidade incompatível',
       'Ingestão de álcool', 'Outras', 'Defeito mecânico em veículo',
       'Animais na Pista', 'Defeito na via', 'Ultrapassagem indevida',
       'Desobediência à sinalização',
       'Não guardar distância de segurança'], dtype=object)

In [ ]:
df13_curva_capot_acids = pd.DataFrame()
for hora in range(24):
    top5 = df13_curva_capot[df13_curva_capot.Hora == hora]['causa_acidente'].value_counts(normalize=True).head(5)
    df13_curva_capot_acids[f'causa_curva_capot_{hora}h'] = top5.index
    df13_curva_capot_acids[f'%_curva_capot_{hora}h'] = top5.values

df13_curva_capot_acids

,causa_curva_capot_0h,%_curva_capot_0h,causa_curva_capot_1h,%_curva_capot_1h,causa_curva_capot_2h,%_curva_capot_2h,causa_curva_capot_3h,%_curva_capot_3h,causa_curva_capot_4h,%_curva_capot_4h,...,causa_curva_capot_19h,%_curva_capot_19h,causa_curva_capot_20h,%_curva_capot_20h,causa_curva_capot_21h,%_curva_capot_21h,causa_curva_capot_22h,%_curva_capot_22h,causa_curva_capot_23h,%_curva_capot_23h
0,Velocidade incompatível,0.479167,Outras,0.313253,Outras,0.324324,Outras,0.412698,Velocidade incompatível,0.350877,...,Outras,0.410853,Outras,0.376068,Velocidade incompatível,0.419355,Velocidade incompatível,0.514563,Velocidade incompatível,0.466019
1,Outras,0.166667,Velocidade incompatível,0.253012,Velocidade incompatível,0.310811,Dormindo,0.206349,Outras,0.236842,...,Velocidade incompatível,0.395349,Velocidade incompatível,0.333333,Outras,0.408602,Outras,0.213592,Outras,0.271845
2,Falta de atenção,0.125000,Falta de atenção,0.156627,Ingestão de álcool,0.135135,Velocidade incompatível,0.174603,Dormindo,0.140351,...,Falta de atenção,0.100775,Falta de atenção,0.102564,Falta de atenção,0.086022,Falta de atenção,0.087379,Falta de atenção,0.116505
3,Ingestão de álcool,0.083333,Ingestão de álcool,0.120482,Dormindo,0.121622,Falta de atenção,0.095238,Ingestão de álcool,0.114035,...,Ingestão de álcool,0.038760,Ingestão de álcool,0.094017,Defeito mecânico em veículo,0.032258,Ingestão de álcool,0.048544,Ingestão de álcool,0.097087
4,Dormindo,0.062500,Dormindo,0.108434,Falta de atenção,0.094595,Ingestão de álcool,0.063492,Falta de atenção,0.087719,...,Defeito mecânico em veículo,0.031008,Dormindo,0.034188,Ingestão de álcool,0.021505,Dormindo,0.048544,Dormindo,0.019417


In [ ]:
# df13_curva_capot_acids.to_csv('df13_curva_capot_acids.csv', index=False,
#                               encoding='ISO-8859-1', sep=';')

**2014**

In [ ]:
df14_curva_capot.causa_acidente.unique()

array(['Outras', 'Velocidade incompatível', 'Falta de atenção',
       'Animais na Pista', 'Dormindo', 'Ingestão de álcool',
       'Defeito na via', 'Defeito mecânico em veículo',
       'Ultrapassagem indevida', 'Desobediência à sinalização',
       'Não guardar distância de segurança'], dtype=object)

In [ ]:
df14_curva_capot_acids = pd.DataFrame()
for hora in range(24):
    top5 = df14_curva_capot[df14_curva_capot.Hora == hora]['causa_acidente'].value_counts(normalize=True).head(5)
    df14_curva_capot_acids[f'causa_curva_capot_{hora}h'] = top5.index
    df14_curva_capot_acids[f'%_curva_capot_{hora}h'] = top5.values

df14_curva_capot_acids

,causa_curva_capot_0h,%_curva_capot_0h,causa_curva_capot_1h,%_curva_capot_1h,causa_curva_capot_2h,%_curva_capot_2h,causa_curva_capot_3h,%_curva_capot_3h,causa_curva_capot_4h,%_curva_capot_4h,...,causa_curva_capot_19h,%_curva_capot_19h,causa_curva_capot_20h,%_curva_capot_20h,causa_curva_capot_21h,%_curva_capot_21h,causa_curva_capot_22h,%_curva_capot_22h,causa_curva_capot_23h,%_curva_capot_23h
0,Velocidade incompatível,0.376623,Velocidade incompatível,0.500000,Velocidade incompatível,0.467742,Velocidade incompatível,0.358209,Velocidade incompatível,0.386139,...,Velocidade incompatível,0.513514,Velocidade incompatível,0.456140,Velocidade incompatível,0.423423,Velocidade incompatível,0.493506,Velocidade incompatível,0.547619
1,Outras,0.233766,Outras,0.272727,Outras,0.274194,Outras,0.238806,Outras,0.316832,...,Outras,0.261261,Outras,0.228070,Outras,0.306306,Outras,0.246753,Outras,0.178571
2,Ingestão de álcool,0.142857,Falta de atenção,0.079545,Dormindo,0.112903,Ingestão de álcool,0.134328,Ingestão de álcool,0.099010,...,Ingestão de álcool,0.072072,Ingestão de álcool,0.096491,Falta de atenção,0.072072,Falta de atenção,0.116883,Falta de atenção,0.130952
3,Falta de atenção,0.129870,Ingestão de álcool,0.056818,Ingestão de álcool,0.080645,Dormindo,0.104478,Falta de atenção,0.059406,...,Falta de atenção,0.063063,Falta de atenção,0.078947,Ingestão de álcool,0.054054,Ingestão de álcool,0.051948,Ingestão de álcool,0.083333
4,Defeito mecânico em veículo,0.038961,Dormindo,0.045455,Falta de atenção,0.048387,Falta de atenção,0.074627,Dormindo,0.049505,...,Defeito na via,0.045045,Defeito na via,0.052632,Defeito mecânico em veículo,0.054054,Dormindo,0.051948,Defeito na via,0.035714


In [ ]:
# df14_curva_capot_acids.to_csv('df14_curva_capot_acids.csv', index=False,
#                               encoding='ISO-8859-1', sep=';')

**2015**

In [ ]:
df15_curva_capot.causa_acidente.unique()

array(['Ingestão de álcool', 'Dormindo', 'Outras', 'Falta de atenção',
       'Velocidade incompatível', 'Defeito mecânico em veículo',
       'Animais na Pista', 'Defeito na via', 'Ultrapassagem indevida',
       'Desobediência à sinalização',
       'Não guardar distância de segurança'], dtype=object)

In [ ]:
df15_curva_capot_acids = pd.DataFrame()
for hora in range(24):
    top5 = df15_curva_capot[df15_curva_capot.Hora == hora]['causa_acidente'].value_counts(normalize=True).head(5)
    df15_curva_capot_acids[f'causa_curva_capot_{hora}h'] = top5.index
    df15_curva_capot_acids[f'%_curva_capot_{hora}h'] = top5.values

df15_curva_capot_acids

,causa_curva_capot_0h,%_curva_capot_0h,causa_curva_capot_1h,%_curva_capot_1h,causa_curva_capot_2h,%_curva_capot_2h,causa_curva_capot_3h,%_curva_capot_3h,causa_curva_capot_4h,%_curva_capot_4h,...,causa_curva_capot_19h,%_curva_capot_19h,causa_curva_capot_20h,%_curva_capot_20h,causa_curva_capot_21h,%_curva_capot_21h,causa_curva_capot_22h,%_curva_capot_22h,causa_curva_capot_23h,%_curva_capot_23h
0,Velocidade incompatível,0.409091,Velocidade incompatível,0.376812,Velocidade incompatível,0.407407,Velocidade incompatível,0.323077,Velocidade incompatível,0.304348,...,Velocidade incompatível,0.575472,Velocidade incompatível,0.407407,Velocidade incompatível,0.439560,Velocidade incompatível,0.455696,Velocidade incompatível,0.371795
1,Outras,0.257576,Outras,0.260870,Outras,0.314815,Outras,0.246154,Outras,0.202899,...,Outras,0.226415,Outras,0.345679,Outras,0.318681,Outras,0.367089,Outras,0.333333
2,Ingestão de álcool,0.136364,Falta de atenção,0.159420,Ingestão de álcool,0.166667,Ingestão de álcool,0.215385,Ingestão de álcool,0.188406,...,Falta de atenção,0.094340,Falta de atenção,0.123457,Ingestão de álcool,0.109890,Falta de atenção,0.101266,Ingestão de álcool,0.141026
3,Dormindo,0.075758,Dormindo,0.072464,Falta de atenção,0.055556,Dormindo,0.138462,Falta de atenção,0.159420,...,Ingestão de álcool,0.037736,Ingestão de álcool,0.037037,Falta de atenção,0.065934,Dormindo,0.025316,Falta de atenção,0.064103
4,Falta de atenção,0.060606,Ingestão de álcool,0.072464,Dormindo,0.055556,Falta de atenção,0.076923,Dormindo,0.101449,...,Dormindo,0.028302,Defeito mecânico em veículo,0.024691,Defeito mecânico em veículo,0.021978,Ingestão de álcool,0.025316,Dormindo,0.038462


In [ ]:
# df15_curva_capot_acids.to_csv('df15_curva_capot_acids.csv', index=False,
#                               encoding='ISO-8859-1', sep=';')

**2016**

In [ ]:
df16_curva_capot.causa_acidente.unique()

array(['Velocidade incompatível', 'Falta de atenção', 'Defeito na via',
       'Defeito mecânico em veículo', 'Ingestão de álcool', 'Outras',
       'Animais na Pista', 'Dormindo', 'Ultrapassagem indevida',
       'Desobediência à sinalização',
       'Não guardar distância de segurança'], dtype=object)

In [ ]:
df16_curva_capot_acids = pd.DataFrame()
for hora in range(24):
    top5 = df16_curva_capot[df16_curva_capot.Hora == hora]['causa_acidente'].value_counts(normalize=True).head(5)
    df16_curva_capot_acids[f'causa_curva_capot_{hora}h'] = top5.index
    df16_curva_capot_acids[f'%_curva_capot_{hora}h'] = top5.values

df16_curva_capot_acids

,causa_curva_capot_0h,%_curva_capot_0h,causa_curva_capot_1h,%_curva_capot_1h,causa_curva_capot_2h,%_curva_capot_2h,causa_curva_capot_3h,%_curva_capot_3h,causa_curva_capot_4h,%_curva_capot_4h,...,causa_curva_capot_19h,%_curva_capot_19h,causa_curva_capot_20h,%_curva_capot_20h,causa_curva_capot_21h,%_curva_capot_21h,causa_curva_capot_22h,%_curva_capot_22h,causa_curva_capot_23h,%_curva_capot_23h
0,Velocidade incompatível,0.461538,Velocidade incompatível,0.454545,Velocidade incompatível,0.411765,Velocidade incompatível,0.309859,Velocidade incompatível,0.297619,...,Velocidade incompatível,0.421875,Velocidade incompatível,0.451613,Velocidade incompatível,0.442105,Velocidade incompatível,0.447761,Velocidade incompatível,0.402597
1,Outras,0.246154,Falta de atenção,0.151515,Outras,0.333333,Outras,0.281690,Outras,0.238095,...,Outras,0.351562,Outras,0.311828,Outras,0.273684,Outras,0.238806,Outras,0.324675
2,Ingestão de álcool,0.107692,Outras,0.136364,Ingestão de álcool,0.137255,Ingestão de álcool,0.140845,Ingestão de álcool,0.154762,...,Falta de atenção,0.109375,Falta de atenção,0.107527,Falta de atenção,0.115789,Ingestão de álcool,0.149254,Falta de atenção,0.077922
3,Falta de atenção,0.107692,Ingestão de álcool,0.136364,Dormindo,0.098039,Dormindo,0.140845,Dormindo,0.130952,...,Ingestão de álcool,0.039062,Ingestão de álcool,0.043011,Ingestão de álcool,0.063158,Falta de atenção,0.089552,Dormindo,0.051948
4,Defeito mecânico em veículo,0.030769,Dormindo,0.090909,Falta de atenção,0.019608,Falta de atenção,0.084507,Falta de atenção,0.119048,...,Defeito na via,0.031250,Defeito na via,0.043011,Dormindo,0.031579,Defeito mecânico em veículo,0.044776,Ingestão de álcool,0.051948


In [ ]:
# df16_curva_capot_acids.to_csv('df16_curva_capot_acids.csv', index=False,
#                               encoding='ISO-8859-1', sep=';')

**2017**

In [ ]:
df17_curva_capot.causa_acidente.unique()

array(['Carga excessiva e/ou mal acondicionada',
       'Velocidade Incompatível', 'Condutor Dormindo',
       'Falta de Atenção à Condução', 'Defeito Mecânico no Veículo',
       'Ultrapassagem Indevida', 'Pista Escorregadia', 'Defeito na Via',
       'Mal Súbito', 'Avarias e/ou desgaste excessivo no pneu',
       'Objeto estático sobre o leito carroçável',
       'Restrição de Visibilidade',
       'Desobediência às normas de trânsito pelo condutor',
       'Não guardar distância de segurança', 'Ingestão de Álcool',
       'Fenômenos da Natureza', 'Animais na Pista',
       'Sinalização da via insuficiente ou inadequada'], dtype=object)

In [ ]:
df17_curva_capot_acids = pd.DataFrame()
for hora in range(24):
    top5 = df17_curva_capot[df17_curva_capot.Hora == hora]['causa_acidente'].value_counts(normalize=True).head(5)
    df17_curva_capot_acids[f'causa_curva_capot_{hora}h'] = top5.index
    df17_curva_capot_acids[f'%_curva_capot_{hora}h'] = top5.values

df17_curva_capot_acids

,causa_curva_capot_0h,%_curva_capot_0h,causa_curva_capot_1h,%_curva_capot_1h,causa_curva_capot_2h,%_curva_capot_2h,causa_curva_capot_3h,%_curva_capot_3h,causa_curva_capot_4h,%_curva_capot_4h,...,causa_curva_capot_19h,%_curva_capot_19h,causa_curva_capot_20h,%_curva_capot_20h,causa_curva_capot_21h,%_curva_capot_21h,causa_curva_capot_22h,%_curva_capot_22h,causa_curva_capot_23h,%_curva_capot_23h
0,Velocidade Incompatível,0.333333,Velocidade Incompatível,0.363636,Falta de Atenção à Condução,0.250000,Velocidade Incompatível,0.470588,Velocidade Incompatível,0.456140,...,Velocidade Incompatível,0.350649,Velocidade Incompatível,0.373134,Velocidade Incompatível,0.459459,Velocidade Incompatível,0.500000,Velocidade Incompatível,0.468085
1,Falta de Atenção à Condução,0.214286,Falta de Atenção à Condução,0.181818,Velocidade Incompatível,0.227273,Falta de Atenção à Condução,0.196078,Condutor Dormindo,0.157895,...,Falta de Atenção à Condução,0.233766,Falta de Atenção à Condução,0.313433,Falta de Atenção à Condução,0.216216,Falta de Atenção à Condução,0.159091,Falta de Atenção à Condução,0.191489
2,Pista Escorregadia,0.142857,Ingestão de Álcool,0.159091,Ingestão de Álcool,0.181818,Condutor Dormindo,0.117647,Falta de Atenção à Condução,0.157895,...,Pista Escorregadia,0.220779,Pista Escorregadia,0.164179,Pista Escorregadia,0.108108,Pista Escorregadia,0.113636,Pista Escorregadia,0.106383
3,Condutor Dormindo,0.119048,Condutor Dormindo,0.090909,Condutor Dormindo,0.136364,Ingestão de Álcool,0.098039,Ingestão de Álcool,0.070175,...,Ultrapassagem Indevida,0.051948,Ingestão de Álcool,0.029851,Condutor Dormindo,0.067568,Ingestão de Álcool,0.068182,Condutor Dormindo,0.085106
4,Ingestão de Álcool,0.095238,Defeito Mecânico no Veículo,0.045455,Pista Escorregadia,0.113636,Pista Escorregadia,0.039216,Defeito Mecânico no Veículo,0.052632,...,Defeito Mecânico no Veículo,0.038961,Defeito Mecânico no Veículo,0.014925,Ingestão de Álcool,0.054054,Desobediência às normas de trânsito pelo condutor,0.022727,Ingestão de Álcool,0.085106


In [ ]:
# df17_curva_capot_acids.to_csv('df17_curva_capot_acids.csv', index=False,
#                               encoding='ISO-8859-1', sep=';')

## **Análises das taxas**

In [ ]:
print(df_novas_taxas.shape)
df_novas_taxas.head(3)

(24, 13)


,hora,reta_tomb_15,reta_tomb_16,reta_tomb_17,curva_tomb_15,curva_tomb_16,curva_tomb_17,reta_capot_15,reta_capot_16,reta_capot_17,curva_capot_15,curva_capot_16,curva_capot_17
0,1,0.061583,0.066493,0.071589,0.050382,0.056333,0.050344,0.058884,0.080476,0.072178,0.062854,0.058616,0.051969
1,2,0.079076,0.090659,0.071237,0.081005,0.075725,0.067674,0.098942,0.090150,0.100490,0.088331,0.080006,0.073185
2,3,0.086799,0.114448,0.081742,0.070771,0.077045,0.088994,0.087157,0.090473,0.110523,0.083730,0.074881,0.088643


In [ ]:
df_max = df_novas_taxas.copy()
cols = df_max.iloc[:,1:].columns

for col in cols:
    df_max[col] = [1 if x == df_max[col].max() else 0 for x in df_max[col]]

df_max['max'] = df_max.iloc[:,1:].sum(axis=1)
df_max.head(3)

,hora,reta_tomb_15,reta_tomb_16,reta_tomb_17,curva_tomb_15,curva_tomb_16,curva_tomb_17,reta_capot_15,reta_capot_16,reta_capot_17,curva_capot_15,curva_capot_16,curva_capot_17,max
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,1,0,0,0,0,0,1
2,3,0,1,0,0,0,0,0,0,0,0,0,0,1


In [ ]:
df_max['max']

0     0
1     1
2     1
3     9
4     1
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
20    0
21    0
22    0
23    0
Name: max, dtype: int64

In [ ]:
import plotly.express as px

fig = px.histogram(df_max, x='hora', y='max', nbins=24,
                   title='Accident rate peak per hour')
fig.update_layout(xaxis=dict(tickvals=df_max.hora),
                  xaxis_title='Hour',
                  yaxis_title='Occurrences')
fig.show()

In [ ]:
df_max2 = df_novas_taxas.copy()
cols = df_max2.iloc[:,1:].columns

for col in cols:
    df_max2[col] = [1 if x == df_max2[col].nlargest(2).values[-1] else 0 for x in df_max2[col]]

df_max2['max2'] = df_max2.iloc[:,1:].sum(axis=1)
df_max2.head(3)

,hora,reta_tomb_15,reta_tomb_16,reta_tomb_17,curva_tomb_15,curva_tomb_16,curva_tomb_17,reta_capot_15,reta_capot_16,reta_capot_17,curva_capot_15,curva_capot_16,curva_capot_17,max2
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,1,0,0,0,0,0,1,0,0,2
2,3,1,0,1,0,1,1,0,1,1,0,0,1,7


In [ ]:
df_max2['max2']

0     0
1     2
2     7
3     2
4     1
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
20    0
21    0
22    0
23    0
Name: max2, dtype: int64

In [ ]:
import plotly.express as px

fig = px.histogram(df_max2, x='hora', y='max2', nbins=24,
                   title='Accident rate second peak per hour')
fig.update_layout(xaxis = dict(tickvals=df_max2.hora),
                  xaxis_title='Hour',
                  yaxis_title='Occurrences')
fig.show()

In [ ]:
df_max3 = df_novas_taxas.copy()
cols = df_max3.iloc[:,1:].columns

for col in cols:
    df_max3[col] = [1 if x == df_max3[col].nlargest(3).values[-1] else 0 for x in df_max3[col]]

df_max3['max3'] = df_max3.iloc[:,1:].sum(axis=1)
df_max3.head(3)

,hora,reta_tomb_15,reta_tomb_16,reta_tomb_17,curva_tomb_15,curva_tomb_16,curva_tomb_17,reta_capot_15,reta_capot_16,reta_capot_17,curva_capot_15,curva_capot_16,curva_capot_17,max3
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,0,1,0,0,1,1,0,1,1,0,1,0,6
2,3,0,0,0,0,0,0,1,0,0,1,0,0,2


In [ ]:
df_max3['max3']

0     0
1     6
2     2
3     1
4     3
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
20    0
21    0
22    0
23    0
Name: max3, dtype: int64

In [ ]:
import plotly.express as px

fig = px.histogram(df_max3, x='hora', y='max3', nbins=24,
                   title='Accident rate third peak per hour')
fig.update_layout(xaxis=dict(tickvals=df_max3.hora),
                  xaxis_title='Hour',
                  yaxis_title='Occurrences')
fig.show()

In [ ]:
df_transp = df_novas_taxas.set_index('hora').T
df_transp.head(2)

hora,1,2,3,4,5,6,7,8,9,10,...,15,16,17,18,19,20,21,22,23,24
reta_tomb_15,0.061583,0.079076,0.086799,0.100726,0.08667,0.055813,0.032218,0.028085,0.027458,0.023373,...,0.025724,0.02272,0.031251,0.026356,0.023241,0.028104,0.027841,0.039144,0.040957,0.057271
reta_tomb_16,0.066493,0.090659,0.114448,0.093535,0.08332,0.049135,0.032732,0.024627,0.027727,0.021162,...,0.025956,0.02773,0.021012,0.022320,0.026090,0.024713,0.026810,0.033243,0.038998,0.062511


In [ ]:
mean_7am_7pm = df_transp.iloc[:, 6:19].values.mean()
mean_7am_7pm

0.027426783158845433

In [ ]:
import plotly.graph_objects as go

mean_list = []

fig = go.Figure()
for col in range(1, 25):
    fig.add_trace(go.Box(y=df_transp[col]/mean_7am_7pm, name=col,
                         marker_color = 'black', showlegend=False))
    mean_list.append(df_transp[col].mean()/mean_7am_7pm)

fig.add_trace(go.Scatter(x=df_transp.columns.values, y=mean_list,
                         name='Accidents rate',
                         line = dict(color='black', dash='dot')))
fig.update_layout(showlegend=True,
                  xaxis_title='Hour',
                  yaxis_title='Rate/mean_7am_7pm',
                  title='Accidents rate, divided by rate mean between 7am and 7pm, per hour',
                  paper_bgcolor='rgba(0,0,0,0)',
                  plot_bgcolor='rgba(0,0,0,0)')
fig.update_layout(legend=dict(orientation='h', yanchor='bottom', y=1.01,
                              xanchor='right', x=0.95))

fig.show()

In [ ]:
# Download images folder
!zip -r /content/images.zip /content/images

from google.colab import files
files.download('/content/images.zip')

# **Outras análises - Ingestão de álcool**

1.   Todos os veículos, em linha reta, um único veículo, tombamento
2.   Todos os veículos, em curva, um único veículo, tombamento
3.   Todos os veículos, em linha reta, um único veículo, capotamento
4.   Todos os veículos, em curva, um único veículo, capotamento

In [ ]:
def filtro_tracado_acidente_causa(df, tipo_tracado, tipo_acidente, tipo_causa):

    via = df.tracado_via.isin([tipo_tracado])
    acid = df.tipo_acidente.isin([tipo_acidente])
    causa = df.causa_acidente.isin([tipo_causa])

    return df[(via) & (acid) & (causa)]

## **Dados**

In [ ]:
%%capture

# Fluxo
!gdown 1Kru8e3M6AjhEDuVXH7Tqx36wZjz4-Xc0

dfFluxo = pd.read_csv('DNIT.csv')
dfDNIT = pd.DataFrame(dfFluxo['avgtotal'].loc[2::3].values.reshape(4,24)).T

In [ ]:
%%capture
# !gdown 1rLrFckRbr_oScvziSCodjC2_UxoEOAMk # PRF 2013 - 1 veículo
# !gdown 1--JAT8gfd9CBtIlNBBj5yB_C5ipq9p0u # PRF 2014 - 1 veículo
!gdown 1-4vkJOgGW9W__P9zFhbVQkT3Uarwmbbv # PRF 2015 - 1 veículo
!gdown 1-6cj1H9JsUtdIZtO78x3HhdSFjpsqPj_ # PRF 2016 - 1 veículo
!gdown 1-6tc2J-omr-39k9n3FWApTqnJrZFIbnp # PRF 2017 - 1 veículo

# Datasets já filtrados para veículo único
# df_prf13 = pd.read_csv('prf_2013_1veiculo.csv')
# df_prf13['Hora'] = pd.to_datetime(df_prf13['horario'], format='%H:%M:%S').dt.hour
# df_prf14 = pd.read_csv('prf_2014_1veiculo.csv')
# df_prf14['Hora'] = pd.to_datetime(df_prf14['horario'], format='%H:%M:%S').dt.hour
df_prf15 = pd.read_csv('prf_2015_1veiculo.csv')
df_prf15['Hora'] = pd.to_datetime(df_prf15['horario'], format='%H:%M:%S').dt.hour
df_prf16 = pd.read_csv('prf_2016_1veiculo.csv')
df_prf16['Hora'] = pd.to_datetime(df_prf16['horario'], format='%H:%M:%S').dt.hour
df_prf17 = pd.read_csv('prf_2017_1veiculo.csv')
df_prf17['Hora'] = pd.to_datetime(df_prf17['horario'], format='%H:%M:%S').dt.hour

## **1. Todos os veículos, em linha reta, um único veículo, tombamento**

**2015**

In [ ]:
# Filtragem 'Reta' + 'Tombamento' + 'Ingestão de álcool'
df15_reta_tomb_alc = filtro_tracado_acidente_causa(df_prf15, 'Reta', 'Tombamento', 'Ingestão de álcool')

In [ ]:
df15_reta_tomb_alc.tracado_via.unique()

array(['Reta'], dtype=object)

In [ ]:
df15_reta_tomb_alc.tipo_acidente.unique()

array(['Tombamento'], dtype=object)

In [ ]:
df15_reta_tomb_alc.causa_acidente.unique()

array(['Ingestão de álcool'], dtype=object)

In [ ]:
counts, bins = np.histogram(df15_reta_tomb_alc['Hora'], bins=range(0, 25, 1))
acidentes15 = np.array(counts)/sum(np.array(counts))
fluxo15 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa15 = (acidentes15/fluxo15)/sum(acidentes15/fluxo15)

In [ ]:
graphics_plot_prf(y=[taxa15, fluxo15, acidentes15],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Overturning of one vehicle on a straight section of the road, only by alcohol intake (2015)',
                  x_tick=4, save=True, save_name='alc_reta_tomb_2015')

**2016**

In [ ]:
# Filtragem 'Reta' + 'Tombamento' + 'Ingestão de álcool'
df16_reta_tomb_alc = filtro_tracado_acidente_causa(df_prf16, 'Reta', 'Tombamento', 'Ingestão de álcool')

In [ ]:
df16_reta_tomb_alc.tracado_via.unique()

array(['Reta'], dtype=object)

In [ ]:
df16_reta_tomb_alc.tipo_acidente.unique()

array(['Tombamento'], dtype=object)

In [ ]:
df16_reta_tomb_alc.causa_acidente.unique()

array(['Ingestão de álcool'], dtype=object)

In [ ]:
counts, bins = np.histogram(df16_reta_tomb_alc['Hora'], bins=range(0, 25, 1))
acidentes16 = np.array(counts)/sum(np.array(counts))
fluxo16 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa16 = (acidentes16/fluxo16)/sum(acidentes16/fluxo16)

In [ ]:
graphics_plot_prf(y=[taxa16, fluxo16, acidentes16],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Overturning of one vehicle on a straight section of the road, only by alcohol intake (2016)',
                  x_tick=4, save=True, save_name='alc_reta_tomb_2016')

**2017**

In [ ]:
# Filtragem 'Reta' + 'Tombamento' + 'Ingestão de álcool'
df17_reta_tomb_alc = filtro_tracado_acidente_causa(df_prf17, 'Reta', 'Tombamento', 'Ingestão de Álcool')

In [ ]:
df17_reta_tomb_alc.tracado_via.unique()

array(['Reta'], dtype=object)

In [ ]:
df17_reta_tomb_alc.tipo_acidente.unique()

array(['Tombamento'], dtype=object)

In [ ]:
df17_reta_tomb_alc.causa_acidente.unique()

array(['Ingestão de Álcool'], dtype=object)

In [ ]:
counts, bins = np.histogram(df17_reta_tomb_alc['Hora'], bins=range(0, 25, 1))
acidentes17 = np.array(counts)/sum(np.array(counts))
fluxo17= np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa17 = (acidentes17/fluxo17)/sum(acidentes17/fluxo17)

In [ ]:
graphics_plot_prf(y=[taxa17, fluxo17, acidentes17],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Overturning of one vehicle on a straight section of the road, only by alcohol intake (2017)',
                  x_tick=4, save=True, save_name='alc_reta_tomb_2017')

In [ ]:
# df_novas_taxas = pd.DataFrame(data=range(1,25), columns=['hora'])
# # df_novas_taxas['reta_tomb_13'] = taxa13
# # df_novas_taxas['reta_tomb_14'] = taxa14
# df_novas_taxas['reta_tomb_15'] = taxa15
# df_novas_taxas['reta_tomb_16'] = taxa16
# df_novas_taxas['reta_tomb_17'] = taxa17

# df_novas_taxas.head(3)

## **2. Todos os veículos, em curva, um único veículo, tombamento**

**2015**

In [ ]:
# Filtragem 'Curva' + 'Tombamento' + 'Ingestão de álcool'
df15_curva_tomb_alc = filtro_tracado_acidente_causa(df_prf15, 'Curva', 'Tombamento', 'Ingestão de álcool')

In [ ]:
df15_curva_tomb_alc.tracado_via.unique()

array(['Curva'], dtype=object)

In [ ]:
df15_curva_tomb_alc.tipo_acidente.unique()

array(['Tombamento'], dtype=object)

In [ ]:
df15_curva_tomb_alc.causa_acidente.unique()

array(['Ingestão de álcool'], dtype=object)

In [ ]:
counts, bins = np.histogram(df15_curva_tomb_alc['Hora'], bins=range(0, 25, 1))
acidentes15 = np.array(counts)/sum(np.array(counts))
fluxo15 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa15 = (acidentes15/fluxo15)/sum(acidentes15/fluxo15)

In [ ]:
graphics_plot_prf(y=[taxa15, fluxo15, acidentes15],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Overturning of one vehicle on a curve section of the road, only by alcohol intake (2015)',
                  x_tick=4, save=True, save_name='alc_curva_tomb_2015')

**2016**

In [ ]:
# Filtragem 'Curva' + 'Tombamento' + 'Ingestão de álcool'
df16_curva_tomb_alc = filtro_tracado_acidente_causa(df_prf16, 'Curva', 'Tombamento', 'Ingestão de álcool')

In [ ]:
df16_curva_tomb_alc.tracado_via.unique()

array(['Curva'], dtype=object)

In [ ]:
df16_curva_tomb_alc.tipo_acidente.unique()

array(['Tombamento'], dtype=object)

In [ ]:
df16_curva_tomb_alc.causa_acidente.unique()

array(['Ingestão de álcool'], dtype=object)

In [ ]:
counts, bins = np.histogram(df16_curva_tomb_alc['Hora'], bins=range(0, 25, 1))
acidentes16 = np.array(counts)/sum(np.array(counts))
fluxo16 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa16 = (acidentes16/fluxo16)/sum(acidentes16/fluxo16)

In [ ]:
graphics_plot_prf(y=[taxa16, fluxo16, acidentes16],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Overturning of one vehicle on a curve section of the road, only by alcohol intake (2016)',
                  x_tick=4, save=True, save_name='alc_curva_tomb_2016')

**2017**

In [ ]:
# Filtragem 'Curva' + 'Tombamento' + 'Ingestão de álcool'
df17_curva_tomb_alc = filtro_tracado_acidente_causa(df_prf17, 'Curva', 'Tombamento', 'Ingestão de Álcool')

In [ ]:
df17_curva_tomb_alc.tracado_via.unique()

array(['Curva'], dtype=object)

In [ ]:
df17_curva_tomb_alc.tipo_acidente.unique()

array(['Tombamento'], dtype=object)

In [ ]:
df17_curva_tomb_alc.causa_acidente.unique()

array(['Ingestão de Álcool'], dtype=object)

In [ ]:
counts, bins = np.histogram(df17_curva_tomb_alc['Hora'], bins=range(0, 25, 1))
acidentes17 = np.array(counts)/sum(np.array(counts))
fluxo17= np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa17 = (acidentes17/fluxo17)/sum(acidentes17/fluxo17)

In [ ]:
graphics_plot_prf(y=[taxa17, fluxo17, acidentes17],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Overturning of one vehicle on a curve section of the road, only by alcohol intake (2017)',
                  x_tick=4, save=True, save_name='alc_curva_tomb_2017')

In [ ]:
# # df_novas_taxas['curva_tomb_13'] = taxa13
# # df_novas_taxas['curva_tomb_14'] = taxa14
# df_novas_taxas['curva_tomb_15'] = taxa15
# df_novas_taxas['curva_tomb_16'] = taxa16
# df_novas_taxas['curva_tomb_17'] = taxa17

# df_novas_taxas.head(3)

## **3. Todos os veículos, em linha reta, um único veículo, capotamento**

**2015**

In [ ]:
# Filtragem 'Reta' + 'Capotamento' + 'Ingestão de álcool'
df15_reta_capot_alc = filtro_tracado_acidente_causa(df_prf15, 'Reta', 'Capotamento', 'Ingestão de álcool')

In [ ]:
df15_reta_capot_alc.tracado_via.unique()

array(['Reta'], dtype=object)

In [ ]:
df15_reta_capot_alc.tipo_acidente.unique()

array(['Capotamento'], dtype=object)

In [ ]:
df15_reta_capot_alc.causa_acidente.unique()

array(['Ingestão de álcool'], dtype=object)

In [ ]:
counts, bins = np.histogram(df15_reta_capot_alc['Hora'], bins=range(0, 25, 1))
acidentes15 = np.array(counts)/sum(np.array(counts))
fluxo15 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa15 = (acidentes15/fluxo15)/sum(acidentes15/fluxo15)

In [ ]:
graphics_plot_prf(y=[taxa15, fluxo15, acidentes15],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Rollover of one vehicle on a straight section of the road, only by alcohol intake (2015)',
                  x_tick=4, save=True, save_name='alc_reta_capot_2015')

**2016**

In [ ]:
# Filtragem 'Reta' + 'Capotamento' + 'Ingestão de álcool'
df16_reta_capot_alc = filtro_tracado_acidente_causa(df_prf16, 'Reta', 'Capotamento', 'Ingestão de álcool')

In [ ]:
df16_reta_capot_alc.tracado_via.unique()

array(['Reta'], dtype=object)

In [ ]:
df16_reta_capot_alc.tipo_acidente.unique()

array(['Capotamento'], dtype=object)

In [ ]:
df16_reta_capot_alc.causa_acidente.unique()

array(['Ingestão de álcool'], dtype=object)

In [ ]:
counts, bins = np.histogram(df16_reta_capot_alc['Hora'], bins=range(0, 25, 1))
acidentes16 = np.array(counts)/sum(np.array(counts))
fluxo16 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa16 = (acidentes16/fluxo16)/sum(acidentes16/fluxo16)

In [ ]:
graphics_plot_prf(y=[taxa16, fluxo16, acidentes16],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Rollover of one vehicle on a straight section of the road, only by alcohol intake (2016)',
                  x_tick=4, save=True, save_name='alc_reta_capot_2016')

**2017**

In [ ]:
# Filtragem 'Reta' + 'Capotamento' + 'Ingestão de álcool'
df17_reta_capot_alc = filtro_tracado_acidente_causa(df_prf17, 'Reta', 'Capotamento', 'Ingestão de Álcool')

In [ ]:
df17_reta_capot_alc.tracado_via.unique()

array(['Reta'], dtype=object)

In [ ]:
df17_reta_capot_alc.tipo_acidente.unique()

array(['Capotamento'], dtype=object)

In [ ]:
df17_reta_capot_alc.causa_acidente.unique()

array(['Ingestão de Álcool'], dtype=object)

In [ ]:
counts, bins = np.histogram(df17_reta_capot_alc['Hora'], bins=range(0, 25, 1))
acidentes17 = np.array(counts)/sum(np.array(counts))
fluxo17= np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa17 = (acidentes17/fluxo17)/sum(acidentes17/fluxo17)

In [ ]:
graphics_plot_prf(y=[taxa17, fluxo17, acidentes17],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Rollover of one vehicle on a straight section of the road, only by alcohol intake (2017)',
                  x_tick=4, save=True, save_name='alc_reta_capot_2017')

In [ ]:
# # df_novas_taxas['reta_capot_13'] = taxa13
# # df_novas_taxas['reta_capot_14'] = taxa14
# df_novas_taxas['reta_capot_15'] = taxa15
# df_novas_taxas['reta_capot_16'] = taxa16
# df_novas_taxas['reta_capot_17'] = taxa17

# df_novas_taxas.head(3)

## **4. Todos os veículos, em curva, um único veículo, capotamento**

**2015**

In [ ]:
# Filtragem 'Curva' + 'Capotamento' + 'Ingestão de álcool'
df15_curva_capot_alc = filtro_tracado_acidente_causa(df_prf15, 'Curva', 'Capotamento', 'Ingestão de álcool')

In [ ]:
df15_curva_capot_alc.tracado_via.unique()

array(['Curva'], dtype=object)

In [ ]:
df15_curva_capot_alc.tipo_acidente.unique()

array(['Capotamento'], dtype=object)

In [ ]:
df15_curva_capot_alc.causa_acidente.unique()

array(['Ingestão de álcool'], dtype=object)

In [ ]:
counts, bins = np.histogram(df15_curva_capot_alc['Hora'], bins=range(0, 25, 1))
acidentes15 = np.array(counts)/sum(np.array(counts))
fluxo15 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa15 = (acidentes15/fluxo15)/sum(acidentes15/fluxo15)

In [ ]:
graphics_plot_prf(y=[taxa15, fluxo15, acidentes15],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Rollover of one vehicle on a curve section of the road, only by alcohol intake (2015)',
                  x_tick=4, save=True, save_name='alc_curva_capot_2015')

**2016**

In [ ]:
# Filtragem 'Curva' + 'Capotamento' + 'Ingestão de álcool'
df16_curva_capot_alc = filtro_tracado_acidente_causa(df_prf16, 'Curva', 'Capotamento', 'Ingestão de álcool')

In [ ]:
df16_curva_capot_alc.tracado_via.unique()

array(['Curva'], dtype=object)

In [ ]:
df16_curva_capot_alc.tipo_acidente.unique()

array(['Capotamento'], dtype=object)

In [ ]:
df16_curva_capot_alc.causa_acidente.unique()

array(['Ingestão de álcool'], dtype=object)

In [ ]:
counts, bins = np.histogram(df16_curva_capot_alc['Hora'], bins=range(0, 25, 1))
acidentes16 = np.array(counts)/sum(np.array(counts))
fluxo16 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa16 = (acidentes16/fluxo16)/sum(acidentes16/fluxo16)

In [ ]:
graphics_plot_prf(y=[taxa16, fluxo16, acidentes16],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Rollover of one vehicle on a curve section of the road, only by alcohol intake (2016)',
                  x_tick=4, save=True, save_name='alc_curva_capot_2016')

**2017**

In [ ]:
# Filtragem 'Curva' + 'Capotamento' + 'Ingestão de álcool'
df17_curva_capot_alc = filtro_tracado_acidente_causa(df_prf17, 'Curva', 'Capotamento', 'Ingestão de Álcool')

In [ ]:
df17_curva_capot_alc.tracado_via.unique()

array(['Curva'], dtype=object)

In [ ]:
df17_curva_capot_alc.tipo_acidente.unique()

array(['Capotamento'], dtype=object)

In [ ]:
df17_curva_capot_alc.causa_acidente.unique()

array(['Ingestão de Álcool'], dtype=object)

In [ ]:
counts, bins = np.histogram(df17_curva_capot_alc['Hora'], bins=range(0, 25, 1))
acidentes17 = np.array(counts)/sum(np.array(counts))
fluxo17= np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa17 = (acidentes17/fluxo17)/sum(acidentes17/fluxo17)

In [ ]:
graphics_plot_prf(y=[taxa17, fluxo17, acidentes17],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Rollover of one vehicle on a curve section of the road, only by alcohol intake (2017)',
                  x_tick=4, save=True, save_name='alc_curva_capot_2017')

In [ ]:
# # Download images folder
# !zip -r /content/images.zip /content/images

# from google.colab import files
# files.download('/content/images.zip')

# **Outras análises - Falta de atenção**

1.   Todos os veículos, em linha reta, um único veículo, tombamento
2.   Todos os veículos, em curva, um único veículo, tombamento
3.   Todos os veículos, em linha reta, um único veículo, capotamento
4.   Todos os veículos, em curva, um único veículo, capotamento

## **Dados**

In [ ]:
%%capture

# Fluxo
!gdown 1Kru8e3M6AjhEDuVXH7Tqx36wZjz4-Xc0

dfFluxo = pd.read_csv('DNIT.csv')
dfDNIT = pd.DataFrame(dfFluxo['avgtotal'].loc[2::3].values.reshape(4,24)).T

In [ ]:
%%capture
# !gdown 1rLrFckRbr_oScvziSCodjC2_UxoEOAMk # PRF 2013 - 1 veículo
# !gdown 1--JAT8gfd9CBtIlNBBj5yB_C5ipq9p0u # PRF 2014 - 1 veículo
!gdown 1-4vkJOgGW9W__P9zFhbVQkT3Uarwmbbv # PRF 2015 - 1 veículo
!gdown 1-6cj1H9JsUtdIZtO78x3HhdSFjpsqPj_ # PRF 2016 - 1 veículo
!gdown 1-6tc2J-omr-39k9n3FWApTqnJrZFIbnp # PRF 2017 - 1 veículo

# Datasets já filtrados para veículo único
# df_prf13 = pd.read_csv('prf_2013_1veiculo.csv')
# df_prf13['Hora'] = pd.to_datetime(df_prf13['horario'], format='%H:%M:%S').dt.hour
# df_prf14 = pd.read_csv('prf_2014_1veiculo.csv')
# df_prf14['Hora'] = pd.to_datetime(df_prf14['horario'], format='%H:%M:%S').dt.hour
df_prf15 = pd.read_csv('prf_2015_1veiculo.csv')
df_prf15['Hora'] = pd.to_datetime(df_prf15['horario'], format='%H:%M:%S').dt.hour
df_prf16 = pd.read_csv('prf_2016_1veiculo.csv')
df_prf16['Hora'] = pd.to_datetime(df_prf16['horario'], format='%H:%M:%S').dt.hour
df_prf17 = pd.read_csv('prf_2017_1veiculo.csv')
df_prf17['Hora'] = pd.to_datetime(df_prf17['horario'], format='%H:%M:%S').dt.hour

## **1. Todos os veículos, em linha reta, um único veículo, tombamento**

**2015**

In [ ]:
# Filtragem 'Reta' + 'Tombamento' + 'Falta de atenção'
df15_reta_tomb_aten = filtro_tracado_acidente_causa(df_prf15, 'Reta', 'Tombamento', 'Falta de atenção')

In [ ]:
df15_reta_tomb_aten.tracado_via.unique()

array(['Reta'], dtype=object)

In [ ]:
df15_reta_tomb_aten.tipo_acidente.unique()

array(['Tombamento'], dtype=object)

In [ ]:
df15_reta_tomb_aten.causa_acidente.unique()

array(['Falta de atenção'], dtype=object)

In [ ]:
counts, bins = np.histogram(df15_reta_tomb_aten['Hora'], bins=range(0, 25, 1))
acidentes15 = np.array(counts)/sum(np.array(counts))
fluxo15 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa15 = (acidentes15/fluxo15)/sum(acidentes15/fluxo15)

In [ ]:
graphics_plot_prf(y=[taxa15, fluxo15, acidentes15],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Overturning of one vehicle on a straight section of the road, only by lack of attention (2015)',
                  x_tick=4, save=True, save_name='aten_reta_tomb_2015')

**2016**

In [ ]:
# Filtragem 'Reta' + 'Tombamento' + 'Falta de atenção'
df16_reta_tomb_aten = filtro_tracado_acidente_causa(df_prf16, 'Reta', 'Tombamento', 'Falta de atenção')

In [ ]:
df16_reta_tomb_aten.tracado_via.unique()

array(['Reta'], dtype=object)

In [ ]:
df16_reta_tomb_aten.tipo_acidente.unique()

array(['Tombamento'], dtype=object)

In [ ]:
df16_reta_tomb_aten.causa_acidente.unique()

array(['Falta de atenção'], dtype=object)

In [ ]:
counts, bins = np.histogram(df16_reta_tomb_aten['Hora'], bins=range(0, 25, 1))
acidentes16 = np.array(counts)/sum(np.array(counts))
fluxo16 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa16 = (acidentes16/fluxo16)/sum(acidentes16/fluxo16)

In [ ]:
graphics_plot_prf(y=[taxa16, fluxo16, acidentes16],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Overturning of one vehicle on a straight section of the road, only by lack of attention (2016)',
                  x_tick=4, save=True, save_name='aten_reta_tomb_2016')

**2017**

In [ ]:
# Filtragem 'Reta' + 'Tombamento' + 'Falta de atenção'
df17_reta_tomb_aten = filtro_tracado_acidente_causa(df_prf17, 'Reta', 'Tombamento', 'Falta de Atenção à Condução')

In [ ]:
df17_reta_tomb_aten.tracado_via.unique()

array(['Reta'], dtype=object)

In [ ]:
df17_reta_tomb_aten.tipo_acidente.unique()

array(['Tombamento'], dtype=object)

In [ ]:
df17_reta_tomb_aten.causa_acidente.unique()

array(['Falta de Atenção à Condução'], dtype=object)

In [ ]:
counts, bins = np.histogram(df17_reta_tomb_aten['Hora'], bins=range(0, 25, 1))
acidentes17 = np.array(counts)/sum(np.array(counts))
fluxo17= np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa17 = (acidentes17/fluxo17)/sum(acidentes17/fluxo17)

In [ ]:
graphics_plot_prf(y=[taxa17, fluxo17, acidentes17],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Overturning of one vehicle on a straight section of the road, only by lack of attention (2017)',
                  x_tick=4, save=True, save_name='aten_reta_tomb_2017')

In [ ]:
# df_novas_taxas = pd.DataFrame(data=range(1,25), columns=['hora'])
# # df_novas_taxas['reta_tomb_13'] = taxa13
# # df_novas_taxas['reta_tomb_14'] = taxa14
# df_novas_taxas['reta_tomb_15'] = taxa15
# df_novas_taxas['reta_tomb_16'] = taxa16
# df_novas_taxas['reta_tomb_17'] = taxa17

# df_novas_taxas.head(3)

## **2. Todos os veículos, em curva, um único veículo, tombamento**

**2015**

In [ ]:
# Filtragem 'Curva' + 'Tombamento' + 'Falta de atenção'
df15_curva_tomb_aten = filtro_tracado_acidente_causa(df_prf15, 'Curva', 'Tombamento', 'Falta de atenção')

In [ ]:
df15_curva_tomb_aten.tracado_via.unique()

array(['Curva'], dtype=object)

In [ ]:
df15_curva_tomb_aten.tipo_acidente.unique()

array(['Tombamento'], dtype=object)

In [ ]:
df15_curva_tomb_aten.causa_acidente.unique()

array(['Falta de atenção'], dtype=object)

In [ ]:
counts, bins = np.histogram(df15_curva_tomb_aten['Hora'], bins=range(0, 25, 1))
acidentes15 = np.array(counts)/sum(np.array(counts))
fluxo15 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa15 = (acidentes15/fluxo15)/sum(acidentes15/fluxo15)

In [ ]:
graphics_plot_prf(y=[taxa15, fluxo15, acidentes15],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Overturning of one vehicle on a curve section of the road, only by lack of attention (2015)',
                  x_tick=4, save=True, save_name='aten_curva_tomb_2015')

**2016**

In [ ]:
# Filtragem 'Curva' + 'Tombamento' + 'Falta de atenção'
df16_curva_tomb_aten = filtro_tracado_acidente_causa(df_prf16, 'Curva', 'Tombamento', 'Falta de atenção')

In [ ]:
df16_curva_tomb_aten.tracado_via.unique()

array(['Curva'], dtype=object)

In [ ]:
df16_curva_tomb_aten.tipo_acidente.unique()

array(['Tombamento'], dtype=object)

In [ ]:
df16_curva_tomb_aten.causa_acidente.unique()

array(['Falta de atenção'], dtype=object)

In [ ]:
counts, bins = np.histogram(df16_curva_tomb_aten['Hora'], bins=range(0, 25, 1))
acidentes16 = np.array(counts)/sum(np.array(counts))
fluxo16 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa16 = (acidentes16/fluxo16)/sum(acidentes16/fluxo16)

In [ ]:
graphics_plot_prf(y=[taxa16, fluxo16, acidentes16],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Overturning of one vehicle on a curve section of the road, only by lack of attention (2016)',
                  x_tick=4, save=True, save_name='aten_curva_tomb_2016')

**2017**

In [ ]:
# Filtragem 'Curva' + 'Tombamento' + 'Falta de atenção'
df17_curva_tomb_aten = filtro_tracado_acidente_causa(df_prf17, 'Curva', 'Tombamento', 'Falta de Atenção à Condução')

In [ ]:
df17_curva_tomb_aten.tracado_via.unique()

array(['Curva'], dtype=object)

In [ ]:
df17_curva_tomb_aten.tipo_acidente.unique()

array(['Tombamento'], dtype=object)

In [ ]:
df17_curva_tomb_aten.causa_acidente.unique()

array(['Falta de Atenção à Condução'], dtype=object)

In [ ]:
counts, bins = np.histogram(df17_curva_tomb_aten['Hora'], bins=range(0, 25, 1))
acidentes17 = np.array(counts)/sum(np.array(counts))
fluxo17= np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa17 = (acidentes17/fluxo17)/sum(acidentes17/fluxo17)

In [ ]:
graphics_plot_prf(y=[taxa17, fluxo17, acidentes17],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Overturning of one vehicle on a curve section of the road, only by lack of attention (2017)',
                  x_tick=4, save=True, save_name='aten_curva_tomb_2017')

In [ ]:
# # df_novas_taxas['curva_tomb_13'] = taxa13
# # df_novas_taxas['curva_tomb_14'] = taxa14
# df_novas_taxas['curva_tomb_15'] = taxa15
# df_novas_taxas['curva_tomb_16'] = taxa16
# df_novas_taxas['curva_tomb_17'] = taxa17

# df_novas_taxas.head(3)

## **3. Todos os veículos, em linha reta, um único veículo, capotamento**

**2015**

In [ ]:
# Filtragem 'Reta' + 'Capotamento' + 'Falta de atenção'
df15_reta_capot_aten = filtro_tracado_acidente_causa(df_prf15, 'Reta', 'Capotamento', 'Falta de atenção')

In [ ]:
df15_reta_capot_aten.tracado_via.unique()

array(['Reta'], dtype=object)

In [ ]:
df15_reta_capot_aten.tipo_acidente.unique()

array(['Capotamento'], dtype=object)

In [ ]:
df15_reta_capot_aten.causa_acidente.unique()

array(['Falta de atenção'], dtype=object)

In [ ]:
counts, bins = np.histogram(df15_reta_capot_aten['Hora'], bins=range(0, 25, 1))
acidentes15 = np.array(counts)/sum(np.array(counts))
fluxo15 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa15 = (acidentes15/fluxo15)/sum(acidentes15/fluxo15)

In [ ]:
graphics_plot_prf(y=[taxa15, fluxo15, acidentes15],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Rollover of one vehicle on a straight section of the road, only by lack of attention (2015)',
                  x_tick=4, save=True, save_name='aten_reta_capot_2015')

**2016**

In [ ]:
# Filtragem 'Reta' + 'Capotamento' + 'Falta de atenção'
df16_reta_capot_aten = filtro_tracado_acidente_causa(df_prf16, 'Reta', 'Capotamento', 'Falta de atenção')

In [ ]:
df16_reta_capot_aten.tracado_via.unique()

array(['Reta'], dtype=object)

In [ ]:
df16_reta_capot_aten.tipo_acidente.unique()

array(['Capotamento'], dtype=object)

In [ ]:
df16_reta_capot_aten.causa_acidente.unique()

array(['Falta de atenção'], dtype=object)

In [ ]:
counts, bins = np.histogram(df16_reta_capot_aten['Hora'], bins=range(0, 25, 1))
acidentes16 = np.array(counts)/sum(np.array(counts))
fluxo16 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa16 = (acidentes16/fluxo16)/sum(acidentes16/fluxo16)

In [ ]:
graphics_plot_prf(y=[taxa16, fluxo16, acidentes16],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Rollover of one vehicle on a straight section of the road, only by lack of attention (2016)',
                  x_tick=4, save=True, save_name='aten_reta_capot_2016')

**2017**

In [ ]:
# Filtragem 'Reta' + 'Capotamento' + 'Falta de atenção'
df17_reta_capot_aten = filtro_tracado_acidente_causa(df_prf17, 'Reta', 'Capotamento', 'Falta de Atenção à Condução')

In [ ]:
df17_reta_capot_aten.tracado_via.unique()

array(['Reta'], dtype=object)

In [ ]:
df17_reta_capot_aten.tipo_acidente.unique()

array(['Capotamento'], dtype=object)

In [ ]:
df17_reta_capot_aten.causa_acidente.unique()

array(['Falta de Atenção à Condução'], dtype=object)

In [ ]:
counts, bins = np.histogram(df17_reta_capot_aten['Hora'], bins=range(0, 25, 1))
acidentes17 = np.array(counts)/sum(np.array(counts))
fluxo17= np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa17 = (acidentes17/fluxo17)/sum(acidentes17/fluxo17)

In [ ]:
graphics_plot_prf(y=[taxa17, fluxo17, acidentes17],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Rollover of one vehicle on a straight section of the road, only by lack of attention (2017)',
                  x_tick=4, save=True, save_name='aten_reta_capot_2017')

In [ ]:
# # df_novas_taxas['reta_capot_13'] = taxa13
# # df_novas_taxas['reta_capot_14'] = taxa14
# df_novas_taxas['reta_capot_15'] = taxa15
# df_novas_taxas['reta_capot_16'] = taxa16
# df_novas_taxas['reta_capot_17'] = taxa17

# df_novas_taxas.head(3)

## **4. Todos os veículos, em curva, um único veículo, capotamento**

**2015**

In [ ]:
# Filtragem 'Curva' + 'Capotamento' + 'Falta de atenção'
df15_curva_capot_aten = filtro_tracado_acidente_causa(df_prf15, 'Curva', 'Capotamento', 'Falta de atenção')

In [ ]:
df15_curva_capot_aten.tracado_via.unique()

array(['Curva'], dtype=object)

In [ ]:
df15_curva_capot_aten.tipo_acidente.unique()

array(['Capotamento'], dtype=object)

In [ ]:
df15_curva_capot_aten.causa_acidente.unique()

array(['Falta de atenção'], dtype=object)

In [ ]:
counts, bins = np.histogram(df15_curva_capot_aten['Hora'], bins=range(0, 25, 1))
acidentes15 = np.array(counts)/sum(np.array(counts))
fluxo15 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa15 = (acidentes15/fluxo15)/sum(acidentes15/fluxo15)

In [ ]:
graphics_plot_prf(y=[taxa15, fluxo15, acidentes15],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Rollover of one vehicle on a curve section of the road, only by lack of attention (2015)',
                  x_tick=4, save=True, save_name='aten_curva_capot_2015')

**2016**

In [ ]:
# Filtragem 'Curva' + 'Capotamento' + 'Falta de atenção'
df16_curva_capot_aten = filtro_tracado_acidente_causa(df_prf16, 'Curva', 'Capotamento', 'Falta de atenção')

In [ ]:
df16_curva_capot_aten.tracado_via.unique()

array(['Curva'], dtype=object)

In [ ]:
df16_curva_capot_aten.tipo_acidente.unique()

array(['Capotamento'], dtype=object)

In [ ]:
df16_curva_capot_aten.causa_acidente.unique()

array(['Falta de atenção'], dtype=object)

In [ ]:
counts, bins = np.histogram(df16_curva_capot_aten['Hora'], bins=range(0, 25, 1))
acidentes16 = np.array(counts)/sum(np.array(counts))
fluxo16 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa16 = (acidentes16/fluxo16)/sum(acidentes16/fluxo16)

In [ ]:
graphics_plot_prf(y=[taxa16, fluxo16, acidentes16],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Rollover of one vehicle on a curve section of the road, only by lack of attention (2016)',
                  x_tick=4, save=True, save_name='aten_curva_capot_2016')

**2017**

In [ ]:
# Filtragem 'Curva' + 'Capotamento' + 'Falta de atenção'
df17_curva_capot_aten = filtro_tracado_acidente_causa(df_prf17, 'Curva', 'Capotamento', 'Falta de Atenção à Condução')

In [ ]:
df17_curva_capot_aten.tracado_via.unique()

array(['Curva'], dtype=object)

In [ ]:
df17_curva_capot_aten.tipo_acidente.unique()

array(['Capotamento'], dtype=object)

In [ ]:
df17_curva_capot_aten.causa_acidente.unique()

array(['Falta de Atenção à Condução'], dtype=object)

In [ ]:
counts, bins = np.histogram(df17_curva_capot_aten['Hora'], bins=range(0, 25, 1))
acidentes17 = np.array(counts)/sum(np.array(counts))
fluxo17= np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa17 = (acidentes17/fluxo17)/sum(acidentes17/fluxo17)

In [ ]:
graphics_plot_prf(y=[taxa17, fluxo17, acidentes17],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Rollover of one vehicle on a curve section of the road, only by lack of attention (2017)',
                  x_tick=4, save=True, save_name='aten_curva_capot_2017')

In [ ]:
# Download images folder
!zip -r /content/images.zip /content/images

from google.colab import files
files.download('/content/images.zip')

# **Outras análises - Dormindo**

1.   Todos os veículos, em linha reta, um único veículo, tombamento
2.   Todos os veículos, em curva, um único veículo, tombamento
3.   Todos os veículos, em linha reta, um único veículo, capotamento
4.   Todos os veículos, em curva, um único veículo, capotamento

## **Dados**

In [ ]:
%%capture

# Fluxo
!gdown 1Kru8e3M6AjhEDuVXH7Tqx36wZjz4-Xc0

dfFluxo = pd.read_csv('DNIT.csv')
dfDNIT = pd.DataFrame(dfFluxo['avgtotal'].loc[2::3].values.reshape(4,24)).T

In [ ]:
%%capture
# !gdown 1rLrFckRbr_oScvziSCodjC2_UxoEOAMk # PRF 2013 - 1 veículo
# !gdown 1--JAT8gfd9CBtIlNBBj5yB_C5ipq9p0u # PRF 2014 - 1 veículo
!gdown 1-4vkJOgGW9W__P9zFhbVQkT3Uarwmbbv # PRF 2015 - 1 veículo
!gdown 1-6cj1H9JsUtdIZtO78x3HhdSFjpsqPj_ # PRF 2016 - 1 veículo
!gdown 1-6tc2J-omr-39k9n3FWApTqnJrZFIbnp # PRF 2017 - 1 veículo

# Datasets já filtrados para veículo único
# df_prf13 = pd.read_csv('prf_2013_1veiculo.csv')
# df_prf13['Hora'] = pd.to_datetime(df_prf13['horario'], format='%H:%M:%S').dt.hour
# df_prf14 = pd.read_csv('prf_2014_1veiculo.csv')
# df_prf14['Hora'] = pd.to_datetime(df_prf14['horario'], format='%H:%M:%S').dt.hour
df_prf15 = pd.read_csv('prf_2015_1veiculo.csv')
df_prf15['Hora'] = pd.to_datetime(df_prf15['horario'], format='%H:%M:%S').dt.hour
df_prf16 = pd.read_csv('prf_2016_1veiculo.csv')
df_prf16['Hora'] = pd.to_datetime(df_prf16['horario'], format='%H:%M:%S').dt.hour
df_prf17 = pd.read_csv('prf_2017_1veiculo.csv')
df_prf17['Hora'] = pd.to_datetime(df_prf17['horario'], format='%H:%M:%S').dt.hour

## **1. Todos os veículos, em linha reta, um único veículo, tombamento**

**2015**

In [ ]:
# Filtragem 'Reta' + 'Tombamento' + 'Dormindo'
df15_reta_tomb_dorm = filtro_tracado_acidente_causa(df_prf15, 'Reta', 'Tombamento', 'Dormindo')

In [ ]:
df15_reta_tomb_dorm.tracado_via.unique()

array(['Reta'], dtype=object)

In [ ]:
df15_reta_tomb_dorm.tipo_acidente.unique()

array(['Tombamento'], dtype=object)

In [ ]:
df15_reta_tomb_dorm.causa_acidente.unique()

array(['Dormindo'], dtype=object)

In [ ]:
counts, bins = np.histogram(df15_reta_tomb_dorm['Hora'], bins=range(0, 25, 1))
acidentes15 = np.array(counts)/sum(np.array(counts))
fluxo15 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa15 = (acidentes15/fluxo15)/sum(acidentes15/fluxo15)

In [ ]:
graphics_plot_prf(y=[taxa15, fluxo15, acidentes15],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Overturning of one vehicle on a straight section of the road, only by sleeping (2015)',
                  x_tick=4, save=True, save_name='dorm_reta_tomb_2015')

**2016**

In [ ]:
# Filtragem 'Reta' + 'Tombamento' + 'Dormindo'
df16_reta_tomb_dorm = filtro_tracado_acidente_causa(df_prf16, 'Reta', 'Tombamento', 'Dormindo')

In [ ]:
df16_reta_tomb_dorm.tracado_via.unique()

array(['Reta'], dtype=object)

In [ ]:
df16_reta_tomb_dorm.tipo_acidente.unique()

array(['Tombamento'], dtype=object)

In [ ]:
df16_reta_tomb_dorm.causa_acidente.unique()

array(['Dormindo'], dtype=object)

In [ ]:
counts, bins = np.histogram(df16_reta_tomb_dorm['Hora'], bins=range(0, 25, 1))
acidentes16 = np.array(counts)/sum(np.array(counts))
fluxo16 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa16 = (acidentes16/fluxo16)/sum(acidentes16/fluxo16)

In [ ]:
graphics_plot_prf(y=[taxa16, fluxo16, acidentes16],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Overturning of one vehicle on a straight section of the road, only by sleeping (2016)',
                  x_tick=4, save=True, save_name='dorm_reta_tomb_2016')

**2017**

In [ ]:
# Filtragem 'Reta' + 'Tombamento' + 'Dormindo'
df17_reta_tomb_dorm = filtro_tracado_acidente_causa(df_prf17, 'Reta', 'Tombamento', 'Condutor Dormindo')

In [ ]:
df17_reta_tomb_dorm.tracado_via.unique()

array(['Reta'], dtype=object)

In [ ]:
df17_reta_tomb_dorm.tipo_acidente.unique()

array(['Tombamento'], dtype=object)

In [ ]:
df17_reta_tomb_dorm.causa_acidente.unique()

array(['Condutor Dormindo'], dtype=object)

In [ ]:
counts, bins = np.histogram(df17_reta_tomb_dorm['Hora'], bins=range(0, 25, 1))
acidentes17 = np.array(counts)/sum(np.array(counts))
fluxo17= np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa17 = (acidentes17/fluxo17)/sum(acidentes17/fluxo17)

In [ ]:
graphics_plot_prf(y=[taxa17, fluxo17, acidentes17],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Overturning of one vehicle on a straight section of the road, only by sleeping (2017)',
                  x_tick=4, save=True, save_name='dorm_reta_tomb_2017')

In [ ]:
# df_novas_taxas = pd.DataFrame(data=range(1,25), columns=['hora'])
# # df_novas_taxas['reta_tomb_13'] = taxa13
# # df_novas_taxas['reta_tomb_14'] = taxa14
# df_novas_taxas['reta_tomb_15'] = taxa15
# df_novas_taxas['reta_tomb_16'] = taxa16
# df_novas_taxas['reta_tomb_17'] = taxa17

# df_novas_taxas.head(3)

## **2. Todos os veículos, em curva, um único veículo, tombamento**

**2015**

In [ ]:
# Filtragem 'Curva' + 'Tombamento' + 'Dormindo'
df15_curva_tomb_dorm = filtro_tracado_acidente_causa(df_prf15, 'Curva', 'Tombamento', 'Dormindo')

In [ ]:
df15_curva_tomb_dorm.tracado_via.unique()

array(['Curva'], dtype=object)

In [ ]:
df15_curva_tomb_dorm.tipo_acidente.unique()

array(['Tombamento'], dtype=object)

In [ ]:
df15_curva_tomb_dorm.causa_acidente.unique()

array(['Dormindo'], dtype=object)

In [ ]:
counts, bins = np.histogram(df15_curva_tomb_dorm['Hora'], bins=range(0, 25, 1))
acidentes15 = np.array(counts)/sum(np.array(counts))
fluxo15 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa15 = (acidentes15/fluxo15)/sum(acidentes15/fluxo15)

In [ ]:
graphics_plot_prf(y=[taxa15, fluxo15, acidentes15],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Overturning of one vehicle on a curve section of the road, only by sleeping (2015)',
                  x_tick=4, save=True, save_name='dorm_curva_tomb_2015')

**2016**

In [ ]:
# Filtragem 'Curva' + 'Tombamento' + 'Dormindo'
df16_curva_tomb_dorm = filtro_tracado_acidente_causa(df_prf16, 'Curva', 'Tombamento', 'Dormindo')

In [ ]:
df16_curva_tomb_dorm.tracado_via.unique()

array(['Curva'], dtype=object)

In [ ]:
df16_curva_tomb_dorm.tipo_acidente.unique()

array(['Tombamento'], dtype=object)

In [ ]:
df16_curva_tomb_dorm.causa_acidente.unique()

array(['Dormindo'], dtype=object)

In [ ]:
counts, bins = np.histogram(df16_curva_tomb_dorm['Hora'], bins=range(0, 25, 1))
acidentes16 = np.array(counts)/sum(np.array(counts))
fluxo16 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa16 = (acidentes16/fluxo16)/sum(acidentes16/fluxo16)

In [ ]:
graphics_plot_prf(y=[taxa16, fluxo16, acidentes16],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Overturning of one vehicle on a curve section of the road, only by sleeping (2016)',
                  x_tick=4, save=True, save_name='dorm_curva_tomb_2016')

**2017**

In [ ]:
# Filtragem 'Curva' + 'Tombamento' + 'Dormindo'
df17_curva_tomb_dorm = filtro_tracado_acidente_causa(df_prf17, 'Curva', 'Tombamento', 'Condutor Dormindo')

In [ ]:
df17_curva_tomb_dorm.tracado_via.unique()

array(['Curva'], dtype=object)

In [ ]:
df17_curva_tomb_dorm.tipo_acidente.unique()

array(['Tombamento'], dtype=object)

In [ ]:
df17_curva_tomb_dorm.causa_acidente.unique()

array(['Condutor Dormindo'], dtype=object)

In [ ]:
counts, bins = np.histogram(df17_curva_tomb_dorm['Hora'], bins=range(0, 25, 1))
acidentes17 = np.array(counts)/sum(np.array(counts))
fluxo17= np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa17 = (acidentes17/fluxo17)/sum(acidentes17/fluxo17)

In [ ]:
graphics_plot_prf(y=[taxa17, fluxo17, acidentes17],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Overturning of one vehicle on a curve section of the road, only by sleeping (2017)',
                  x_tick=4, save=True, save_name='dorm_curva_tomb_2017')

In [ ]:
# # df_novas_taxas['curva_tomb_13'] = taxa13
# # df_novas_taxas['curva_tomb_14'] = taxa14
# df_novas_taxas['curva_tomb_15'] = taxa15
# df_novas_taxas['curva_tomb_16'] = taxa16
# df_novas_taxas['curva_tomb_17'] = taxa17

# df_novas_taxas.head(3)

## **3. Todos os veículos, em linha reta, um único veículo, capotamento**

**2015**

In [ ]:
# Filtragem 'Reta' + 'Capotamento' + 'Dormindo'
df15_reta_capot_dorm = filtro_tracado_acidente_causa(df_prf15, 'Reta', 'Capotamento', 'Dormindo')

In [ ]:
df15_reta_capot_dorm.tracado_via.unique()

array(['Reta'], dtype=object)

In [ ]:
df15_reta_capot_dorm.tipo_acidente.unique()

array(['Capotamento'], dtype=object)

In [ ]:
df15_reta_capot_dorm.causa_acidente.unique()

array(['Dormindo'], dtype=object)

In [ ]:
counts, bins = np.histogram(df15_reta_capot_dorm['Hora'], bins=range(0, 25, 1))
acidentes15 = np.array(counts)/sum(np.array(counts))
fluxo15 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa15 = (acidentes15/fluxo15)/sum(acidentes15/fluxo15)

In [ ]:
graphics_plot_prf(y=[taxa15, fluxo15, acidentes15],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Rollover of one vehicle on a straight section of the road, only by sleeping (2015)',
                  x_tick=4, save=True, save_name='dorm_reta_capot_2015')

**2016**

In [ ]:
# Filtragem 'Reta' + 'Capotamento' + 'Dormindo'
df16_reta_capot_dorm = filtro_tracado_acidente_causa(df_prf16, 'Reta', 'Capotamento', 'Dormindo')

In [ ]:
df16_reta_capot_dorm.tracado_via.unique()

array(['Reta'], dtype=object)

In [ ]:
df16_reta_capot_dorm.tipo_acidente.unique()

array(['Capotamento'], dtype=object)

In [ ]:
df16_reta_capot_dorm.causa_acidente.unique()

array(['Dormindo'], dtype=object)

In [ ]:
counts, bins = np.histogram(df16_reta_capot_dorm['Hora'], bins=range(0, 25, 1))
acidentes16 = np.array(counts)/sum(np.array(counts))
fluxo16 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa16 = (acidentes16/fluxo16)/sum(acidentes16/fluxo16)

In [ ]:
graphics_plot_prf(y=[taxa16, fluxo16, acidentes16],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Rollover of one vehicle on a straight section of the road, only by sleeping (2016)',
                  x_tick=4, save=True, save_name='dorm_reta_capot_2016')

**2017**

In [ ]:
# Filtragem 'Reta' + 'Capotamento' + 'Dormindo'
df17_reta_capot_dorm = filtro_tracado_acidente_causa(df_prf17, 'Reta', 'Capotamento', 'Condutor Dormindo')

In [ ]:
df17_reta_capot_dorm.tracado_via.unique()

array(['Reta'], dtype=object)

In [ ]:
df17_reta_capot_dorm.tipo_acidente.unique()

array(['Capotamento'], dtype=object)

In [ ]:
df17_reta_capot_dorm.causa_acidente.unique()

array(['Condutor Dormindo'], dtype=object)

In [ ]:
counts, bins = np.histogram(df17_reta_capot_dorm['Hora'], bins=range(0, 25, 1))
acidentes17 = np.array(counts)/sum(np.array(counts))
fluxo17= np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa17 = (acidentes17/fluxo17)/sum(acidentes17/fluxo17)

In [ ]:
graphics_plot_prf(y=[taxa17, fluxo17, acidentes17],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Rollover of one vehicle on a straight section of the road, only by sleeping (2017)',
                  x_tick=4, save=True, save_name='dorm_reta_capot_2017')

In [ ]:
# # df_novas_taxas['reta_capot_13'] = taxa13
# # df_novas_taxas['reta_capot_14'] = taxa14
# df_novas_taxas['reta_capot_15'] = taxa15
# df_novas_taxas['reta_capot_16'] = taxa16
# df_novas_taxas['reta_capot_17'] = taxa17

# df_novas_taxas.head(3)

## **4. Todos os veículos, em curva, um único veículo, capotamento**

**2015**

In [ ]:
# Filtragem 'Curva' + 'Capotamento' + 'Dormindo'
df15_curva_capot_dorm = filtro_tracado_acidente_causa(df_prf15, 'Curva', 'Capotamento', 'Dormindo')

In [ ]:
df15_curva_capot_dorm.tracado_via.unique()

array(['Curva'], dtype=object)

In [ ]:
df15_curva_capot_dorm.tipo_acidente.unique()

array(['Capotamento'], dtype=object)

In [ ]:
df15_curva_capot_dorm.causa_acidente.unique()

array(['Dormindo'], dtype=object)

In [ ]:
counts, bins = np.histogram(df15_curva_capot_dorm['Hora'], bins=range(0, 25, 1))
acidentes15 = np.array(counts)/sum(np.array(counts))
fluxo15 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa15 = (acidentes15/fluxo15)/sum(acidentes15/fluxo15)

In [ ]:
graphics_plot_prf(y=[taxa15, fluxo15, acidentes15],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Rollover of one vehicle on a curve section of the road, only by sleeping (2015)',
                  x_tick=4, save=True, save_name='dorm_curva_capot_2015')

**2016**

In [ ]:
# Filtragem 'Curva' + 'Capotamento' + 'Dormindo'
df16_curva_capot_dorm = filtro_tracado_acidente_causa(df_prf16, 'Curva', 'Capotamento', 'Dormindo')

In [ ]:
df16_curva_capot_dorm.tracado_via.unique()

array(['Curva'], dtype=object)

In [ ]:
df16_curva_capot_dorm.tipo_acidente.unique()

array(['Capotamento'], dtype=object)

In [ ]:
df16_curva_capot_dorm.causa_acidente.unique()

array(['Dormindo'], dtype=object)

In [ ]:
counts, bins = np.histogram(df16_curva_capot_dorm['Hora'], bins=range(0, 25, 1))
acidentes16 = np.array(counts)/sum(np.array(counts))
fluxo16 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa16 = (acidentes16/fluxo16)/sum(acidentes16/fluxo16)

In [ ]:
graphics_plot_prf(y=[taxa16, fluxo16, acidentes16],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Rollover of one vehicle on a curve section of the road, only by sleeping (2016)',
                  x_tick=4, save=True, save_name='dorm_curva_capot_2016')

**2017**

In [ ]:
# Filtragem 'Curva' + 'Capotamento' + 'Dormindo'
df17_curva_capot_dorm = filtro_tracado_acidente_causa(df_prf17, 'Curva', 'Capotamento', 'Condutor Dormindo')

In [ ]:
df17_curva_capot_dorm.tracado_via.unique()

array(['Curva'], dtype=object)

In [ ]:
df17_curva_capot_dorm.tipo_acidente.unique()

array(['Capotamento'], dtype=object)

In [ ]:
df17_curva_capot_dorm.causa_acidente.unique()

array(['Condutor Dormindo'], dtype=object)

In [ ]:
counts, bins = np.histogram(df17_curva_capot_dorm['Hora'], bins=range(0, 25, 1))
acidentes17 = np.array(counts)/sum(np.array(counts))
fluxo17= np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa17 = (acidentes17/fluxo17)/sum(acidentes17/fluxo17)

In [ ]:
graphics_plot_prf(y=[taxa17, fluxo17, acidentes17],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Rollover of one vehicle on a curve section of the road, only by sleeping (2017)',
                  x_tick=4, save=True, save_name='dorm_curva_capot_2017')

In [ ]:
# Download images folder
!zip -r /content/images.zip /content/images

from google.colab import files
files.download('/content/images.zip')

# **+ 3 análises (Somente 2015, 2016 e 2017)**

1. Um único veículo, capotamento e tombamento em curva e reta

2. Um único veículo, capotamento em curva e reta

3. Um único veículo, tombamento em curva e reta

## **Dados**

In [ ]:
%%capture

# Fluxo
!gdown 1Kru8e3M6AjhEDuVXH7Tqx36wZjz4-Xc0

dfFluxo = pd.read_csv('DNIT.csv')
dfDNIT = pd.DataFrame(dfFluxo['avgtotal'].loc[2::3].values.reshape(4,24)).T

In [ ]:
%%capture
# !gdown 1rLrFckRbr_oScvziSCodjC2_UxoEOAMk # PRF 2013 - 1 veículo
# !gdown 1--JAT8gfd9CBtIlNBBj5yB_C5ipq9p0u # PRF 2014 - 1 veículo
!gdown 1-4vkJOgGW9W__P9zFhbVQkT3Uarwmbbv # PRF 2015 - 1 veículo
!gdown 1-6cj1H9JsUtdIZtO78x3HhdSFjpsqPj_ # PRF 2016 - 1 veículo
!gdown 1-6tc2J-omr-39k9n3FWApTqnJrZFIbnp # PRF 2017 - 1 veículo

# Datasets já filtrados para veículo único
# df_prf13 = pd.read_csv('prf_2013_1veiculo.csv')
# df_prf13['Hora'] = pd.to_datetime(df_prf13['horario'], format='%H:%M:%S').dt.hour
# df_prf14 = pd.read_csv('prf_2014_1veiculo.csv')
# df_prf14['Hora'] = pd.to_datetime(df_prf14['horario'], format='%H:%M:%S').dt.hour
df_prf15 = pd.read_csv('prf_2015_1veiculo.csv')
df_prf15['Hora'] = pd.to_datetime(df_prf15['horario'], format='%H:%M:%S').dt.hour
df_prf16 = pd.read_csv('prf_2016_1veiculo.csv')
df_prf16['Hora'] = pd.to_datetime(df_prf16['horario'], format='%H:%M:%S').dt.hour
df_prf17 = pd.read_csv('prf_2017_1veiculo.csv')
df_prf17['Hora'] = pd.to_datetime(df_prf17['horario'], format='%H:%M:%S').dt.hour

## **1. Um único veículo, capotamento e tombamento em curva e reta**

**2015**

In [ ]:
df_prf15.tipo_veiculo.unique()

array(['Automóvel', 'Caminhão', 'Motocicletas', 'Caminhão-Trator',
       '(null)', 'Camioneta', 'Motoneta', 'Utilitário', 'Caminhonete',
       'Microônibus', 'Não identificado', 'Ônibus', 'Bicicleta',
       'Ciclomotor', 'Trator de rodas', 'Caminhão-Tanque', 'Triciclo',
       'Trator de esteiras', 'Semi-Reboque', 'Reboque', 'Trator misto',
       'Quadriciclo', 'Carroça', 'Motor-Casa'], dtype=object)

In [ ]:
df_prf15.tracado_via.unique()

array(['Reta', 'Cruzamento', 'Curva', '(null)'], dtype=object)

In [ ]:
df_prf15.tipo_acidente.unique()

array(['Saída de Pista', 'Colisão com objeto fixo', 'Tombamento',
       'Queda de motocicleta / bicicleta / veículo',
       'Atropelamento de animal', 'Capotamento',
       'Atropelamento de pessoa', 'Colisão com objeto móvel', 'Incêndio',
       'Danos Eventuais', 'Derramamento de Carga'], dtype=object)

In [ ]:
# Filtragem 'Reta' ou 'Curva' + 'Capotamento' ou 'Tombamento'
via = df_prf15.tracado_via.isin(['Reta', 'Curva'])
acid = df_prf15.tipo_acidente.isin(['Capotamento', 'Tombamento'])

df15_reta_curva_capot_tomb = df_prf15[(via) & (acid)]

In [ ]:
df15_reta_curva_capot_tomb.tracado_via.unique()

array(['Reta', 'Curva'], dtype=object)

In [ ]:
df15_reta_curva_capot_tomb.tipo_acidente.unique()

array(['Tombamento', 'Capotamento'], dtype=object)

In [ ]:
counts, bins = np.histogram(df15_reta_curva_capot_tomb['Hora'], bins=range(0, 25, 1))
acidentes15 = np.array(counts)/sum(np.array(counts))
fluxo15 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa15 = (acidentes15/fluxo15)/sum(acidentes15/fluxo15)

In [ ]:
graphics_plot_prf(y=[taxa15, fluxo15, acidentes15],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='One vehicle, overturning or rollover on a straight or curve section of the road (2015) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='reta_curva_capot_tomb_15')

**2016**

In [ ]:
df_prf16.tipo_veiculo.unique()

array(['Motocicletas', 'Camioneta', 'Automóvel', 'Caminhão-Trator',
       'Caminhonete', 'Microônibus', 'Caminhão', 'Ônibus', nan,
       'Utilitário', 'Motoneta', 'Ciclomotor', 'Não identificado',
       'Bicicleta', 'Caminhão-Tanque', 'Semi-Reboque', 'Motor-Casa',
       'Triciclo', 'Trator de rodas', 'Carroça', 'Reboque',
       'Trator de esteiras', 'Carro-de-mao', 'Charrete'], dtype=object)

In [ ]:
df_prf16.tracado_via.unique()

array(['Reta', 'Curva', 'Cruzamento'], dtype=object)

In [ ]:
df_prf16.tipo_acidente.unique()

array(['Queda de motocicleta / bicicleta / veículo', 'Capotamento',
       'Saída de Pista', 'Colisão com objeto fixo', 'Incêndio',
       'Tombamento', 'Atropelamento de pessoa',
       'Colisão com objeto móvel', 'Atropelamento de animal',
       'Derramamento de Carga', 'Danos Eventuais'], dtype=object)

In [ ]:
# Filtragem 'Reta' ou 'Curva' + 'Capotamento' ou 'Tombamento'
via = df_prf16.tracado_via.isin(['Reta', 'Curva'])
acid = df_prf16.tipo_acidente.isin(['Capotamento', 'Tombamento'])

df16_reta_curva_capot_tomb = df_prf16[(via) & (acid)]

In [ ]:
df16_reta_curva_capot_tomb.tracado_via.unique()

array(['Reta', 'Curva'], dtype=object)

In [ ]:
df16_reta_curva_capot_tomb.tipo_acidente.unique()

array(['Capotamento', 'Tombamento'], dtype=object)

In [ ]:
counts, bins = np.histogram(df16_reta_curva_capot_tomb['Hora'], bins=range(0, 25, 1))
acidentes16 = np.array(counts)/sum(np.array(counts))
fluxo16 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa16 = (acidentes16/fluxo16)/sum(acidentes16/fluxo16)

In [ ]:
graphics_plot_prf(y=[taxa16, fluxo16, acidentes16],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='One vehicle, overturning or rollover on a straight or curve section of the road (2016) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='reta_curva_capot_tomb_16')

**2017**

In [ ]:
df_prf17.tipo_veiculo.unique()

array(['Caminhão-trator', 'Caminhão', 'Semireboque', 'Reboque',
       'Motocicleta', 'Automóvel', 'Caminhonete', 'Bicicleta',
       'Camioneta', 'Motoneta', 'Micro-ônibus', 'Utilitário', 'Outros',
       'Ônibus', 'Não Informado', 'Trator de rodas', 'Ciclomotor',
       'Trator de esteira', 'Triciclo', 'Trator misto'], dtype=object)

In [ ]:
df_prf17.tracado_via.unique()

array(['Desvio Temporário', 'Não Informado', 'Reta', 'Curva', 'Viaduto',
       'Rotatória', 'Interseção de vias', 'Túnel',
       'Retorno Regulamentado', 'Ponte'], dtype=object)

In [ ]:
df_prf17.tipo_acidente.unique()

array(['Colisão com objeto estático', 'Tombamento',
       'Derramamento de carga', 'Atropelamento de Pedestre',
       'Saída de leito carroçável', 'Incêndio', 'Atropelamento de Animal',
       'Capotamento', 'Colisão com objeto em movimento',
       'Danos eventuais', 'Colisão lateral', 'Colisão traseira',
       'Queda de ocupante de veículo', 'Colisão frontal', 'Engavetamento',
       'Colisão transversal'], dtype=object)

In [ ]:
# Filtragem 'Reta' ou 'Curva' + 'Capotamento' ou 'Tombamento'
via = df_prf17.tracado_via.isin(['Reta', 'Curva'])
acid = df_prf17.tipo_acidente.isin(['Capotamento', 'Tombamento'])

df17_reta_curva_capot_tomb = df_prf17[(via) & (acid)]

In [ ]:
df17_reta_curva_capot_tomb.tracado_via.unique()

array(['Reta', 'Curva'], dtype=object)

In [ ]:
df17_reta_curva_capot_tomb.tipo_acidente.unique()

array(['Tombamento', 'Capotamento'], dtype=object)

In [ ]:
counts, bins = np.histogram(df17_reta_curva_capot_tomb['Hora'], bins=range(0, 25, 1))
acidentes17 = np.array(counts)/sum(np.array(counts))
fluxo17= np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa17 = (acidentes17/fluxo17)/sum(acidentes17/fluxo17)

In [ ]:
graphics_plot_prf(y=[taxa17, fluxo17, acidentes17],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='One vehicle, overturning or rollover on a straight or curve section of the road (2017) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='reta_curva_capot_tomb_17')

## **2. Um único veículo, capotamento em curva e reta**

**2015**

In [ ]:
df_prf15.tipo_veiculo.unique()

array(['Automóvel', 'Caminhão', 'Motocicletas', 'Caminhão-Trator',
       '(null)', 'Camioneta', 'Motoneta', 'Utilitário', 'Caminhonete',
       'Microônibus', 'Não identificado', 'Ônibus', 'Bicicleta',
       'Ciclomotor', 'Trator de rodas', 'Caminhão-Tanque', 'Triciclo',
       'Trator de esteiras', 'Semi-Reboque', 'Reboque', 'Trator misto',
       'Quadriciclo', 'Carroça', 'Motor-Casa'], dtype=object)

In [ ]:
df_prf15.tracado_via.unique()

array(['Reta', 'Cruzamento', 'Curva', '(null)'], dtype=object)

In [ ]:
df_prf15.tipo_acidente.unique()

array(['Saída de Pista', 'Colisão com objeto fixo', 'Tombamento',
       'Queda de motocicleta / bicicleta / veículo',
       'Atropelamento de animal', 'Capotamento',
       'Atropelamento de pessoa', 'Colisão com objeto móvel', 'Incêndio',
       'Danos Eventuais', 'Derramamento de Carga'], dtype=object)

In [ ]:
# Filtragem 'Reta' ou 'Curva' + 'Capotamento'
via = df_prf15.tracado_via.isin(['Reta', 'Curva'])
acid = df_prf15.tipo_acidente.isin(['Capotamento'])

df15_reta_curva_capot = df_prf15[(via) & (acid)]

In [ ]:
df15_reta_curva_capot.tracado_via.unique()

array(['Curva', 'Reta'], dtype=object)

In [ ]:
df15_reta_curva_capot.tipo_acidente.unique()

array(['Capotamento'], dtype=object)

In [ ]:
counts, bins = np.histogram(df15_reta_curva_capot['Hora'], bins=range(0, 25, 1))
acidentes15 = np.array(counts)/sum(np.array(counts))
fluxo15 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa15 = (acidentes15/fluxo15)/sum(acidentes15/fluxo15)

In [ ]:
graphics_plot_prf(y=[taxa15, fluxo15, acidentes15],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='One vehicle, rollover on a straight or curve section of the road (2015) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='reta_curva_capot_15')

**2016**

In [ ]:
df_prf16.tipo_veiculo.unique()

array(['Motocicletas', 'Camioneta', 'Automóvel', 'Caminhão-Trator',
       'Caminhonete', 'Microônibus', 'Caminhão', 'Ônibus', nan,
       'Utilitário', 'Motoneta', 'Ciclomotor', 'Não identificado',
       'Bicicleta', 'Caminhão-Tanque', 'Semi-Reboque', 'Motor-Casa',
       'Triciclo', 'Trator de rodas', 'Carroça', 'Reboque',
       'Trator de esteiras', 'Carro-de-mao', 'Charrete'], dtype=object)

In [ ]:
df_prf16.tracado_via.unique()

array(['Reta', 'Curva', 'Cruzamento'], dtype=object)

In [ ]:
df_prf16.tipo_acidente.unique()

array(['Queda de motocicleta / bicicleta / veículo', 'Capotamento',
       'Saída de Pista', 'Colisão com objeto fixo', 'Incêndio',
       'Tombamento', 'Atropelamento de pessoa',
       'Colisão com objeto móvel', 'Atropelamento de animal',
       'Derramamento de Carga', 'Danos Eventuais'], dtype=object)

In [ ]:
# Filtragem 'Reta' ou 'Curva' + 'Capotamento'
via = df_prf16.tracado_via.isin(['Reta', 'Curva'])
acid = df_prf16.tipo_acidente.isin(['Capotamento'])

df16_reta_curva_capot = df_prf16[(via) & (acid)]

In [ ]:
df16_reta_curva_capot.tracado_via.unique()

array(['Reta', 'Curva'], dtype=object)

In [ ]:
df16_reta_curva_capot.tipo_acidente.unique()

array(['Capotamento'], dtype=object)

In [ ]:
counts, bins = np.histogram(df16_reta_curva_capot['Hora'], bins=range(0, 25, 1))
acidentes16 = np.array(counts)/sum(np.array(counts))
fluxo16 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa16 = (acidentes16/fluxo16)/sum(acidentes16/fluxo16)

In [ ]:
graphics_plot_prf(y=[taxa16, fluxo16, acidentes16],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='One vehicle, rollover on a straight or curve section of the road (2016) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='reta_curva_capot_16')

**2017**

In [ ]:
df_prf17.tipo_veiculo.unique()

array(['Caminhão-trator', 'Caminhão', 'Semireboque', 'Reboque',
       'Motocicleta', 'Automóvel', 'Caminhonete', 'Bicicleta',
       'Camioneta', 'Motoneta', 'Micro-ônibus', 'Utilitário', 'Outros',
       'Ônibus', 'Não Informado', 'Trator de rodas', 'Ciclomotor',
       'Trator de esteira', 'Triciclo', 'Trator misto'], dtype=object)

In [ ]:
df_prf17.tracado_via.unique()

array(['Desvio Temporário', 'Não Informado', 'Reta', 'Curva', 'Viaduto',
       'Rotatória', 'Interseção de vias', 'Túnel',
       'Retorno Regulamentado', 'Ponte'], dtype=object)

In [ ]:
df_prf17.tipo_acidente.unique()

array(['Colisão com objeto estático', 'Tombamento',
       'Derramamento de carga', 'Atropelamento de Pedestre',
       'Saída de leito carroçável', 'Incêndio', 'Atropelamento de Animal',
       'Capotamento', 'Colisão com objeto em movimento',
       'Danos eventuais', 'Colisão lateral', 'Colisão traseira',
       'Queda de ocupante de veículo', 'Colisão frontal', 'Engavetamento',
       'Colisão transversal'], dtype=object)

In [ ]:
# Filtragem 'Reta' ou 'Curva' + 'Capotamento'
via = df_prf17.tracado_via.isin(['Reta', 'Curva'])
acid = df_prf17.tipo_acidente.isin(['Capotamento'])

df17_reta_curva_capot = df_prf17[(via) & (acid)]

In [ ]:
df17_reta_curva_capot.tracado_via.unique()

array(['Curva', 'Reta'], dtype=object)

In [ ]:
df17_reta_curva_capot.tipo_acidente.unique()

array(['Capotamento'], dtype=object)

In [ ]:
counts, bins = np.histogram(df17_reta_curva_capot['Hora'], bins=range(0, 25, 1))
acidentes17 = np.array(counts)/sum(np.array(counts))
fluxo17= np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa17 = (acidentes17/fluxo17)/sum(acidentes17/fluxo17)

In [ ]:
graphics_plot_prf(y=[taxa17, fluxo17, acidentes17],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='One vehicle, rollover on a straight or curve section of the road (2017) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='reta_curva_capot_17')

## **3. Um único veículo, tombamento em curva e reta**

**2015**

In [ ]:
df_prf15.tipo_veiculo.unique()

array(['Automóvel', 'Caminhão', 'Motocicletas', 'Caminhão-Trator',
       '(null)', 'Camioneta', 'Motoneta', 'Utilitário', 'Caminhonete',
       'Microônibus', 'Não identificado', 'Ônibus', 'Bicicleta',
       'Ciclomotor', 'Trator de rodas', 'Caminhão-Tanque', 'Triciclo',
       'Trator de esteiras', 'Semi-Reboque', 'Reboque', 'Trator misto',
       'Quadriciclo', 'Carroça', 'Motor-Casa'], dtype=object)

In [ ]:
df_prf15.tracado_via.unique()

array(['Reta', 'Cruzamento', 'Curva', '(null)'], dtype=object)

In [ ]:
df_prf15.tipo_acidente.unique()

array(['Saída de Pista', 'Colisão com objeto fixo', 'Tombamento',
       'Queda de motocicleta / bicicleta / veículo',
       'Atropelamento de animal', 'Capotamento',
       'Atropelamento de pessoa', 'Colisão com objeto móvel', 'Incêndio',
       'Danos Eventuais', 'Derramamento de Carga'], dtype=object)

In [ ]:
# Filtragem 'Reta' ou 'Curva' + 'Tombamento'
via = df_prf15.tracado_via.isin(['Reta', 'Curva'])
acid = df_prf15.tipo_acidente.isin(['Tombamento'])

df15_reta_curva_tomb = df_prf15[(via) & (acid)]

In [ ]:
df15_reta_curva_tomb.tracado_via.unique()

array(['Reta', 'Curva'], dtype=object)

In [ ]:
df15_reta_curva_tomb.tipo_acidente.unique()

array(['Tombamento'], dtype=object)

In [ ]:
counts, bins = np.histogram(df15_reta_curva_tomb['Hora'], bins=range(0, 25, 1))
acidentes15 = np.array(counts)/sum(np.array(counts))
fluxo15 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa15 = (acidentes15/fluxo15)/sum(acidentes15/fluxo15)

In [ ]:
graphics_plot_prf(y=[taxa15, fluxo15, acidentes15],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='One vehicle, overturning on a straight or curve section of the road (2015) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='reta_curva_tomb_15')

**2016**

In [ ]:
df_prf16.tipo_veiculo.unique()

array(['Motocicletas', 'Camioneta', 'Automóvel', 'Caminhão-Trator',
       'Caminhonete', 'Microônibus', 'Caminhão', 'Ônibus', nan,
       'Utilitário', 'Motoneta', 'Ciclomotor', 'Não identificado',
       'Bicicleta', 'Caminhão-Tanque', 'Semi-Reboque', 'Motor-Casa',
       'Triciclo', 'Trator de rodas', 'Carroça', 'Reboque',
       'Trator de esteiras', 'Carro-de-mao', 'Charrete'], dtype=object)

In [ ]:
df_prf16.tracado_via.unique()

array(['Reta', 'Curva', 'Cruzamento'], dtype=object)

In [ ]:
df_prf16.tipo_acidente.unique()

array(['Queda de motocicleta / bicicleta / veículo', 'Capotamento',
       'Saída de Pista', 'Colisão com objeto fixo', 'Incêndio',
       'Tombamento', 'Atropelamento de pessoa',
       'Colisão com objeto móvel', 'Atropelamento de animal',
       'Derramamento de Carga', 'Danos Eventuais'], dtype=object)

In [ ]:
# Filtragem 'Reta' ou 'Curva' + 'Tombamento'
via = df_prf16.tracado_via.isin(['Reta', 'Curva'])
acid = df_prf16.tipo_acidente.isin(['Tombamento'])

df16_reta_curva_tomb = df_prf16[(via) & (acid)]

In [ ]:
df16_reta_curva_tomb.tracado_via.unique()

array(['Curva', 'Reta'], dtype=object)

In [ ]:
df16_reta_curva_tomb.tipo_acidente.unique()

array(['Tombamento'], dtype=object)

In [ ]:
counts, bins = np.histogram(df16_reta_curva_tomb['Hora'], bins=range(0, 25, 1))
acidentes16 = np.array(counts)/sum(np.array(counts))
fluxo16 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa16 = (acidentes16/fluxo16)/sum(acidentes16/fluxo16)

In [ ]:
graphics_plot_prf(y=[taxa16, fluxo16, acidentes16],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='One vehicle, overturning on a straight or curve section of the road (2016) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='reta_curva_tomb_16')

**2017**

In [ ]:
df_prf17.tipo_veiculo.unique()

array(['Caminhão-trator', 'Caminhão', 'Semireboque', 'Reboque',
       'Motocicleta', 'Automóvel', 'Caminhonete', 'Bicicleta',
       'Camioneta', 'Motoneta', 'Micro-ônibus', 'Utilitário', 'Outros',
       'Ônibus', 'Não Informado', 'Trator de rodas', 'Ciclomotor',
       'Trator de esteira', 'Triciclo', 'Trator misto'], dtype=object)

In [ ]:
df_prf17.tracado_via.unique()

array(['Desvio Temporário', 'Não Informado', 'Reta', 'Curva', 'Viaduto',
       'Rotatória', 'Interseção de vias', 'Túnel',
       'Retorno Regulamentado', 'Ponte'], dtype=object)

In [ ]:
df_prf17.tipo_acidente.unique()

array(['Colisão com objeto estático', 'Tombamento',
       'Derramamento de carga', 'Atropelamento de Pedestre',
       'Saída de leito carroçável', 'Incêndio', 'Atropelamento de Animal',
       'Capotamento', 'Colisão com objeto em movimento',
       'Danos eventuais', 'Colisão lateral', 'Colisão traseira',
       'Queda de ocupante de veículo', 'Colisão frontal', 'Engavetamento',
       'Colisão transversal'], dtype=object)

In [ ]:
# Filtragem 'Reta' ou 'Curva' + 'Tombamento'
via = df_prf17.tracado_via.isin(['Reta', 'Curva'])
acid = df_prf17.tipo_acidente.isin(['Tombamento'])

df17_reta_curva_tomb = df_prf17[(via) & (acid)]

In [ ]:
df17_reta_curva_tomb.tracado_via.unique()

array(['Reta', 'Curva'], dtype=object)

In [ ]:
df17_reta_curva_tomb.tipo_acidente.unique()

array(['Tombamento'], dtype=object)

In [ ]:
counts, bins = np.histogram(df17_reta_curva_tomb['Hora'], bins=range(0, 25, 1))
acidentes17 = np.array(counts)/sum(np.array(counts))
fluxo17= np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa17 = (acidentes17/fluxo17)/sum(acidentes17/fluxo17)

In [ ]:
graphics_plot_prf(y=[taxa17, fluxo17, acidentes17],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='One vehicle, overturning on a straight or curve section of the road (2017) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='reta_curva_tomb_17')

In [ ]:
# # Download images folder
# !zip -r /content/images.zip /content/images

# from google.colab import files
# files.download('/content/images.zip')

# **+ 3 análises (Somente 2015, 2016 e 2017 e caminhões)**

1. Um único veículo, capotamento e tombamento em curva e reta

2. Um único veículo, capotamento em curva e reta

3. Um único veículo, tombamento em curva e reta

## **Dados fluxo de caminhões**

In [ ]:
dfFluxo.columns

Index(['avgoutros', 'stdevcam7e', 'avgpasseio', 'stdevcam8e', 'avgtotal',
       'stdevcam9e', 'avgcam4e', 'stdevmoto', 'avgcam5e', 'direction',
       'stdevonicam2e', 'avgcam6e', 'stdevonicam3e', 'avgcam7e', 'stdevoutros',
       'avgcam8e', 'hour1', 'stdevpasseio', 'avgcam9e', 'stdevtotal',
       'avgmoto', 'stdevcam4e', 'year1', 'avgonicam2e', 'stdevcam5e',
       'avgonicam3e', 'stdevcam6e'],
      dtype='object')

In [ ]:
dfFluxo['avgcam'] = dfFluxo['avgcam4e']+dfFluxo['avgcam5e']+dfFluxo['avgcam6e']+dfFluxo['avgcam7e']+dfFluxo['avgcam8e']+dfFluxo['avgcam9e']
df_cam = pd.DataFrame(dfFluxo['avgcam'].loc[2::3].values.reshape(4,24).T, columns=[2014, 2015, 2016, 2017])

In [ ]:
df_cam.head(3)

,2014,2015,2016,2017
0,16.989094,14.856208,13.056651,12.186218
1,14.203662,12.133953,10.739333,9.903532
2,12.560059,10.760971,9.537287,8.797717


## **1. Caminhões, um único veículo, capotamento e tombamento em curva e reta**

**2015**

In [ ]:
df_prf15.tipo_veiculo.unique()

array(['Automóvel', 'Caminhão', 'Motocicletas', 'Caminhão-Trator',
       '(null)', 'Camioneta', 'Motoneta', 'Utilitário', 'Caminhonete',
       'Microônibus', 'Não identificado', 'Ônibus', 'Bicicleta',
       'Ciclomotor', 'Trator de rodas', 'Caminhão-Tanque', 'Triciclo',
       'Trator de esteiras', 'Semi-Reboque', 'Reboque', 'Trator misto',
       'Quadriciclo', 'Carroça', 'Motor-Casa'], dtype=object)

In [ ]:
df_prf15.tracado_via.unique()

array(['Reta', 'Cruzamento', 'Curva', '(null)'], dtype=object)

In [ ]:
df_prf15.tipo_acidente.unique()

array(['Saída de Pista', 'Colisão com objeto fixo', 'Tombamento',
       'Queda de motocicleta / bicicleta / veículo',
       'Atropelamento de animal', 'Capotamento',
       'Atropelamento de pessoa', 'Colisão com objeto móvel', 'Incêndio',
       'Danos Eventuais', 'Derramamento de Carga'], dtype=object)

In [ ]:
# Filtragem Caminhões em 'Reta' ou 'Curva' + 'Capotamento' ou 'Tombamento'
via = df_prf15.tracado_via.isin(['Reta', 'Curva'])
acid = df_prf15.tipo_acidente.isin(['Capotamento', 'Tombamento'])
cam = df_prf15.tipo_veiculo.isin(['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque'])

df15cam_reta_curva_capot_tomb = df_prf15[(via) & (acid) & (cam)]

In [ ]:
df15cam_reta_curva_capot_tomb.tipo_veiculo.unique()

array(['Caminhão-Trator', 'Caminhão', 'Caminhão-Tanque'], dtype=object)

In [ ]:
df15cam_reta_curva_capot_tomb.tracado_via.unique()

array(['Reta', 'Curva'], dtype=object)

In [ ]:
df15cam_reta_curva_capot_tomb.tipo_acidente.unique()

array(['Tombamento', 'Capotamento'], dtype=object)

In [ ]:
counts, bins = np.histogram(df15cam_reta_curva_capot_tomb['Hora'], bins=range(0, 25, 1))
acidentes15 = np.array(counts)/sum(np.array(counts))
fluxo15 = np.array(df_cam[2015]/sum(df_cam[2015]))

taxa15 = (acidentes15/fluxo15)/sum(acidentes15/fluxo15)

In [ ]:
graphics_plot_prf(y=[taxa15, fluxo15, acidentes15],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks, overturning or rollover on a straight or curve section of the road (2015) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='cam_reta_curva_capot_tomb_15')

**2016**

In [ ]:
df_prf16.tipo_veiculo.unique()

array(['Motocicletas', 'Camioneta', 'Automóvel', 'Caminhão-Trator',
       'Caminhonete', 'Microônibus', 'Caminhão', 'Ônibus', nan,
       'Utilitário', 'Motoneta', 'Ciclomotor', 'Não identificado',
       'Bicicleta', 'Caminhão-Tanque', 'Semi-Reboque', 'Motor-Casa',
       'Triciclo', 'Trator de rodas', 'Carroça', 'Reboque',
       'Trator de esteiras', 'Carro-de-mao', 'Charrete'], dtype=object)

In [ ]:
df_prf16.tracado_via.unique()

array(['Reta', 'Curva', 'Cruzamento'], dtype=object)

In [ ]:
df_prf16.tipo_acidente.unique()

array(['Queda de motocicleta / bicicleta / veículo', 'Capotamento',
       'Saída de Pista', 'Colisão com objeto fixo', 'Incêndio',
       'Tombamento', 'Atropelamento de pessoa',
       'Colisão com objeto móvel', 'Atropelamento de animal',
       'Derramamento de Carga', 'Danos Eventuais'], dtype=object)

In [ ]:
# Filtragem Caminhões em 'Reta' ou 'Curva' + 'Capotamento' ou 'Tombamento'
via = df_prf16.tracado_via.isin(['Reta', 'Curva'])
acid = df_prf16.tipo_acidente.isin(['Capotamento', 'Tombamento'])
cam = df_prf16.tipo_veiculo.isin(['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque'])

df16cam_reta_curva_capot_tomb = df_prf16[(via) & (acid) & (cam)]

In [ ]:
df16cam_reta_curva_capot_tomb.tipo_veiculo.unique()

array(['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque'], dtype=object)

In [ ]:
df16cam_reta_curva_capot_tomb.tracado_via.unique()

array(['Reta', 'Curva'], dtype=object)

In [ ]:
df16cam_reta_curva_capot_tomb.tipo_acidente.unique()

array(['Tombamento', 'Capotamento'], dtype=object)

In [ ]:
counts, bins = np.histogram(df16cam_reta_curva_capot_tomb['Hora'], bins=range(0, 25, 1))
acidentes16 = np.array(counts)/sum(np.array(counts))
fluxo16 = np.array(df_cam[2016]/sum(df_cam[2016]))

taxa16 = (acidentes16/fluxo16)/sum(acidentes16/fluxo16)

In [ ]:
graphics_plot_prf(y=[taxa16, fluxo16, acidentes16],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks, overturning or rollover on a straight or curve section of the road (2016) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='cam_reta_curva_capot_tomb_16')

**2017**

In [ ]:
df_prf17.tipo_veiculo.unique()

array(['Caminhão-trator', 'Caminhão', 'Semireboque', 'Reboque',
       'Motocicleta', 'Automóvel', 'Caminhonete', 'Bicicleta',
       'Camioneta', 'Motoneta', 'Micro-ônibus', 'Utilitário', 'Outros',
       'Ônibus', 'Não Informado', 'Trator de rodas', 'Ciclomotor',
       'Trator de esteira', 'Triciclo', 'Trator misto'], dtype=object)

In [ ]:
df_prf17.tracado_via.unique()

array(['Desvio Temporário', 'Não Informado', 'Reta', 'Curva', 'Viaduto',
       'Rotatória', 'Interseção de vias', 'Túnel',
       'Retorno Regulamentado', 'Ponte'], dtype=object)

In [ ]:
df_prf17.tipo_acidente.unique()

array(['Colisão com objeto estático', 'Tombamento',
       'Derramamento de carga', 'Atropelamento de Pedestre',
       'Saída de leito carroçável', 'Incêndio', 'Atropelamento de Animal',
       'Capotamento', 'Colisão com objeto em movimento',
       'Danos eventuais', 'Colisão lateral', 'Colisão traseira',
       'Queda de ocupante de veículo', 'Colisão frontal', 'Engavetamento',
       'Colisão transversal'], dtype=object)

In [ ]:
# Filtragem Caminhões em 'Reta' ou 'Curva' + 'Capotamento' ou 'Tombamento'
via = df_prf17.tracado_via.isin(['Reta', 'Curva'])
acid = df_prf17.tipo_acidente.isin(['Capotamento', 'Tombamento'])
cam = df_prf17.tipo_veiculo.isin(['Caminhão', 'Caminhão-trator'])

df17cam_reta_curva_capot_tomb = df_prf17[(via) & (acid) & (cam)]

In [ ]:
df17cam_reta_curva_capot_tomb.tipo_veiculo.unique()

array(['Caminhão-trator', 'Caminhão'], dtype=object)

In [ ]:
df17cam_reta_curva_capot_tomb.tracado_via.unique()

array(['Reta', 'Curva'], dtype=object)

In [ ]:
df17cam_reta_curva_capot_tomb.tipo_acidente.unique()

array(['Tombamento', 'Capotamento'], dtype=object)

In [ ]:
counts, bins = np.histogram(df17cam_reta_curva_capot_tomb['Hora'], bins=range(0, 25, 1))
acidentes17 = np.array(counts)/sum(np.array(counts))
fluxo17= np.array(df_cam[2017]/sum(df_cam[2017]))

taxa17 = (acidentes17/fluxo17)/sum(acidentes17/fluxo17)

In [ ]:
graphics_plot_prf(y=[taxa17, fluxo17, acidentes17],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks, overturning or rollover on a straight or curve section of the road (2017) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='cam_reta_curva_capot_tomb_17')

## **2. Caminhões, um único veículo, capotamento em curva e reta**

**2015**

In [ ]:
df_prf15.tipo_veiculo.unique()

array(['Automóvel', 'Caminhão', 'Motocicletas', 'Caminhão-Trator',
       '(null)', 'Camioneta', 'Motoneta', 'Utilitário', 'Caminhonete',
       'Microônibus', 'Não identificado', 'Ônibus', 'Bicicleta',
       'Ciclomotor', 'Trator de rodas', 'Caminhão-Tanque', 'Triciclo',
       'Trator de esteiras', 'Semi-Reboque', 'Reboque', 'Trator misto',
       'Quadriciclo', 'Carroça', 'Motor-Casa'], dtype=object)

In [ ]:
df_prf15.tracado_via.unique()

array(['Reta', 'Cruzamento', 'Curva', '(null)'], dtype=object)

In [ ]:
df_prf15.tipo_acidente.unique()

array(['Saída de Pista', 'Colisão com objeto fixo', 'Tombamento',
       'Queda de motocicleta / bicicleta / veículo',
       'Atropelamento de animal', 'Capotamento',
       'Atropelamento de pessoa', 'Colisão com objeto móvel', 'Incêndio',
       'Danos Eventuais', 'Derramamento de Carga'], dtype=object)

In [ ]:
# Filtragem Caminhões em 'Reta' ou 'Curva' + 'Capotamento'
via = df_prf15.tracado_via.isin(['Reta', 'Curva'])
acid = df_prf15.tipo_acidente.isin(['Capotamento'])
cam = df_prf15.tipo_veiculo.isin(['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque'])

df15cam_reta_curva_capot = df_prf15[(via) & (acid) & (cam)]

In [ ]:
df15cam_reta_curva_capot.tipo_veiculo.unique()

array(['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque'], dtype=object)

In [ ]:
df15cam_reta_curva_capot.tracado_via.unique()

array(['Reta', 'Curva'], dtype=object)

In [ ]:
df15cam_reta_curva_capot.tipo_acidente.unique()

array(['Capotamento'], dtype=object)

In [ ]:
counts, bins = np.histogram(df15cam_reta_curva_capot['Hora'], bins=range(0, 25, 1))
acidentes15 = np.array(counts)/sum(np.array(counts))
fluxo15 = np.array(df_cam[2015]/sum(df_cam[2015]))

taxa15 = (acidentes15/fluxo15)/sum(acidentes15/fluxo15)

In [ ]:
graphics_plot_prf(y=[taxa15, fluxo15, acidentes15],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks, rollover on a straight or curve section of the road (2015) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='cam_reta_curva_capot_15')

**2016**

In [ ]:
df_prf16.tipo_veiculo.unique()

array(['Motocicletas', 'Camioneta', 'Automóvel', 'Caminhão-Trator',
       'Caminhonete', 'Microônibus', 'Caminhão', 'Ônibus', nan,
       'Utilitário', 'Motoneta', 'Ciclomotor', 'Não identificado',
       'Bicicleta', 'Caminhão-Tanque', 'Semi-Reboque', 'Motor-Casa',
       'Triciclo', 'Trator de rodas', 'Carroça', 'Reboque',
       'Trator de esteiras', 'Carro-de-mao', 'Charrete'], dtype=object)

In [ ]:
df_prf16.tracado_via.unique()

array(['Reta', 'Curva', 'Cruzamento'], dtype=object)

In [ ]:
df_prf16.tipo_acidente.unique()

array(['Queda de motocicleta / bicicleta / veículo', 'Capotamento',
       'Saída de Pista', 'Colisão com objeto fixo', 'Incêndio',
       'Tombamento', 'Atropelamento de pessoa',
       'Colisão com objeto móvel', 'Atropelamento de animal',
       'Derramamento de Carga', 'Danos Eventuais'], dtype=object)

In [ ]:
# Filtragem Caminhões em 'Reta' ou 'Curva' + 'Capotamento'
via = df_prf16.tracado_via.isin(['Reta', 'Curva'])
acid = df_prf16.tipo_acidente.isin(['Capotamento'])
cam = df_prf16.tipo_veiculo.isin(['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque'])

df16cam_reta_curva_capot = df_prf16[(via) & (acid) & (cam)]

In [ ]:
df16cam_reta_curva_capot.tipo_veiculo.unique()

array(['Caminhão', 'Caminhão-Trator'], dtype=object)

In [ ]:
df16cam_reta_curva_capot.tracado_via.unique()

array(['Curva', 'Reta'], dtype=object)

In [ ]:
df16cam_reta_curva_capot.tipo_acidente.unique()

array(['Capotamento'], dtype=object)

In [ ]:
counts, bins = np.histogram(df16cam_reta_curva_capot['Hora'], bins=range(0, 25, 1))
acidentes16 = np.array(counts)/sum(np.array(counts))
fluxo16 = np.array(df_cam[2016]/sum(df_cam[2016]))

taxa16 = (acidentes16/fluxo16)/sum(acidentes16/fluxo16)

In [ ]:
graphics_plot_prf(y=[taxa16, fluxo16, acidentes16],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks, rollover on a straight or curve section of the road (2016) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='cam_reta_curva_capot_16')

**2017**

In [ ]:
df_prf17.tipo_veiculo.unique()

array(['Caminhão-trator', 'Caminhão', 'Semireboque', 'Reboque',
       'Motocicleta', 'Automóvel', 'Caminhonete', 'Bicicleta',
       'Camioneta', 'Motoneta', 'Micro-ônibus', 'Utilitário', 'Outros',
       'Ônibus', 'Não Informado', 'Trator de rodas', 'Ciclomotor',
       'Trator de esteira', 'Triciclo', 'Trator misto'], dtype=object)

In [ ]:
df_prf17.tracado_via.unique()

array(['Desvio Temporário', 'Não Informado', 'Reta', 'Curva', 'Viaduto',
       'Rotatória', 'Interseção de vias', 'Túnel',
       'Retorno Regulamentado', 'Ponte'], dtype=object)

In [ ]:
df_prf17.tipo_acidente.unique()

array(['Colisão com objeto estático', 'Tombamento',
       'Derramamento de carga', 'Atropelamento de Pedestre',
       'Saída de leito carroçável', 'Incêndio', 'Atropelamento de Animal',
       'Capotamento', 'Colisão com objeto em movimento',
       'Danos eventuais', 'Colisão lateral', 'Colisão traseira',
       'Queda de ocupante de veículo', 'Colisão frontal', 'Engavetamento',
       'Colisão transversal'], dtype=object)

In [ ]:
# Filtragem Caminhões em 'Reta' ou 'Curva' + 'Capotamento'
via = df_prf17.tracado_via.isin(['Reta', 'Curva'])
acid = df_prf17.tipo_acidente.isin(['Capotamento'])
cam = df_prf17.tipo_veiculo.isin(['Caminhão', 'Caminhão-trator'])

df17cam_reta_curva_capot = df_prf17[(via) & (acid) & (cam)]

In [ ]:
df17cam_reta_curva_capot.tipo_veiculo.unique()

array(['Caminhão-trator', 'Caminhão'], dtype=object)

In [ ]:
df17cam_reta_curva_capot.tracado_via.unique()

array(['Curva', 'Reta'], dtype=object)

In [ ]:
df17cam_reta_curva_capot.tipo_acidente.unique()

array(['Capotamento'], dtype=object)

In [ ]:
counts, bins = np.histogram(df17cam_reta_curva_capot['Hora'], bins=range(0, 25, 1))
acidentes17 = np.array(counts)/sum(np.array(counts))
fluxo17= np.array(df_cam[2017]/sum(df_cam[2017]))

taxa17 = (acidentes17/fluxo17)/sum(acidentes17/fluxo17)

In [ ]:
graphics_plot_prf(y=[taxa17, fluxo17, acidentes17],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks, rollover on a straight or curve section of the road (2017) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='cam_reta_curva_capot_17')

## **3. Caminhões, um único veículo, tombamento em curva e reta**

**2015**

In [ ]:
df_prf15.tipo_veiculo.unique()

array(['Automóvel', 'Caminhão', 'Motocicletas', 'Caminhão-Trator',
       '(null)', 'Camioneta', 'Motoneta', 'Utilitário', 'Caminhonete',
       'Microônibus', 'Não identificado', 'Ônibus', 'Bicicleta',
       'Ciclomotor', 'Trator de rodas', 'Caminhão-Tanque', 'Triciclo',
       'Trator de esteiras', 'Semi-Reboque', 'Reboque', 'Trator misto',
       'Quadriciclo', 'Carroça', 'Motor-Casa'], dtype=object)

In [ ]:
df_prf15.tracado_via.unique()

array(['Reta', 'Cruzamento', 'Curva', '(null)'], dtype=object)

In [ ]:
df_prf15.tipo_acidente.unique()

array(['Saída de Pista', 'Colisão com objeto fixo', 'Tombamento',
       'Queda de motocicleta / bicicleta / veículo',
       'Atropelamento de animal', 'Capotamento',
       'Atropelamento de pessoa', 'Colisão com objeto móvel', 'Incêndio',
       'Danos Eventuais', 'Derramamento de Carga'], dtype=object)

In [ ]:
# Filtragem Caminhões em 'Reta' ou 'Curva' + 'Tombamento'
via = df_prf15.tracado_via.isin(['Reta', 'Curva'])
acid = df_prf15.tipo_acidente.isin(['Tombamento'])
cam = df_prf15.tipo_veiculo.isin(['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque'])

df15cam_reta_curva_tomb = df_prf15[(via) & (acid) & (cam)]

In [ ]:
df15cam_reta_curva_tomb.tipo_veiculo.unique()

array(['Caminhão-Trator', 'Caminhão', 'Caminhão-Tanque'], dtype=object)

In [ ]:
df15cam_reta_curva_tomb.tracado_via.unique()

array(['Reta', 'Curva'], dtype=object)

In [ ]:
df15cam_reta_curva_tomb.tipo_acidente.unique()

array(['Tombamento'], dtype=object)

In [ ]:
counts, bins = np.histogram(df15cam_reta_curva_tomb['Hora'], bins=range(0, 25, 1))
acidentes15 = np.array(counts)/sum(np.array(counts))
fluxo15 = np.array(df_cam[2015]/sum(df_cam[2015]))

taxa15 = (acidentes15/fluxo15)/sum(acidentes15/fluxo15)

In [ ]:
graphics_plot_prf(y=[taxa15, fluxo15, acidentes15],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks, overturning on a straight or curve section of the road (2015) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='cam_reta_curva_tomb_15')

**2016**

In [ ]:
df_prf16.tipo_veiculo.unique()

array(['Motocicletas', 'Camioneta', 'Automóvel', 'Caminhão-Trator',
       'Caminhonete', 'Microônibus', 'Caminhão', 'Ônibus', nan,
       'Utilitário', 'Motoneta', 'Ciclomotor', 'Não identificado',
       'Bicicleta', 'Caminhão-Tanque', 'Semi-Reboque', 'Motor-Casa',
       'Triciclo', 'Trator de rodas', 'Carroça', 'Reboque',
       'Trator de esteiras', 'Carro-de-mao', 'Charrete'], dtype=object)

In [ ]:
df_prf16.tracado_via.unique()

array(['Reta', 'Curva', 'Cruzamento'], dtype=object)

In [ ]:
df_prf16.tipo_acidente.unique()

array(['Queda de motocicleta / bicicleta / veículo', 'Capotamento',
       'Saída de Pista', 'Colisão com objeto fixo', 'Incêndio',
       'Tombamento', 'Atropelamento de pessoa',
       'Colisão com objeto móvel', 'Atropelamento de animal',
       'Derramamento de Carga', 'Danos Eventuais'], dtype=object)

In [ ]:
# Filtragem Caminhões em 'Reta' ou 'Curva' + 'Tombamento'
via = df_prf16.tracado_via.isin(['Reta', 'Curva'])
acid = df_prf16.tipo_acidente.isin(['Tombamento'])
cam = df_prf16.tipo_veiculo.isin(['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque'])

df16cam_reta_curva_tomb = df_prf16[(via) & (acid) & (cam)]

In [ ]:
df16cam_reta_curva_tomb.tipo_veiculo.unique()

array(['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque'], dtype=object)

In [ ]:
df16cam_reta_curva_tomb.tracado_via.unique()

array(['Reta', 'Curva'], dtype=object)

In [ ]:
df16cam_reta_curva_tomb.tipo_acidente.unique()

array(['Tombamento'], dtype=object)

In [ ]:
counts, bins = np.histogram(df16cam_reta_curva_tomb['Hora'], bins=range(0, 25, 1))
acidentes16 = np.array(counts)/sum(np.array(counts))
fluxo16 = np.array(df_cam[2016]/sum(df_cam[2016]))

taxa16 = (acidentes16/fluxo16)/sum(acidentes16/fluxo16)

In [ ]:
graphics_plot_prf(y=[taxa16, fluxo16, acidentes16],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks, overturning on a straight or curve section of the road (2016) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='cam_reta_curva_tomb_16')

**2017**

In [ ]:
df_prf17.tipo_veiculo.unique()

array(['Caminhão-trator', 'Caminhão', 'Semireboque', 'Reboque',
       'Motocicleta', 'Automóvel', 'Caminhonete', 'Bicicleta',
       'Camioneta', 'Motoneta', 'Micro-ônibus', 'Utilitário', 'Outros',
       'Ônibus', 'Não Informado', 'Trator de rodas', 'Ciclomotor',
       'Trator de esteira', 'Triciclo', 'Trator misto'], dtype=object)

In [ ]:
df_prf17.tracado_via.unique()

array(['Desvio Temporário', 'Não Informado', 'Reta', 'Curva', 'Viaduto',
       'Rotatória', 'Interseção de vias', 'Túnel',
       'Retorno Regulamentado', 'Ponte'], dtype=object)

In [ ]:
df_prf17.tipo_acidente.unique()

array(['Colisão com objeto estático', 'Tombamento',
       'Derramamento de carga', 'Atropelamento de Pedestre',
       'Saída de leito carroçável', 'Incêndio', 'Atropelamento de Animal',
       'Capotamento', 'Colisão com objeto em movimento',
       'Danos eventuais', 'Colisão lateral', 'Colisão traseira',
       'Queda de ocupante de veículo', 'Colisão frontal', 'Engavetamento',
       'Colisão transversal'], dtype=object)

In [ ]:
# Filtragem Caminhões em 'Reta' ou 'Curva' + 'Tombamento'
via = df_prf17.tracado_via.isin(['Reta', 'Curva'])
acid = df_prf17.tipo_acidente.isin(['Tombamento'])
cam = df_prf17.tipo_veiculo.isin(['Caminhão', 'Caminhão-trator'])

df17cam_reta_curva_tomb = df_prf17[(via) & (acid) & (cam)]

In [ ]:
df17cam_reta_curva_tomb.tipo_veiculo.unique()

array(['Caminhão-trator', 'Caminhão'], dtype=object)

In [ ]:
df17cam_reta_curva_tomb.tracado_via.unique()

array(['Reta', 'Curva'], dtype=object)

In [ ]:
df17cam_reta_curva_tomb.tipo_acidente.unique()

array(['Tombamento'], dtype=object)

In [ ]:
counts, bins = np.histogram(df17cam_reta_curva_tomb['Hora'], bins=range(0, 25, 1))
acidentes17 = np.array(counts)/sum(np.array(counts))
fluxo17= np.array(df_cam[2017]/sum(df_cam[2017]))

taxa17 = (acidentes17/fluxo17)/sum(acidentes17/fluxo17)

In [ ]:
graphics_plot_prf(y=[taxa17, fluxo17, acidentes17],
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'],
                  title='Trucks, overturning on a straight or curve section of the road (2017) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='cam_reta_curva_tomb_17')

In [ ]:
# Download images folder
!zip -r /content/images.zip /content/images

from google.colab import files
files.download('/content/images.zip')

# **R² e p-value**

## **Fluxo 2015, 2016 e 2017**

In [ ]:
%%capture
!gdown 1Kru8e3M6AjhEDuVXH7Tqx36wZjz4-Xc0

In [ ]:
# Fluxo
dfFluxo = pd.read_csv('DNIT.csv')

# dfFluxo['avgtotal'].loc[2::3]   ## [start:stop:step] - se ficar livre vai até o final
dfDNIT = pd.DataFrame(dfFluxo['avgtotal'].loc[2::3].values.reshape(4,24).T,
                      columns=[2014, 2015, 2016, 2017])
dfDNIT.tail(3)

,2014,2015,2016,2017
21,217.674901,172.611702,164.695324,160.408976
22,183.319164,143.476845,136.172760,132.808623
23,131.100690,104.440947,97.717366,94.281760


In [ ]:
# 2015 e 2016
show_R2_pvalue(dfDNIT[2015], dfDNIT[2016])

R²: 0.99979
p-value: 6.81040e-42


In [ ]:
# 2015 e 2017
show_R2_pvalue(dfDNIT[2015], dfDNIT[2017])

R²: 0.99983
p-value: 7.92491e-43


In [ ]:
# 2016 e 2017
show_R2_pvalue(dfDNIT[2016], dfDNIT[2017])

R²: 0.99954
p-value: 2.94595e-38


## **Acidentes totais em 2015, 2016 e 2017**

In [ ]:
%%capture
!gdown 13f3pn98iHub-p5hN1Hzho49Xo9UHp96N # 2015
!gdown 13Zc1iIdYWnQ2oQSn8o6CcxZHwrtn6jcu # 2016
!gdown 143SYBlyOq2CMO1ySbIvDFkMn9PW8Vi7d # 2017

In [ ]:
# Leitura dos dados
df15 = pd.read_csv('datatran2015.csv', encoding = 'ISO-8859-1', sep=';')
df15['Hora'] = pd.to_datetime(df15['horario'], format='%H:%M:%S').dt.hour

df16 = pd.read_csv('datatran2016.csv', encoding = 'ISO-8859-1', sep=';')
df16['Hora'] = pd.to_datetime(df16['horario'], format='%H:%M:%S').dt.hour

df17 = pd.read_csv('datatran2017.csv', encoding = 'ISO-8859-1', sep=';')
df17['Hora'] = pd.to_datetime(df17['horario'], format='%H:%M:%S').dt.hour

**Acidentes em 2015, 2016 e 2017**

In [ ]:
# 2015
counts, bins = np.histogram(df15['Hora'], bins=range(0, 25, 1))
acidentes15 = np.array(counts)/sum(np.array(counts))

# 2016
counts, bins = np.histogram(df16['Hora'], bins=range(0, 25, 1))
acidentes16 = np.array(counts)/sum(np.array(counts))

# 2017
counts, bins = np.histogram(df17['Hora'], bins=range(0, 25, 1))
acidentes17 = np.array(counts)/sum(np.array(counts))

In [ ]:
# 2015 e 2016
show_R2_pvalue(acidentes15, acidentes16)

R²: 0.98884
p-value: 5.64258e-23


In [ ]:
# 2015 e 2017
show_R2_pvalue(acidentes15, acidentes17)

R²: 0.98548
p-value: 1.02140e-21


In [ ]:
# 2016 e 2017
show_R2_pvalue(acidentes16, acidentes17)

R²: 0.99611
p-value: 5.24924e-28


## **1. Todos os veículos, em linha reta, um único veículo, tombamento**

In [ ]:
%%capture

# Fluxo
!gdown 1Kru8e3M6AjhEDuVXH7Tqx36wZjz4-Xc0

dfFluxo = pd.read_csv('DNIT.csv')
dfDNIT = pd.DataFrame(dfFluxo['avgtotal'].loc[2::3].values.reshape(4,24)).T

In [ ]:
%%capture
# !gdown 1rLrFckRbr_oScvziSCodjC2_UxoEOAMk # PRF 2013 - 1 veículo
# !gdown 1--JAT8gfd9CBtIlNBBj5yB_C5ipq9p0u # PRF 2014 - 1 veículo
!gdown 1-4vkJOgGW9W__P9zFhbVQkT3Uarwmbbv # PRF 2015 - 1 veículo
!gdown 1-6cj1H9JsUtdIZtO78x3HhdSFjpsqPj_ # PRF 2016 - 1 veículo
!gdown 1-6tc2J-omr-39k9n3FWApTqnJrZFIbnp # PRF 2017 - 1 veículo

# Datasets já filtrados para veículo único
# df_prf13 = pd.read_csv('prf_2013_1veiculo.csv')
# df_prf13['Hora'] = pd.to_datetime(df_prf13['horario'], format='%H:%M:%S').dt.hour
# df_prf14 = pd.read_csv('prf_2014_1veiculo.csv')
# df_prf14['Hora'] = pd.to_datetime(df_prf14['horario'], format='%H:%M:%S').dt.hour
df_prf15 = pd.read_csv('prf_2015_1veiculo.csv')
df_prf15['Hora'] = pd.to_datetime(df_prf15['horario'], format='%H:%M:%S').dt.hour
df_prf16 = pd.read_csv('prf_2016_1veiculo.csv')
df_prf16['Hora'] = pd.to_datetime(df_prf16['horario'], format='%H:%M:%S').dt.hour
df_prf17 = pd.read_csv('prf_2017_1veiculo.csv')
df_prf17['Hora'] = pd.to_datetime(df_prf17['horario'], format='%H:%M:%S').dt.hour

**2015**

In [ ]:
# Filtragem 'Reta' + 'Tombamento'
via = df_prf15.tracado_via == 'Reta'
acid = df_prf15.tipo_acidente == 'Tombamento'

df15_reta_tomb = df_prf15[(via) & (acid)]

In [ ]:
df15_reta_tomb.tracado_via.unique()

array(['Reta'], dtype=object)

In [ ]:
df15_reta_tomb.tipo_acidente.unique()

array(['Tombamento'], dtype=object)

In [ ]:
counts, bins = np.histogram(df15_reta_tomb['Hora'], bins=range(0, 25, 1))
acidentes15 = np.array(counts)/sum(np.array(counts))
fluxo15 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa15 = (acidentes15/fluxo15)/sum(acidentes15/fluxo15)

**2016**

In [ ]:
# Filtragem 'Reta' + 'Tombamento'
via = df_prf16.tracado_via == 'Reta'
acid = df_prf16.tipo_acidente == 'Tombamento'

df16_reta_tomb = df_prf16[(via) & (acid)]

In [ ]:
df16_reta_tomb.tracado_via.unique()

array(['Reta'], dtype=object)

In [ ]:
df16_reta_tomb.tipo_acidente.unique()

array(['Tombamento'], dtype=object)

In [ ]:
counts, bins = np.histogram(df16_reta_tomb['Hora'], bins=range(0, 25, 1))
acidentes16 = np.array(counts)/sum(np.array(counts))
fluxo16 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa16 = (acidentes16/fluxo16)/sum(acidentes16/fluxo16)

**2017**

In [ ]:
# Filtragem 'Reta' + 'Tombamento'
via = df_prf17.tracado_via == 'Reta'
acid = df_prf17.tipo_acidente == 'Tombamento'

df17_reta_tomb = df_prf17[(via) & (acid)]

In [ ]:
df17_reta_tomb.tracado_via.unique()

array(['Reta'], dtype=object)

In [ ]:
df17_reta_tomb.tipo_acidente.unique()

array(['Tombamento'], dtype=object)

In [ ]:
counts, bins = np.histogram(df17_reta_tomb['Hora'], bins=range(0, 25, 1))
acidentes17 = np.array(counts)/sum(np.array(counts))
fluxo17= np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa17 = (acidentes17/fluxo17)/sum(acidentes17/fluxo17)

**R² e p-value**

In [ ]:
# 2015 e 2016
show_R2_pvalue(taxa15, taxa16)

R²: 0.93446
p-value: 1.66362e-14


In [ ]:
# 2015 e 2017
show_R2_pvalue(taxa15, taxa17)

R²: 0.96293
p-value: 3.11212e-17


In [ ]:
# 2016 e 2017
show_R2_pvalue(taxa16, taxa17)

R²: 0.90672
p-value: 8.18036e-13


## **2. Todos os veículos, em curva, um único veículo, tombamento**

**2015**

In [ ]:
# Filtragem 'Curva' + 'Tombamento'
via = df_prf15.tracado_via == 'Curva'
acid = df_prf15.tipo_acidente == 'Tombamento'

df15_curva_tomb = df_prf15[(via) & (acid)]

In [ ]:
df15_curva_tomb.tracado_via.unique()

array(['Curva'], dtype=object)

In [ ]:
df15_curva_tomb.tipo_acidente.unique()

array(['Tombamento'], dtype=object)

In [ ]:
counts, bins = np.histogram(df15_curva_tomb['Hora'], bins=range(0, 25, 1))
acidentes15 = np.array(counts)/sum(np.array(counts))
fluxo15 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa15 = (acidentes15/fluxo15)/sum(acidentes15/fluxo15)

**2016**

In [ ]:
# Filtragem 'Curva' + 'Tombamento'
via = df_prf16.tracado_via == 'Curva'
acid = df_prf16.tipo_acidente == 'Tombamento'

df16_curva_tomb = df_prf16[(via) & (acid)]

In [ ]:
df16_curva_tomb.tracado_via.unique()

array(['Curva'], dtype=object)

In [ ]:
df16_curva_tomb.tipo_acidente.unique()

array(['Tombamento'], dtype=object)

In [ ]:
counts, bins = np.histogram(df16_curva_tomb['Hora'], bins=range(0, 25, 1))
acidentes16 = np.array(counts)/sum(np.array(counts))
fluxo16 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa16 = (acidentes16/fluxo16)/sum(acidentes16/fluxo16)

**2017**

In [ ]:
# Filtragem 'Curva' + 'Tombamento'
via = df_prf17.tracado_via == 'Curva'
acid = df_prf17.tipo_acidente == 'Tombamento'

df17_curva_tomb = df_prf17[(via) & (acid)]

In [ ]:
df17_curva_tomb.tracado_via.unique()

array(['Curva'], dtype=object)

In [ ]:
df17_curva_tomb.tipo_acidente.unique()

array(['Tombamento'], dtype=object)

In [ ]:
counts, bins = np.histogram(df17_curva_tomb['Hora'], bins=range(0, 25, 1))
acidentes17 = np.array(counts)/sum(np.array(counts))
fluxo17= np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa17 = (acidentes17/fluxo17)/sum(acidentes17/fluxo17)

**R² e p-value**

In [ ]:
# 2015 e 2016
show_R2_pvalue(taxa15, taxa16)

R²: 0.89373
p-value: 3.45694e-12


In [ ]:
# 2015 e 2017
show_R2_pvalue(taxa15, taxa17)

R²: 0.74807
p-value: 4.97686e-08


In [ ]:
# 2016 e 2017
show_R2_pvalue(taxa16, taxa17)

R²: 0.88289
p-value: 1.01115e-11


## **3. Todos os veículos, em linha reta, um único veículo, capotamento**

**2015**

In [ ]:
# Filtragem 'Reta' + 'Capotamento'
via = df_prf15.tracado_via == 'Reta'
acid = df_prf15.tipo_acidente == 'Capotamento'

df15_reta_capot = df_prf15[(via) & (acid)]

In [ ]:
df15_reta_capot.tracado_via.unique()

array(['Reta'], dtype=object)

In [ ]:
df15_reta_capot.tipo_acidente.unique()

array(['Capotamento'], dtype=object)

In [ ]:
counts, bins = np.histogram(df15_reta_capot['Hora'], bins=range(0, 25, 1))
acidentes15 = np.array(counts)/sum(np.array(counts))
fluxo15 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa15 = (acidentes15/fluxo15)/sum(acidentes15/fluxo15)

**2016**

In [ ]:
# Filtragem 'Reta' + 'Capotamento'
via = df_prf16.tracado_via == 'Reta'
acid = df_prf16.tipo_acidente == 'Capotamento'

df16_reta_capot = df_prf16[(via) & (acid)]

In [ ]:
df16_reta_capot.tracado_via.unique()

array(['Reta'], dtype=object)

In [ ]:
df16_reta_capot.tipo_acidente.unique()

array(['Capotamento'], dtype=object)

In [ ]:
counts, bins = np.histogram(df16_reta_capot['Hora'], bins=range(0, 25, 1))
acidentes16 = np.array(counts)/sum(np.array(counts))
fluxo16 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa16 = (acidentes16/fluxo16)/sum(acidentes16/fluxo16)

**2017**

In [ ]:
# Filtragem 'Reta' + 'Capotamento'
via = df_prf17.tracado_via == 'Reta'
acid = df_prf17.tipo_acidente == 'Capotamento'

df17_reta_capot = df_prf17[(via) & (acid)]

In [ ]:
df17_reta_capot.tracado_via.unique()

array(['Reta'], dtype=object)

In [ ]:
df17_reta_capot.tipo_acidente.unique()

array(['Capotamento'], dtype=object)

In [ ]:
counts, bins = np.histogram(df17_reta_capot['Hora'], bins=range(0, 25, 1))
acidentes17 = np.array(counts)/sum(np.array(counts))
fluxo17= np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa17 = (acidentes17/fluxo17)/sum(acidentes17/fluxo17)

**R² e p-value**

In [ ]:
# 2015 e 2016
show_R2_pvalue(taxa15, taxa16)

R²: 0.94098
p-value: 5.23254e-15


In [ ]:
# 2015 e 2017
show_R2_pvalue(taxa15, taxa17)

R²: 0.89564
p-value: 2.82827e-12


In [ ]:
# 2016 e 2017
show_R2_pvalue(taxa16, taxa17)

R²: 0.94350
p-value: 3.23640e-15


## **4. Todos os veículos, em curva, um único veículo, capotamento**

**2015**

In [ ]:
# Filtragem 'Curva' + 'Capotamento'
via = df_prf15.tracado_via == 'Curva'
acid = df_prf15.tipo_acidente == 'Capotamento'

df15_curva_capot = df_prf15[(via) & (acid)]

In [ ]:
df15_curva_capot.tracado_via.unique()

array(['Curva'], dtype=object)

In [ ]:
df15_curva_capot.tipo_acidente.unique()

array(['Capotamento'], dtype=object)

In [ ]:
counts, bins = np.histogram(df15_curva_capot['Hora'], bins=range(0, 25, 1))
acidentes15 = np.array(counts)/sum(np.array(counts))
fluxo15 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa15 = (acidentes15/fluxo15)/sum(acidentes15/fluxo15)

**2016**

In [ ]:
# Filtragem 'Curva' + 'Capotamento'
via = df_prf16.tracado_via == 'Curva'
acid = df_prf16.tipo_acidente == 'Capotamento'

df16_curva_capot = df_prf16[(via) & (acid)]

In [ ]:
df16_curva_capot.tracado_via.unique()

array(['Curva'], dtype=object)

In [ ]:
df16_curva_capot.tipo_acidente.unique()

array(['Capotamento'], dtype=object)

In [ ]:
counts, bins = np.histogram(df16_curva_capot['Hora'], bins=range(0, 25, 1))
acidentes16 = np.array(counts)/sum(np.array(counts))
fluxo16 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa16 = (acidentes16/fluxo16)/sum(acidentes16/fluxo16)

**2017**

In [ ]:
# Filtragem 'Curva' + 'Capotamento'
via = df_prf17.tracado_via == 'Curva'
acid = df_prf17.tipo_acidente == 'Capotamento'

df17_curva_capot = df_prf17[(via) & (acid)]

In [ ]:
df17_curva_capot.tracado_via.unique()

array(['Curva'], dtype=object)

In [ ]:
df17_curva_capot.tipo_acidente.unique()

array(['Capotamento'], dtype=object)

In [ ]:
counts, bins = np.histogram(df17_curva_capot['Hora'], bins=range(0, 25, 1))
acidentes17 = np.array(counts)/sum(np.array(counts))
fluxo17= np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa17 = (acidentes17/fluxo17)/sum(acidentes17/fluxo17)

**R² e p-value**

In [ ]:
# 2015 e 2016
show_R2_pvalue(taxa15, taxa16)

R²: 0.95909
p-value: 9.21417e-17


In [ ]:
# 2015 e 2017
show_R2_pvalue(taxa15, taxa17)

R²: 0.90242
p-value: 1.34586e-12


In [ ]:
# 2016 e 2017
show_R2_pvalue(taxa16, taxa17)

R²: 0.92035
p-value: 1.42979e-13
